# 0_数据库连接

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

from collections import Counter
from numpy import nan as NA
from dateutil.parser import parse
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [4]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [5]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="huangxq",
          password="f_epz6hDmZ0Hzi0OzI0N",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 1_数据准备

## 1_11.获取_租后数据

In [6]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,tod.product_name, om.order_type, cc.channel_type_id
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
;
'''

df_zhys = query(sql)
# (27787, 62)
df_zhys.shape

(39994, 65)

## 1_12. 剔除商家数据

In [7]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
   

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(37613, 65)

## 1_13.历史订单渠道名称补充

In [8]:
# 1148 条历史订单渠道名称补充
qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

# qudao_name_df.shape
# qudao_name_df[:2]
# qudao_name_df['渠道名称'].value_counts()

# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
dfzh1["channel_name"].isnull().sum()

# dfzh1[['order_number','channel_name','渠道名称']][:10]
# df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]

np.int64(0)

## 1_14.渠道归属

In [9]:
# df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x: clean.qudao_type(x["channel_name"],x["activity_source"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

# pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

## 1_15.判断颜色 内存

In [10]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
# dfzh1["颜色"].value_counts()[1:10]

## 1_16.判断二手、带锁

In [11]:
# dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
# note 20241124 修改口径，用product_name判断是否二手
dfzh1['是否二手'] = np.where(dfzh1['product_name'].str.contains('95新')|dfzh1['product_name'].str.contains('99新')|dfzh1['product_name'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    36543
二手     1070
Name: count, dtype: int64

In [12]:
#读取导入最新出库台账数据,2023.9及以后的数据
# （1）512 带锁
# （2）非512的多头带锁
# （3）抖音带锁
# note 20241204
# （1）512 带锁，但每天豁免一台 512P 不带锁，目前还没量
# （2）非512的命中多头提示带锁，但出库是否带锁由人工控制
# （3）抖音带锁（苹果11-15）
# 维客壹佰2023&2024年台账1204.xlsz 为截止到20241203的数据
path = 'F:/myfile/p站数据/台账数据/'
# f_path_ck = "维客壹佰2023&2024年台账1204.xlsx"
f_path_ck = "维客壹佰2023&2024年台账.xlsx"
dfck = pd.read_excel(path + f_path_ck,sheet_name="2023")   #,skiprows=0 header 
# dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存']=='512G')|(dfck['内存']=='512GB'), 1, 0)
dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存'].str.contains(r'512')), '带锁', '未带锁')
dfck = dfck[[u'订单号',u'是否带锁']]
dfck.shape
dfck['是否带锁'].value_counts()

dfck.drop_duplicates(subset=["订单号"],inplace=True)

# 合并到主表中去
# dfck 的带锁不一定出库，dfzh1 是出库的
dfck = dfck.rename(columns= {'订单号': 'order_number'})
dfzh1 = dfzh1.merge(dfck, on = 'order_number', how = 'left')
dfzh1['是否带锁'] = dfzh1['是否带锁'].fillna('未带锁') 
dfzh1.shape

pd.pivot_table(dfzh1,index='是否二手',columns='是否带锁',values='order_id',aggfunc='count')

(34328, 2)

是否带锁
未带锁    31749
带锁      2579
Name: count, dtype: int64

(37613, 72)

是否带锁,带锁,未带锁
是否二手,,
二手,408,662
全新,1832,34711


## 1_17.日期处理

In [13]:
dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

0   2022-07-05
1   2022-07-13
2   2022-07-15
3   2022-07-16
4   2022-07-17
Name: 下单日期, dtype: datetime64[ns]

In [14]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
4季度    11213
3季度     9679
1季度     9478
2季度     7243
Name: count, dtype: int64

## 1_21.获取_买断数据

In [15]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
# note 20241126 增加distinct，买断可能分多次付款
# 181950     2
# 338547     2
# 1083822    2
# 303756     2
SELECT distinct t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(4069, 7)

## 1_22.租后数据 关联 买断数据

In [16]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.shape
dfzh1.shape

(37613, 80)

(37613, 76)

In [17]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

np.int64(5118)

In [18]:
dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()

if_outpay
未知      33701
已买断      3825
部分买断       87
Name: count, dtype: int64

## 1_31.获取_分期取消数据

In [19]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
# 订单取消
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(2303, 7)

## 1_32.租后数据关联分期取消数据

In [20]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.shape
dfzh2.shape

# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

(37613, 84)

(37613, 81)

## 1_41.按风控策略调整时间节点划分时间段

In [21]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

## 1_51.获取_账期数据（含续租）

In [22]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(497414, 23)

In [23]:
# note 20241227 号卡 应付日期更新
# 分期表
sql = '''-- 租后应收监控  
SELECT distinct hk.order_number, tos.order_id, tos.refund_date as refund_date_1
from db_digua_business.t_order_stages tos
inner join
    (select distinct om.id as order_id, om.order_number
    from db_digua_business.t_order om
    left join db_digua_business.t_platform_activity tpa 
    on tpa.id=om.activity_id
    where tpa.type = 4) hk
on tos.order_id = hk.order_id
'''
tmp_hk = query(sql)
tmp_hk['refund_date_ym'] = tmp_hk['refund_date_1'].dt.strftime('%Y-%m')
tmp_hk.shape
tmp_hk[:2]

df_xzfq['refund_date_ym'] = df_xzfq['refund_date'].dt.strftime('%Y-%m')
df_xzfq = df_xzfq.merge(tmp_hk[['order_id', 'refund_date_1', 'refund_date_ym']], on = ['order_id', 'refund_date_ym'], how = 'left')
df_xzfq['refund_date_2'] = np.where(df_xzfq.refund_date_1.notnull(), df_xzfq.refund_date_1, df_xzfq.refund_date)
df_xzfq.shape

df_xzfq.drop(columns=['refund_date_1','refund_date_ym','refund_date'],inplace = True)
df_xzfq = df_xzfq.rename(columns = {'refund_date_2':'refund_date'})
df_xzfq.shape

(4872, 4)

,order_number,order_id,refund_date_1,refund_date_ym
0,A20240927154904302,1533488,2024-09-30,2024-09
1,A20240927154904302,1533488,2024-10-30,2024-10


(497414, 26)

(497414, 23)

## 1_52.重命名

In [24]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期','reality_refund_date':'实付日期','sort':'当前期数'})

## 1_53.实还金额核查

In [25]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')

## 1_61.数据合并

In [26]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')
# note 20241126 增加
# df_xz_concat[df_xz_concat.当前应付租金.isnull()].shape
# df_xz_concat[df_xz_concat.当前应付租金.isna()].shape
# 实付日期为空，但当前应付租金为空
# 1681669 A20241104144812228
df_xz_concat.实付金额 = df_xz_concat.实付金额 + df_xz_concat.sesame_promo_money_pay + df_xz_concat.promo_money
df_xz_concat = df_xz_concat[df_xz_concat.当前应付租金.notnull()]

# order_id == 1681669 的当前期数为空
# 1681669	A20241104144812228
# 1669581	A2024110109350797
df_xz_concat['当前期数'].notnull().sum()
df_xz_concat.shape
df_xzfq.shape
dfzh3.shape

# df_xz_concat.drop_duplicates(subset=["order_id"]).shape
# dfzh3.drop_duplicates(subset=["order_id"]).shape

# df_xz_concat['order_id'].nunique()
# dfzh3['order_id'].nunique()
# df_xz_concat[["order_id","当前期数"]].info()

np.int64(497491)

(497491, 106)

(497414, 23)

(37613, 84)

## 1_62.去重

In [27]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
# print(all_duplicates.sort_values(by=['order_id','当前期数']))

In [28]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)
df_xz_concat.shape

(497095, 106)

In [29]:
# 119321
# 1083822 303756
# all_duplicates[all_duplicates['order_id'] == 181950].shape
# df_xz_concat[df_xz_concat['order_id'] == 181950].shape

# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

# 2_计算逾期天数

## 2_11.异常查询

In [30]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# 实付金额>0 实付日期为空 finish_date_new非空 :实付日期=订单完成时间
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# part_payment==0 reality_refund_date.notnull() 还款状态== '续租中' part_payment=money 还款状态== '已逾期' reality_refund_date=pd.na
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# 实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消
# dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 部分还款，实付日期非空，完成时间为空 用户： 实付日期清空
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1

# 部分还款，实付日期为空，完成时间非空 用户： 实付日期=完成时间
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1.订单状态值.value_counts()

# 已买断 但无实付日期及金额：实付日回写
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# # dd_test1.订单状态值.value_counts()
# dd_test1

## 2_12.实还日期修正

In [31]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [32]:
# （1）部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)

df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()     

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
339453,A2024110708472957,1692436,457.98,457.98,2025-01-10,2025-01-08,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
339454,A2024110708472957,1692436,457.98,457.98,2025-02-10,2025-02-08,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
339455,A2024110708472957,1692436,457.98,457.98,2025-03-10,2025-03-10,3.0,5.0,还款中,NaT,4,NaT,NaN,NaT
339456,A2024110708472957,1692436,457.98,457.98,2025-04-10,2025-04-06,3.0,6.0,还款中,NaT,4,NaT,NaN,NaT
470872,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,未到首期还款日,NaT,4,NaT,NaN,NaT


In [33]:
# （2）已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()       

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
339456,A2024110708472957,1692436,457.98,457.98,2025-04-10,2025-04-06,3.0,6.0,还款中,NaT,4,NaT,NaN,NaT
376535,A20241210113506313,1839860,695.21,665.65,2025-01-13,NaT,1.0,2.0,还款中,NaT,4,NaT,NaN,NaT
376536,A20241210113506313,1839860,695.21,665.65,2025-02-13,NaT,1.0,3.0,还款中,NaT,4,NaT,NaN,NaT
376537,A20241210113506313,1839860,695.21,665.65,2025-03-13,NaT,1.0,4.0,还款中,NaT,4,NaT,NaN,NaT
470872,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,未到首期还款日,NaT,4,NaT,NaN,NaT


In [34]:
# （3）租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
60964,2024-07-22


In [35]:
# （4）未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
8862,A2023021015233353,125911,633.21,0.0,2023-03-13,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9558,A2023021414250354,132517,334.26,0.0,2023-03-17,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9636,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
10650,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
12255,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17,1.0,11.0,已买断,NaT,8,NaT,NaN,NaT


In [36]:
# （5）实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] .tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
197320,A2024043016091519,1029646,978.25,0.0,2025-10-05,NaT,1.0,18.0,已买断,2025-02-26 14:31:23,8,2025-02-26,NaN,2025-02-26
200895,A2024050819102519,1043314,1056.37,0.0,2025-10-12,NaT,1.0,18.0,已买断,2025-03-14 21:57:00,8,2025-03-14,NaN,2025-03-14
215889,A2024060716074819,1111255,1085.68,0.0,2025-05-10,NaT,1.0,12.0,已买断,2024-12-14 14:59:21,8,2024-12-14,7.0,2024-12-14
223993,A20240620152351112,1150772,770.50,0.0,2026-02-24,NaT,1.0,21.0,已买断,2025-04-01 09:53:49,8,2025-04-01,NaN,2025-04-01
223994,A20240620152351112,1150772,514.32,0.0,2026-03-24,NaT,1.0,22.0,已买断,2025-04-01 09:53:49,8,2025-04-01,NaN,2025-04-01


In [37]:
# （6）部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [38]:
# （7）未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [39]:
# （8）实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [40]:
# （9）属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [41]:
# （10）df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [42]:
# （11）A202207050851373、A20241104144812228 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number.isin(['A202207050851373','A20241104144812228'])),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [43]:
# （12）实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)

In [44]:
# （13）df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 2_13.获取逾期截止时间

In [45]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date'].head()

0   2025-04-11
1   2025-04-11
2   2025-04-11
3   2025-04-11
4   2025-04-11
Name: now_date, dtype: datetime64[s]

## 2_14.重新定义状态

In [46]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

rembursement_status
还款中                    85536
已逾期                    50983
未到首期还款日                28026
续租中                    14710
检测中                       24
Name: count, dtype: int64

订单状态 
租赁中      169863
待归还        6137
已退款        5796
待收货        2962
已完成         220
待发货         187
订单取消         84
待商家收货        22
检测中          21
Name: count, dtype: int64

In [47]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     243115
未到还款日    178093
已逾期支付     35695
已逾期       34176
已取消        6016
Name: count, dtype: int64

In [48]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 2_15.重新定义逾期天数

In [49]:
df_xz_concat['实付日期new'].isnull().sum()
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

np.int64(185295)

Series([], Name: order_number, dtype: object)

In [50]:
# 异常值删除，应付时间为空
df_xz_concat.shape
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(497095, 110)

(497095, 110)

In [51]:
#实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 
#note 20241127 逾期天数 截止目前的逾期天数 ；逾期一直未还的客户，应付日期越靠前，逾期天数越高
# A202207181530165
# A2024092919033313
# df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][['order_number','逾期天数','应付日期','']]
# df_xz_concat[df_xz_concat.order_number == 'A2024092919033313'][['应付日期','实付日期','实付日期new','实付日期1','finish_date_new','当前期数','逾期天数','状态'
# ,'实付金额','当前应付租金','sesame_promo_money_pay','promo_money']]

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

# df_xz_concat['逾期天数'].value_counts()
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        50522
还款中         7093
已买断         6072
续租中         5047
已完成          954
检测中          143
未到首期还款日       27
Name: count, dtype: int64

In [52]:
# liudan
# # 定义一个函数来计算逾期天数
# def calculate_overdue_days(row):
#     if row['状态'] in ['未到还款日', '正常还款', '已取消']:
#         return 0
    
#     # 转换日期为 datetime 类型，并处理 None 和 NaN
#     实付日期new = pd.to_datetime(row['实付日期new'], errors='coerce')
#     应付日期 = pd.to_datetime(row['应付日期'], errors='coerce')
#     now_date = pd.to_datetime(row['now_date'], errors='coerce')

#     if pd.isna(实付日期new) or pd.isna(应付日期):
#         return 0
    
#     if 实付日期new >= 应付日期:
#         return (实付日期new - 应付日期).days
    
#     if pd.isna(实付日期new) and now_date > 应付日期:
#         return (now_date - 应付日期).days
    
#     return 0

# # 应用函数并创建新的 '逾期天数' 列
# df_xz_concat['逾期天数'] = df_xz_concat.apply(calculate_overdue_days, axis=1)

# # 显示结果
# print(df_xz_concat['逾期天数'].value_counts())

## 2_16.逾期天数处理

In [53]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

## 2_17.稽查

In [54]:
# df_ddfq.columns
# df_xz_concat.shape
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape
# check_fq_df.columns
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')


## 2_18.确认是否到表现期

In [55]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [56]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [57]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 

In [58]:
# from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','agr_15d','agr_7d','agr_4d','agr_1d','应付日期']].head()

,agr_30d,agr_15d,agr_7d,agr_4d,agr_1d,应付日期
0,2022-08-04,2022-07-20,2022-07-12,2022-07-09,2022-07-06,2022-07-05
1,2022-09-04,2022-08-20,2022-08-12,2022-08-09,2022-08-06,2022-08-05
2,2022-10-05,2022-09-20,2022-09-12,2022-09-09,2022-09-06,2022-09-05
3,2022-11-04,2022-10-20,2022-10-12,2022-10-09,2022-10-06,2022-10-05
4,2022-12-05,2022-11-20,2022-11-12,2022-11-09,2022-11-06,2022-11-05


In [59]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(5)
# df_xz_concat[(df_xz_concat.agr_7d_cum == 0) & (df_xz_concat.agr_4d_cum == 1)].head()
# df_xz_concat[df_xz_concat.order_id == 254588]

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2025-04-11,1
1,2022-09-04,2022-08-05,2025-04-11,1
2,2022-10-05,2022-09-05,2025-04-11,1
3,2022-11-04,2022-10-05,2025-04-11,1
4,2022-12-05,2022-11-05,2025-04-11,1


In [60]:
# note 20241211 数据检查
# df_xz_concat[(df_xz_concat.应付日期>='2024-12-01')&(df_xz_concat.应付日期<='2024-12-31')&(df_xz_concat.order_number.isin(
# ['A20241101174043208'
# ,'A2024110313351882'
# ,'A20241103155713186'
# ,'A2024110318311457'
# ,'A20241104145450273'
# ,'A20241104164506212'
# ,'A20241105105325369'
# ,'A20241105154302243'
# ,'A2024110515050030'
# ,'A2024110610041421'
# ,'A20241106144302240'
# ,'A2024110709254879'
# ,'A20241107103123201'
# ,'A20241107195535157'
# ,'A20241108104251268'
# ,'A20241108155751456'
# ,'A2024110715142073'
# ,'A2024110817092834'
# ]))][['order_number','agr_1d_cum','agr_4d_cum','agr_7d_cum','agr_4d','应付日期','date_str','agr_1d','下单日期']]

## 2_19.观察日定义 每月月底时间

In [61]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    # note n 续租的没有统计
    for n in range(0,13):  
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # note 20241205 修改 月末的的跳级了，所以不要加1，mob0对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat = mob_date(df_xz_concat)


# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

# A202301011402371 2023-01-01
# A202301290815565 2023-01-29
# A2023013100341134 2023-01-31
# A2023022801142011 2023-02-28
# A2023022709283120  2023-02-27
# A202302050022319  2023-02-05
# A2023040503314720 2023-04-05
# A2023042803470516  2023-04-28
# A2023043000524068 2023-04-30
# A2023063003392242 2023-06-30
# A202306280105556 2023-06-28
# A202306050324407  2023-06-05
# A202310050100222 2023-10-05
# A2023102901224646 2023-10-29
# A2023103101255825 2023-10-31

In [62]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

## 2_20.更新逾期天数

In [63]:
# （1）逾期天数校正
# note 20241213
# 首期实际付款日期为空的订单，为首期取消或者待发货的，贷后表现逾期天数更新为0（各期更新为0）

# 把符合条件的订单拿出来，逾期天数全部更新为0
sql_xz = ''' 
SELECT distinct tprm.order_number,1 as label
# ,ymos.reality_refund_date,ymos.sort as '当前期数',ymos.status
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
where ymos.sort=1 and ymos.reality_refund_date is null
;
'''
tmp = query(sql_xz)
tmp.shape
# tmp[:2]

df_xz_concat = df_xz_concat.merge(tmp, on = 'order_number', how = 'left')
df_xz_concat['label'] = df_xz_concat['label'].fillna(0)
df_xz_concat['label'].value_counts()

df_xz_concat[df_xz_concat['label'] == 1].drop_duplicates(subset = 'order_number').shape

df_xz_concat['逾期天数'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat['逾期天数_new'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].shape
# df_xz_concat[(df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']) & (df_xz_concat.逾期天数 > 30)].shape
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']]
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.max()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.min()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']].to_excel('tmp.xlsx', index = False)

# （2）2025.2.21
df_xz_concat['逾期天数'] = np.where(df_xz_concat.当前期数 == 1, 0, df_xz_concat.逾期天数)

(864, 2)

label
0.0    486835
1.0     10260
Name: count, dtype: int64

(843, 162)

In [64]:
# （2）联通卡号的逾期天数更新

# 3_客户好坏标签

In [65]:
# （1）历史最大逾期--续租的含续租
df_maxoverdue = df_xz_concat.groupby(['order_number','order_id','下单日期','search_time','归属渠道','是否二手','是否带锁']).agg({'逾期天数':'max'}).reset_index()
df_maxoverdue.shape
df_maxoverdue[:1]

(37593, 8)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数
0,A202207050851373,8924,2022-07-05,2022-07,未知渠道,二手,未带锁,4


In [66]:
# （2）FPD SPD TPD
df_xz_concat['当前期数'] = df_xz_concat['当前期数'].astype(int)

df_fstq = df_xz_concat[["order_id","当前期数","逾期天数"]].pivot(index="order_id", columns="当前期数", values="逾期天数")
df_fstq = df_fstq.reset_index()
df_fstq.shape

# df_fstq = df_fstq.rename(columns = {2: 'FPD'})
df_fstq['FPD'] = df_fstq[2]
df_fstq.loc[:,"SPD"] = np.where( df_fstq['FPD'] == 0, df_fstq[3], 0)
df_fstq.loc[:,"TPD"] = np.where((df_fstq['FPD'] == 0) & (df_fstq[3] == 0), df_fstq[4], 0)

# df_fstq = df_fstq[['order_id','FPD','SPD','TPD']]
df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])][['order_id','FPD','SPD','TPD']]
# df_xz_concat[df_xz_concat.order_number == 'A2023073114301076'][['order_id','下单日期','当前期数','应付日期']]
# A2023073114301076 508404
# df_fstq.FPD.max()

(37593, 37)

当前期数,order_id,FPD,SPD,TPD
28,12449,0.0,11.0,0.0
3719,508404,0.0,0.0,0.0
24300,1678492,126.0,0.0,0.0
24564,1692916,8.0,0.0,0.0


In [67]:
# （3）第2-4期逾期天数
df_fstq = df_fstq.rename(columns = {2: '第二期逾期天数', 3: '第三期逾期天数', 4: '第四期逾期天数'})
df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数']]
# tmp = df_xz_concat[df_xz_concat['当前期数'] == 1][['order_id','应付日期']]
# df_fstq = df_fstq.merge(tmp, on = 'order_id', how = 'left')
df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])]

当前期数,order_id,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数
28,12449,0.0,11.0,0.0,0.0,11.0,9.0
3719,508404,0.0,0.0,0.0,0.0,0.0,0.0
24300,1678492,126.0,0.0,0.0,126.0,95.0,64.0
24564,1692916,8.0,0.0,0.0,8.0,0.0,40.0


In [68]:
# （4）表关联
df_analy_data = df_maxoverdue.merge(df_fstq, on = 'order_id', how = 'left')
df_analy_data.shape
df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

(37593, 14)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数
28,A202208052103231,12449,2022-08-05,2022-08,顶部搜索框的搜索结果页,全新,未带锁,19,0.0,11.0,0.0,0.0,11.0,9.0
3719,A2023073114301076,508404,2023-07-31,2023-07,芝麻租物,全新,未带锁,6,0.0,0.0,0.0,0.0,0.0,0.0
24300,A20241103155713186,1678492,2024-11-03,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,126,126.0,0.0,0.0,126.0,95.0,64.0
24564,A20241107103123201,1692916,2024-11-07,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,40,8.0,0.0,0.0,8.0,0.0,40.0


# 4_蚂蚁数据产品

## 4_1.租赁分

### 4_1_1.2025.3之前

<!-- 背景--20241211：
（1）租赁通用分即蚁盾2.0，是蚁盾分的升级，本次测试返回，标签定义 FPD7+
（2）蚂蚁 数控风险等级已经不维护，分析其命中率和区分度
（3）蚂蚁 综合风险等级 进阶版目前切流到33%，免押率比专业版高1个点，分析 专业版和进阶版的差异 -->

In [67]:
# （1）测试数据导入
# afz_v1_score ：租赁通用分，取值 -1 表示库无记录
path_my = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/1_蚂蚁雨云租赁通用分-20241031/'
data_my_out = pd.read_excel(path_my + 'sample1031.xlsx')
data_my_in = pd.read_csv(path_my + '租赁通用分V1_迪瓜租机_241031.csv')
data_my_out.shape
data_my_in.shape

data_my_out = data_my_out.rename(columns = {'order_number_x':'order_number','id_card_num_x':'id_card_num','mobile_x':'mobile','idcard_sha256':'certno_hash','mobile_sha256':'mob_hash'})
data_my_out = data_my_out[['order_number','id_card_num','mobile','order_create_time','certno_hash','mob_hash']]
data_my_out["apply_date"] = data_my_out['order_create_time'].dt.date
data_my_out["apply_date"] = data_my_out["apply_date"].astype('str')
data_my_out["apply_date"] = data_my_out["apply_date"].str.replace('-','')

data_my_in = data_my_in[['certno_hash','mob_hash','apply_date','afz_v1_score']]
data_my_in['afz_v1_score'] = np.where(data_my_in.afz_v1_score == -1, -999, data_my_in.afz_v1_score)
data_my_in["apply_date"] = data_my_in["apply_date"].astype('str')
data_my_in[data_my_in.afz_v1_score == -999].shape

data_my = data_my_out.merge(data_my_in, on = ['certno_hash','mob_hash','apply_date'], how = 'inner')
data_my.shape
data_my.tail()

del data_my_out
del data_my_in

(5352, 15)

(5352, 10)

(5, 4)

(5352, 8)

,order_number,id_card_num,mobile,order_create_time,certno_hash,mob_hash,apply_date,afz_v1_score
5347,A20240806145156287,450922199508104057,15521784001,2024-08-06 14:51:56,27d90f809299b891c57d6fb4149272ac44020f2044826f...,0f2e3193529743f2675d7808f374ced26c3fad5ea3d4f1...,20240806,55.9252
5348,A20240327135748100,32128319861116245X,13401203268,2024-03-27 13:57:49,5dc8aa75d37b64de841d126b6b58333828ac33d8accbe6...,f1005050b7ebfb1b1c96b901434a30e5674a04ee49dca6...,20240327,52.4781
5349,A202408190203227,45092220010915313X,13528154887,2024-08-19 02:03:22,d18db5e3c4c41e1fe80dd095f47dcf519fc16a52499431...,60433e7238205fc922feed05b40cd01056f818974c6c2f...,20240819,63.2945
5350,A2024060816283070,64032220000608252X,18295556426,2024-06-08 16:28:31,8f9842c695ece9a3f1d15cc1632093da2f9de8d9b59bc7...,04df7696fab199941f5cb029d78d99f67f0018b87bd879...,20240608,65.0511
5351,A20240724142738100,500233199711138653,17682444945,2024-07-24 14:27:39,ad795c2230643a00974efbe251f2cffa945f7f0d4cf4c7...,66dca93cd10040cd739d130a5d524debc56f10bb681c1c...,20240724,51.8901


In [ ]:
# （2）蚁盾分取数
# 鑫仲帮忙取数
path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/1_蚂蚁雨云租赁通用分-20241031/'
df_ydf = pd.read_excel(path + '风险等级_更新.xlsx')
df_ydf.shape
df_ydf[:2]

# 跟本次测试相同样本
data_my = data_my.merge(df_ydf[['order_number','拒绝理由','蚁盾分']], on = 'order_number', how = 'left')
data_my['蚁盾分'] = data_my['蚁盾分'].fillna(-999)
data_my['蚁盾分'] = np.where(data_my['蚁盾分'] == -1, -999, data_my['蚁盾分'])
data_my.shape
data_my[:1]

In [164]:
# （3）测试数据关联贷后表现
data_my = data_my.merge(df_analy_data[['order_number', '下单日期', 'search_time', '归属渠道', '逾期天数','FPD']], on = 'order_number', how = 'left')
data_my.shape
data_my[:1]

(5352, 15)

,order_number,id_card_num,mobile,order_create_time,certno_hash,mob_hash,apply_date,afz_v1_score,拒绝理由,蚁盾分,下单日期,search_time,归属渠道,逾期天数,FPD
0,A202301020249553,35062419940424551X,17350298777,2023-01-02 02:49:55,ca7a90c5953d39c4208b4efec1f216d8ce2eb37b5fb29a...,00c98a95df910efef384453e1f0ec662f045e6b8061f27...,20230102,73.3025,NaN,-999.0,2023-01-02,2023-01,系统桌面图标,4,2.0


In [165]:
# （4）样本分析
# 2023.1-2023.12 的样本为历史逾期样本抽样
# 2024 的样本为随机抽样
data_my['y_fraud'] = data_my.apply(lambda x: 1 if x.FPD >= 7 
                                       else 0.5 if x.FPD > 0 and x.FPD < 7
                                       else 0, axis = 1)
data_my['y_credit'] = data_my.apply(lambda x: 1 if x.逾期天数 >= 31 
                                       else 0.5 if x.逾期天数 > 0 and x.逾期天数 < 31
                                       else 0, axis = 1)

df_analy_data['y_fraud'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 7 
                                       else 0.5 if x.FPD > 0 and x.FPD < 7
                                       else 0, axis = 1)
df_analy_data['y_credit'] = df_analy_data.apply(lambda x: 1 if x.逾期天数 >= 31 
                                       else 0.5 if x.逾期天数 > 0 and x.逾期天数 < 31
                                       else 0, axis = 1)
# data_my['y'].value_counts()
# data_my[data_my.逾期天数 > 0].shape
pd.pivot_table(data_my,index='search_time',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my,index='search_time',columns='y_credit',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_data,index='search_time',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_data,index='search_time',columns='y_credit',values='order_number',aggfunc='count')

data_my_train = data_my[data_my.search_time >= '2024-01']
data_my_train.shape


y_fraud,0.0,0.5,1.0
search_time,,,
2023-01,21,15,10
2023-02,20,16,21
2023-03,33,4,13
2023-04,86,29,25
2023-05,65,24,18
2023-06,72,37,19
2023-07,81,53,29
2023-08,78,28,38
2023-09,53,31,29


y_credit,0.0,0.5,1.0
search_time,,,
2023-01,NaN,12.0,34.0
2023-02,NaN,14.0,43.0
2023-03,NaN,13.0,37.0
2023-04,NaN,51.0,89.0
2023-05,NaN,29.0,78.0
2023-06,NaN,34.0,94.0
2023-07,NaN,34.0,129.0
2023-08,NaN,29.0,115.0
2023-09,NaN,24.0,89.0


y_fraud,0.0,0.5,1.0
search_time,,,
2022-07,18.0,1.0,3.0
2022-08,34.0,13.0,9.0
2022-09,35.0,3.0,6.0
2022-10,34.0,4.0,3.0
2022-11,31.0,8.0,4.0
2022-12,53.0,5.0,8.0
2023-01,168.0,35.0,18.0
2023-02,258.0,36.0,38.0
2023-03,276.0,9.0,31.0


y_credit,0.0,0.5,1.0
search_time,,,
2022-07,2.0,10.0,10.0
2022-08,3.0,23.0,30.0
2022-09,4.0,22.0,18.0
2022-10,4.0,21.0,16.0
2022-11,8.0,22.0,13.0
2022-12,14.0,28.0,24.0
2023-01,41.0,88.0,92.0
2023-02,103.0,108.0,121.0
2023-03,112.0,81.0,123.0


(3897, 17)

In [167]:
# （5）分箱
%run C:/Users/Administrator/Desktop/mine/4_专题分析/1_回捞分析/2.model_func.ipynb
continous_merged_dict = {}
var_bin_list = []
trainData_kf_Bin = data_my_train[data_my_train.y_fraud.isin([0,1])]
trainData_kf_Bin['y'] = trainData_kf_Bin['y_fraud']
numeric_desc_var = ['afz_v1_score','蚁盾分']

# start_time = time.time()

for col in numeric_desc_var:
    print ("{} is in processing".format(col))
    if -1111 not in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]): 
        max_interval = 5 
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval,special_attribute=[],minBinPcnt=0)
        trainData_kf_Bin[col+'_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff,special_attribute=[]))
        monotone = BadRateMonotone(trainData_kf_Bin, col+'_Bin', 'y')
        while(not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute = [],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = []))
            if max_interval == 2:
                break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y')
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [])) 
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    else:
        max_interval = 7 #正常取值最多分5箱
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval = max_interval, special_attribute = [-1111, -999],
                                      minBinPcnt = 0)
        trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1111, -999]))
        monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2'])
        while (not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute=[-1111, -999],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))
            if max_interval == 3:
                break
#             if max_interval == 3 and -1111 in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 3 and -1111 not in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 4 and -1111 in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2']) 
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))  
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    continous_merged_dict[col] = cutOff
    
# print("- Time consumed: %s seconds" % int(time.time() - start_time))

afz_v1_score is in processing
蚁盾分 is in processing


In [168]:
# （6）改变映射方式
trainData_kf_Bin_1 = data_my_train[data_my_train.y_fraud.isin([0,1])]
trainData_kf_Bin_1['y'] = trainData_kf_Bin_1['y_fraud']

# start_time = time.time()
for col in numeric_desc_var:
    if -1111 not in set(trainData_kf_Bin_1[col]) and -999 not in set(trainData_kf_Bin_1[col]):
        trainData_kf_Bin_1[col+'_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col]))
        del trainData_kf_Bin_1[col]
    else:
        trainData_kf_Bin_1[col + '_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col],[-1111, -999]))
        del trainData_kf_Bin_1[col]
        
# print("- Time consumed: %s seconds" % int(time.time() - start_time))

In [169]:
# （7）结果统计
# data_my_train = data_my_train[data_my_train.y_fraud.isin([0,1])]
data_my_train['y'] = data_my_train['y_fraud']

data_my_train['afz_v1_score_Bin_new'] = data_my_train.apply(lambda x: '[0,50]' if x.afz_v1_score >=0 and x.afz_v1_score <= 50
                                                                             else '(50,65]' if x.afz_v1_score > 50 and x.afz_v1_score  <= 65
                                                                             else '(65,80]' if x.afz_v1_score > 65 and x.afz_v1_score  <= 80
                                                                             else '(80,inf)' if x.afz_v1_score > 80
                                                                             else '[-999,-999]' if x.afz_v1_score == -999
                                                                             else 'other'
                                                                             ,axis = 1)

data_my_train['蚁盾分_Bin_new'] = data_my_train.apply(lambda x: '[0,50]' if x.蚁盾分 >=0 and x.蚁盾分 <= 50
                                                                             else '(50,65]' if x.蚁盾分 > 50 and x.蚁盾分<= 65
                                                                             else '(65,80]' if x.蚁盾分 > 65 and x.蚁盾分 <= 80
                                                                             else '(80,inf)' if x.蚁盾分 > 80
                                                                             else '[-999,-999]' if x.蚁盾分 == -999
                                                                             else 'other'
                                                                            ,axis = 1)
pd.pivot_table(trainData_kf_Bin_1,index='afz_v1_score_Bin',columns='y',values='order_number',aggfunc='count')
pd.pivot_table(trainData_kf_Bin_1,index='蚁盾分_Bin',columns='y',values='order_number',aggfunc='count')
# pd.pivot_table(data_my_train,index='afz_v1_score_Bin_new',columns='y',values='order_number',aggfunc='count')
# pd.pivot_table(data_my_train,index='蚁盾分_Bin_new',columns='y',values='order_number',aggfunc='count')

pd.pivot_table(data_my_train[data_my_train.归属渠道 != '搜索渠道'],index='afz_v1_score_Bin_new',columns='y',values='order_number',aggfunc='count')
pd.pivot_table(data_my_train[data_my_train.归属渠道 != '搜索渠道'],index='蚁盾分_Bin_new',columns='y',values='order_number',aggfunc='count')

pd.pivot_table(data_my_train[data_my_train.归属渠道 == '搜索渠道'],index='afz_v1_score_Bin_new',columns='y',values='order_number',aggfunc='count')
pd.pivot_table(data_my_train[data_my_train.归属渠道 == '搜索渠道'],index='蚁盾分_Bin_new',columns='y',values='order_number',aggfunc='count')

y,0.0,1.0
afz_v1_score_Bin,,
"(-999,53.1126]",1066.0,30.0
"(53.1126,64.2071]",1957.0,87.0
"(64.2071,inf)",465.0,45.0
"[-999,-999]",2.0,NaN


y,0.0,1.0
蚁盾分_Bin,,
"(-999,6.2025]",552,10
"(19.9365,45.414]",1205,54
"(45.414,inf)",454,36
"(6.2025,19.9365]",1139,51
"[-999,-999]",140,11


y,0.0,0.5,1.0
afz_v1_score_Bin_new,,,
"(50,65]",1979.0,120.0,78.0
"(65,80]",314.0,45.0,26.0
"(80,inf)",3.0,5.0,3.0
"[-999,-999]",1.0,NaN,NaN
"[0,50]",492.0,23.0,13.0


y,0.0,0.5,1.0
蚁盾分_Bin_new,,,
"(50,65]",234.0,16.0,20.0
"(65,80]",84.0,11.0,5.0
"(80,inf)",NaN,NaN,1.0
"[-999,-999]",121.0,13.0,10.0
"[0,50]",2350.0,153.0,84.0


y,0.0,0.5,1.0
afz_v1_score_Bin_new,,,
"(50,65]",532.0,43.0,29.0
"(65,80]",64.0,4.0,8.0
"(80,inf)",3.0,NaN,NaN
"[-999,-999]",1.0,NaN,NaN
"[0,50]",101.0,5.0,5.0


y,0.0,0.5,1.0
蚁盾分_Bin_new,,,
"(50,65]",6,3,1
"[-999,-999]",19,4,1
"[0,50]",676,45,40


In [ ]:
# （8）重新提供测试样本
# 2024.9-2024.11 的出库订单 --评估稳定性、区分度
# 2024.12.1-2024.12.17 的进件 --评估单条规则命中率、独立命中率

# 草稿
# # （8-1）全量三要素 姓名、身份证、手机号
# #   `status` int DEFAULT NULL COMMENT '状态 1,待支付,2,待发货,3,待收货,4,租赁中,5,待归还,6,待商家收货/归还中,8,已完成,10,已退款,\r12,待审核,13,订单取消, 15,检测中',
# sql = '''-- 租后应收监控  
# SELECT  om.id as order_id, om.order_number, om.create_time, om.merchant_name
# ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
# when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
# when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
# , tmu.true_name, tmu.id_card_num, tmu.mobile 
# , tod.sku_attributes
# , top.total_describes
# , pa.name as activity_name        -- 活动名称
# from db_digua_business.t_order om
# left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
# left join  db_digua_business.t_order_details tod on om.id = tod.order_id
# left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
# left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
# ;
# '''

# df_info = query(sql)
# df_info["下单日期"] = df_info['create_time'].dt.date
# df_info["下单日期"] = pd.to_datetime(df_info["下单日期"],errors="coerce")
# df_info.loc[:,"进件"]=np.where((df_info["订单状态"]=="待支付")|(df_info["订单状态"]=="订单取消"), "未进件", "进件")

# # 剔除商家
# df_info.drop(df_info[df_info['merchant_name'] == "深圳优优大数据科技有限公司"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "优优2店"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "小豚租（代收）"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "苏州蚁诺宝"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "租着用电脑数码"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "北京海鸟窝科技有限公司"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "汇客好租"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "澄心优租"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "CPS渠道合作"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
# df_info.drop(df_info[df_info['sku_attributes'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "趣智数码"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "格木木二奢名品"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "广州康基贸易有限公司"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "线下小店"].index, inplace=True)
# df_info.drop(df_info[df_info['merchant_name'] == "乙辉数码"].index, inplace=True)
# # 删去刷单订单
# df_info.drop(df_info[df_info['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
# df_info.drop(df_info[df_info['activity_name']=="1000单秘密计划"].index,inplace=True)
# df_info.drop(df_info[df_info['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
# df_info.drop(df_info[df_info['activity_name']=="1000单曙光计划"].index,inplace=True)
# df_info.drop(df_info[df_info['activity_name']=="线下门店3个月试行"].index,inplace=True)
# # 删除订单状态空值行
# df_info.dropna(subset=["订单状态"],axis=0,inplace=True)
# # 删除身份证空值行
# df_info.dropna(subset=["id_card_num"],axis=0,inplace=True)
# #删除重复订单
# df_info.drop_duplicates(subset=["order_id"],inplace=True)
# df_info.drop_duplicates(subset=["true_name","mobile","id_card_num","下单日期"],keep="last",inplace=True)
# df_info.drop(df_info[df_info['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
# ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# # (1265575, 13)
# df_info.shape

# # # （8-2）

# # df_analy_data[(df_analy_data.search_time >= '2024-09') & (df_analy_data.search_time <= '2024-11')].shape
# # df_analy_data[:1]


# df_info[(df_info.下单日期 >= '2024-12-01') & ((df_info.进件 == '进件'))]['下单日期'].value_counts()
# 



# （8-1）全量三要素 姓名、身份证、手机号、下单日期
sql = '''SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips   
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tomt.reason
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id

where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -150 day ) 
;
'''

df = query(sql)

df["下单日期"] = df['create_time'].dt.date
df["下单日期"] = pd.to_datetime(df["下单日期"],errors="coerce")

dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)

df.drop(df[df['merchant_name'] == "深圳优优大数据科技有限公司"].index, inplace=True)
df.drop(df[df['merchant_name'] == "优优2店"].index, inplace=True)
df.drop(df[df['merchant_name'] == "小豚租（代收）"].index, inplace=True)
df.drop(df[df['merchant_name'] == "苏州蚁诺宝"].index, inplace=True)
df.drop(df[df['merchant_name'] == "租着用电脑数码"].index, inplace=True)
df.drop(df[df['merchant_name'] == "北京海鸟窝科技有限公司"].index, inplace=True)
df.drop(df[df['merchant_name'] == "汇客好租"].index, inplace=True)
df.drop(df[df['merchant_name'] == "澄心优租"].index, inplace=True)
df.drop(df[df['merchant_name'] == "CPS渠道合作"].index, inplace=True)
df.drop(df[df['merchant_name'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
df.drop(df[df['sku_attributes'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
df.drop(df[df['merchant_name'] == "趣智数码"].index, inplace=True)
df.drop(df[df['merchant_name'] == "格木木二奢名品"].index, inplace=True)
df.drop(df[df['merchant_name'] == "广州康基贸易有限公司"].index, inplace=True)
df.drop(df[df['merchant_name'] == "线下小店"].index, inplace=True)
df.drop(df[df['merchant_name'] == "乙辉数码"].index, inplace=True)

#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#删去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除订单状态空值行
df.dropna(subset=["status2"],axis=0,inplace=True)
#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
# df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)

df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 取12月进件
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"), "未进件", "进件")
df_12 = df[(df.进件 == '进件') & (df.下单日期 >= '2024-12-01') & (df.下单日期 <= '2024-12-15')][["true_name","user_mobile","id_card_num","下单日期",'order_id','order_number']]
df_12.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)

# （8-2）关联 2024.9-2024.11 有贷后表现样本的三要素
df_9_11 = df[['true_name','user_mobile','id_card_num','order_number']].merge(df_analy_data[(df_analy_data.search_time >= '2024-09')&(df_analy_data.search_time <= '2024-11')], on = ['order_number'], how = 'right')

# （8-3）表合并
df_my_sample = pd.concat([df_9_11, df_12])
df_my_sample.shape

# （8—4）身份证、手机号 sha256加密
# create_date	idcard_sha256	mobile_sha256
import hashlib
 
def sha256_encryption(var):
    sha256_hash = hashlib.sha256()  # 创建一个md5对象
    sha256_hash.update(var.encode('utf-8'))  # 使用utf-8编码数据
    return sha256_hash.hexdigest()  # 返回加密后的十六进制字符串

df_my_sample['id_card_num'] = df_my_sample['id_card_num'].astype('str')
df_my_sample['user_mobile'] = df_my_sample['user_mobile'].astype('str')
df_my_sample['idcard_sha256'] = df_my_sample.apply(lambda x: sha256_encryption(x.id_card_num), axis = 1)
df_my_sample['mobile_sha256'] = df_my_sample.apply(lambda x: sha256_encryption(x.user_mobile), axis = 1)
df_my_sample.shape
df_my_sample.tail()

df_my_sample['下单日期'] = df_my_sample['下单日期'].astype('str')
# 15228562930	511902199201051212
# abca52676f514401575b35192a28d250a70e99ef12c7826f3f5018f2a8cd53fa  5794bb8b804ffcea6f90d50f3df270f227ec01e9766c561ea5f95ad79576791e
# card_id = '522121199705104611'
# sha256 = hashlib.sha256(card_id.encode())
# sha256.hexdigest()
path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/2_租赁分洞察分测试_20241218/'
df_my_sample.to_excel(path + 'my_sample_20241218.xlsx', index = False)


In [67]:
# （9）二次导入
# 2024.9-2024.11 的出库订单 --评估稳定性、区分度
# 2024.12.1-2024.12.15 的进件 --评估单条规则命中率、独立命中率
# 2024.12.25 返回信贷洞察分、蚁盾分2.0
# afz_v1_score ：租赁通用分，取值 -1 表示库无记录
path_my = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/2_租赁分洞察分测试_20241218/'
data_my_out = pd.read_excel(path_my + 'my_sample_20241218.xlsx')
data_my_in_1 = pd.read_csv(path_my + '租赁通用分V1_迪瓜_241220.csv')
data_my_in_2 = pd.read_csv(path_my + '反欺诈标准版_迪瓜_20241220.csv')

data_my_in_1 = data_my_in_1[['certno_hash','mob_hash','apply_date','afz_v1_score']]
data_my_in_1['afz_v1_score'] = np.where(data_my_in_1.afz_v1_score == -1, -999, data_my_in_1.afz_v1_score)
data_my_in_1["apply_date"] = data_my_in_1["apply_date"].astype('str')
data_my_in_1[data_my_in_1.afz_v1_score == -999].shape
data_my_in_1 = data_my_in_1.drop_duplicates(subset = ['certno_hash','mob_hash','apply_date','afz_v1_score'])

data_my_in_2 = data_my_in_2[['certno_hash','mob_hash','apply_date','aff_v0_mapr_score','aff_v0_hapr_score']]
data_my_in_2['aff_v0_mapr_score'] = np.where(data_my_in_2.aff_v0_mapr_score == -1, -999, data_my_in_2.aff_v0_mapr_score)
data_my_in_2['aff_v0_hapr_score'] = np.where(data_my_in_2.aff_v0_hapr_score == -1, -999, data_my_in_2.aff_v0_hapr_score)
data_my_in_2["apply_date"] = data_my_in_2["apply_date"].astype('str')
data_my_in_2[data_my_in_2.aff_v0_mapr_score == -999].shape
data_my_in_2[data_my_in_2.aff_v0_hapr_score == -999].shape
data_my_in_2 = data_my_in_2.drop_duplicates(subset = ['certno_hash','mob_hash','apply_date','aff_v0_mapr_score','aff_v0_hapr_score'])

data_my_out = data_my_out[['order_number','order_id', '下单日期', '归属渠道','idcard_sha256','mobile_sha256']]
data_my_out = data_my_out.rename(columns = {'idcard_sha256': 'certno_hash', 'mobile_sha256': 'mob_hash'})
data_my_out["apply_date"] = data_my_out["下单日期"].astype('str')
data_my_out["apply_date"] = data_my_out["apply_date"].str.replace('-','')
data_my_out['search_time'] = pd.to_datetime(data_my_out["下单日期"]).dt.strftime('%Y-%m')

data_my_out.shape
data_my_in_1.shape
data_my_in_2.shape

data_my = data_my_out.merge(data_my_in_1, on = ['certno_hash','mob_hash','apply_date'], how = 'left')
data_my = data_my.merge(data_my_in_2, on = ['certno_hash','mob_hash','apply_date'], how = 'left')
data_my.drop(columns=['apply_date','certno_hash','mob_hash'],inplace = True)
data_my.shape
data_my[:2]

del data_my_in_1
del data_my_in_2
del data_my_out

(16, 4)

(0, 5)

(0, 5)

(17764, 8)

(17757, 4)

(17757, 5)

(17764, 8)

,order_number,order_id,下单日期,归属渠道,search_time,afz_v1_score,aff_v0_mapr_score,aff_v0_hapr_score
0,A20240901005306132,1412785,2024-09-01,芝麻租物,2024-09,47.5799,0.289,3.118
1,A20240901005509136,1412789,2024-09-01,芝麻租物,2024-09,57.9553,40.473,83.095


In [67]:
# （10）关联租后表现
data_my = data_my.merge(df_analy_data[['order_number','逾期天数','FPD','SPD','TPD']], on = 'order_number', how = 'left')
data_my['afz_v1_score'] = data_my['afz_v1_score'].fillna(-999)
data_my.shape
data_my[:1]

# data_my[data_my.逾期天数.isnull()].下单日期.min()
# 样本打标签
# 蚁盾分  欺诈风险 Mob1 & FPD7+
# 信贷洞察分 欺诈风险 Mob3 & dpd30+
data_my['y_fraud'] = data_my.apply(lambda x: 1 if x.FPD >= 7 
                                       else 0.5 if x.FPD > 0 and x.FPD < 7
                                       else 0 if x.FPD == 0
                                       else 2 , axis = 1)
data_my['y_credit'] = data_my.apply(lambda x: 1 if x.逾期天数 >= 30 
                                       else 0.5 if x.逾期天数 > 0 and x.逾期天数 < 30
                                       else 0 if x.逾期天数 == 0
                                       else 2, axis = 1)
data_my['y_fraud'].value_counts()
data_my['y_credit'].value_counts()

(17764, 12)

,order_number,order_id,下单日期,归属渠道,search_time,afz_v1_score,aff_v0_mapr_score,aff_v0_hapr_score,逾期天数,FPD,SPD,TPD
0,A20240901005306132,1412785,2024-09-01,芝麻租物,2024-09,47.5799,0.289,3.118,0.0,0.0,0.0,0.0


y_fraud
2.0    9410
0.0    7851
0.5     298
1.0     205
Name: count, dtype: int64

y_credit
2.0    9410
0.0    7297
0.5     860
1.0     197
Name: count, dtype: int64

In [68]:
# （11）关联蚁盾分
path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/4_策略分析/'
my_risk_level = pd.read_excel(path + '风险等级_更新20241225.xlsx')
my_risk_level = my_risk_level[['order_number', '蚁盾分', '拒绝理由']]
my_risk_level = my_risk_level.drop_duplicates(subset = ['order_number'])
my_risk_level.shape
# my_risk_level.tail()

data_my = data_my.merge(my_risk_level[['order_number','蚁盾分','拒绝理由']], on = 'order_number', how = 'left')
data_my['蚁盾分'] = data_my['蚁盾分'].fillna(-999)
data_my['蚁盾分'] = np.where(data_my['蚁盾分'] == -1, -999, data_my['蚁盾分'])
data_my.shape
data_my[:1]

(226881, 3)

(17764, 16)

,order_number,order_id,下单日期,归属渠道,search_time,afz_v1_score,aff_v0_mapr_score,aff_v0_hapr_score,逾期天数,FPD,SPD,TPD,y_fraud,y_credit,蚁盾分,拒绝理由
0,A20240901005306132,1412785,2024-09-01,芝麻租物,2024-09,47.5799,0.289,3.118,0.0,0.0,0.0,0.0,0.0,0.0,0.855,NaN


In [70]:
# （12-1）分箱统计--非搜索渠道
# 算法分箱切的cutoff比较集中
# 2025.1.6 更新租后表现，可以拿到截止2024.11.27的
data_my['afz_v1_score_Bin_new'] = data_my.apply(lambda x: '[0,25]' if x.afz_v1_score >=0 and x.afz_v1_score <= 25
                                                                            else '(25,50]' if x.afz_v1_score > 25 and x.afz_v1_score  <= 50
                                                                             else '(50,65]' if x.afz_v1_score > 50 and x.afz_v1_score  < 65
                                                                             else '[65,75)' if x.afz_v1_score >= 65 and x.afz_v1_score  < 75
                                                                            #  else '(65,70)' if x.afz_v1_score > 65 and x.afz_v1_score  < 70
                                                                            #  else '[70,75)' if x.afz_v1_score >= 70 and x.afz_v1_score  < 75                                        
                                                                             else '[75,80)' if x.afz_v1_score >= 75 and x.afz_v1_score  < 80
                                                                             else '[80,inf)' if x.afz_v1_score >= 80
                                                                             else '[-999,-999]' if x.afz_v1_score == -999
                                                                             else 'other'
                                                                             ,axis = 1)

data_my['蚁盾分_Bin_new'] = data_my.apply(lambda x: '[0,25]' if x.蚁盾分 >=0 and x.蚁盾分 <= 25
                                                                             else '(25,50]' if x.蚁盾分 > 25 and x.蚁盾分  <= 50
                                                                             else '(50,65)' if x.蚁盾分 > 50 and x.蚁盾分< 65
                                                                             else '[65,75)' if x.蚁盾分 >= 65 and x.蚁盾分  < 75
                                                                            #  else '(65,70)' if x.蚁盾分 > 65 and x.蚁盾分  < 70
                                                                            #  else '[70,75)' if x.蚁盾分 >= 70 and x.蚁盾分  < 75
                                                                             else '[75,80)' if x.蚁盾分 >= 75 and x.蚁盾分  < 80
                                                                             else '[80,inf)' if x.蚁盾分 >= 80
                                                                             else '[-999,-999]' if x.蚁盾分 == -999
                                                                             else 'other'
                                                                            ,axis = 1)

# pd.pivot_table(data_my[(data_my.search_time == '2024-09') & (data_my.归属渠道 != '搜索渠道')],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-10') & (data_my.归属渠道 != '搜索渠道')],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-11') & (data_my.下单日期 <= '2024-11-23') & (data_my.归属渠道 != '搜索渠道')],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-09') & (data_my.归属渠道 != '搜索渠道')],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-10') & (data_my.归属渠道 != '搜索渠道')],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-11') & (data_my.下单日期 <= '2024-11-23') & (data_my.归属渠道 != '搜索渠道')],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

pd.pivot_table(data_my[(data_my.search_time == '2024-09') & (data_my.归属渠道 == '芝麻租物')],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-10') & (data_my.归属渠道 == '芝麻租物')],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-11') & (data_my.下单日期 <= '2024-11-27') & (data_my.归属渠道 == '芝麻租物')],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-09') & (data_my.归属渠道 == '芝麻租物')],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-10') & (data_my.归属渠道 == '芝麻租物')],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-11') & (data_my.下单日期 <= '2024-11-27') & (data_my.归属渠道 == '芝麻租物')],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

pd.pivot_table(data_my[(data_my.search_time == '2024-09') & (~data_my.归属渠道.isin(['芝麻租物','搜索渠道']))],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-10') & (~data_my.归属渠道.isin(['芝麻租物','搜索渠道']))],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-11') & (data_my.下单日期 <= '2024-11-27') & (~data_my.归属渠道.isin(['芝麻租物','搜索渠道']))],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-09') & (~data_my.归属渠道.isin(['芝麻租物','搜索渠道']))],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-10') & (~data_my.归属渠道.isin(['芝麻租物','搜索渠道']))],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-11') & (data_my.下单日期 <= '2024-11-27') & (~data_my.归属渠道.isin(['芝麻租物','搜索渠道']))],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')


y_fraud,0.0,0.5,1.0
afz_v1_score_Bin_new,,,
"(25,50]",293.0,11.0,11.0
"(50,65]",1180.0,57.0,39.0
"[0,25]",1.0,NaN,NaN
"[65,75)",126.0,7.0,6.0
"[75,80)",5.0,1.0,2.0
"[80,inf)",1.0,NaN,NaN


y_fraud,0.0,0.5,1.0
afz_v1_score_Bin_new,,,
"(25,50]",111,1,1
"(50,65]",508,22,18
"[65,75)",44,5,3


y_fraud,0.0,0.5,1.0
afz_v1_score_Bin_new,,,
"(25,50]",184.0,5.0,NaN
"(50,65]",679.0,29.0,13.0
"[-999,-999]",2.0,NaN,NaN
"[65,75)",74.0,4.0,3.0
"[75,80)",1.0,NaN,1.0


y_fraud,0.0,0.5,1.0
蚁盾分_Bin_new,,,
"(25,50]",445,20,20
"(50,65)",131,5,10
"[-999,-999]",47,4,2
"[0,25]",909,41,22
"[65,75)",50,5,3
"[75,80)",24,1,1


y_fraud,0.0,0.5,1.0
蚁盾分_Bin_new,,,
"(25,50]",205.0,11.0,10.0
"(50,65)",45.0,3.0,2.0
"[-999,-999]",3.0,NaN,NaN
"[0,25]",377.0,11.0,9.0
"[65,75)",25.0,3.0,1.0
"[75,80)",8.0,NaN,NaN


...

y_fraud,0.0,0.5,1.0
afz_v1_score_Bin_new,,,
"(25,50]",38.0,1.0,1.0
"(50,65]",144.0,4.0,5.0
"[-999,-999]",1.0,NaN,NaN
"[65,75)",12.0,2.0,2.0
"[75,80)",1.0,NaN,NaN


y_fraud,0.0,0.5,1.0
afz_v1_score_Bin_new,,,
"(25,50]",55.0,2.0,1.0
"(50,65]",221.0,8.0,11.0
"[65,75)",13.0,2.0,2.0
"[75,80)",2.0,1.0,NaN


y_fraud,0.0,0.5,1.0
蚁盾分_Bin_new,,,
"(25,50]",62.0,NaN,5.0
"(50,65)",18.0,NaN,3.0
"[-999,-999]",20.0,4.0,3.0
"[0,25]",143.0,9.0,8.0
"[65,75)",5.0,NaN,1.0
"[75,80)",2.0,NaN,NaN
"[80,inf)",1.0,NaN,NaN


y_fraud,0.0,0.5,1.0
蚁盾分_Bin_new,,,
"(25,50]",48.0,2.0,6.0
"(50,65)",13.0,1.0,NaN
"[-999,-999]",1.0,1.0,NaN
"[0,25]",126.0,2.0,2.0
"[65,75)",5.0,1.0,NaN
"[75,80)",3.0,NaN,NaN


y_fraud,0.0,0.5,1.0
蚁盾分_Bin_new,,,
"(25,50]",73.0,6.0,6.0
"(50,65)",26.0,1.0,1.0
"[-999,-999]",20.0,1.0,NaN
"[0,25]",164.0,5.0,7.0
"[65,75)",6.0,NaN,NaN
"[75,80)",1.0,NaN,NaN
"[80,inf)",1.0,NaN,NaN


In [71]:
# （12-2）分箱统计--搜索渠道
# 算法分箱切的cutoff比较集中
data_my['afz_v1_score_Bin_new'] = data_my.apply(lambda x: '[0,25]' if x.afz_v1_score >=0 and x.afz_v1_score <= 25
                                                                             else '(25,45]' if x.afz_v1_score > 25 and x.afz_v1_score  <= 45
                                                                             else '(45,50]' if x.afz_v1_score > 45 and x.afz_v1_score  <= 50
                                                                             else '(50,65)' if x.afz_v1_score > 50 and x.afz_v1_score  < 65
                                                                             else '[65,70)' if x.afz_v1_score >= 65 and x.afz_v1_score  < 70
                                                                             else '[70,75)' if x.afz_v1_score >= 70 and x.afz_v1_score  < 75
                                                                             else '[75,inf)' if x.afz_v1_score >= 75
                                                                             else '[-999,-999]' if x.afz_v1_score == -999
                                                                             else 'other'
                                                                             ,axis = 1)

data_my['蚁盾分_Bin_new'] = data_my.apply(lambda x: '[0,25]' if x.蚁盾分 >=0 and x.蚁盾分 <= 25
                                                                             else '(25,45]' if x.蚁盾分 > 25 and x.蚁盾分  <= 45
                                                                             else '(45,50]' if x.蚁盾分 > 45 and x.蚁盾分  <= 50
                                                                             else '(50,65)' if x.蚁盾分 > 50 and x.蚁盾分  < 65
                                                                             else '[65,70)' if x.蚁盾分 >= 65 and x.蚁盾分  < 70
                                                                             else '[70,75)' if x.蚁盾分 >= 70 and x.蚁盾分  < 75
                                                                             else '[75,inf)' if x.蚁盾分 >= 75
                                                                             else '[-999,-999]' if x.蚁盾分 == -999
                                                                             else 'other'
                                                                             ,axis = 1)

pd.pivot_table(data_my[(data_my.search_time == '2024-09') & (data_my.归属渠道 == '搜索渠道')],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-10') & (data_my.归属渠道 == '搜索渠道')],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-11') & (data_my.下单日期 <= '2024-11-27') & (data_my.归属渠道 == '搜索渠道')],index='afz_v1_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-09') & (data_my.归属渠道 == '搜索渠道')],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-10') & (data_my.归属渠道 == '搜索渠道')],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-11') & (data_my.下单日期 <= '2024-11-27') & (data_my.归属渠道 == '搜索渠道')],index='蚁盾分_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

y_fraud,0.0,0.5,1.0
afz_v1_score_Bin_new,,,
"(25,45]",27.0,1.0,1.0
"(45,50]",84.0,2.0,5.0
"(50,65)",380.0,22.0,13.0
"[-999,-999]",3.0,NaN,1.0
"[0,25]",1.0,NaN,NaN
"[65,70)",25.0,NaN,2.0
"[70,75)",6.0,1.0,1.0


y_fraud,0.0,0.5,1.0
afz_v1_score_Bin_new,,,
"(25,45]",26.0,1.0,NaN
"(45,50]",91.0,3.0,2.0
"(50,65)",476.0,22.0,13.0
"[0,25]",1.0,NaN,NaN
"[65,70)",33.0,2.0,NaN
"[70,75)",11.0,1.0,1.0


y_fraud,0.0,0.5,1.0
afz_v1_score_Bin_new,,,
"(25,45]",27.0,NaN,2.0
"(45,50]",106.0,6.0,4.0
"(50,65)",660.0,25.0,19.0
"[65,70)",61.0,8.0,1.0
"[70,75)",11.0,NaN,1.0
"[75,inf)",4.0,NaN,NaN


y_fraud,0.0,0.5,1.0
蚁盾分_Bin_new,,,
"(25,45]",136.0,8.0,5.0
"(45,50]",15.0,4.0,1.0
"(50,65)",1.0,NaN,NaN
"[-999,-999]",13.0,NaN,1.0
"[0,25]",358.0,14.0,16.0
"[70,75)",2.0,NaN,NaN
"[75,inf)",1.0,NaN,NaN


y_fraud,0.0,0.5,1.0
蚁盾分_Bin_new,,,
"(25,45]",166.0,10.0,4.0
"(45,50]",22.0,2.0,1.0
"[-999,-999]",5.0,NaN,1.0
"[0,25]",445.0,16.0,10.0
"[65,70)",NaN,1.0,NaN


y_fraud,0.0,0.5,1.0
蚁盾分_Bin_new,,,
"(25,45]",279.0,11.0,6.0
"(45,50]",39.0,3.0,2.0
"(50,65)",1.0,NaN,NaN
"[-999,-999]",NaN,NaN,1.0
"[0,25]",550.0,25.0,18.0


In [ ]:
# （13）更新归属渠道
# 更新2024.12 的 归属渠道
path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/4_策略分析/'
my_risk_level = pd.read_excel(path + '风险等级_更新20250103.xlsx')
my_risk_level = my_risk_level[['order_number', '归属渠道']]
my_risk_level[:2]
data_my = data_my.merge(my_risk_level, on = 'order_number', how = 'left')
data_my['归属渠道']  = np.where(data_my.下单日期 >= '2024-12-01', data_my.归属渠道_y, data_my.归属渠道_x)
data_my.drop(columns=['归属渠道_x','归属渠道_y'],inplace = True)
data_my = data_my.drop_duplicates(subset = ['order_number'])
data_my.shape
data_my[:2]

(17764, 18)

,order_number,order_id,下单日期,search_time,afz_v1_score,aff_v0_mapr_score,aff_v0_hapr_score,逾期天数,FPD,SPD,TPD,y_fraud,y_credit,蚁盾分,拒绝理由,afz_v1_score_Bin_new,蚁盾分_Bin_new,归属渠道
0,A20240901005306132,1412785,2024-09-01,2024-09,47.5799,0.289,3.118,0.0,0.0,0.0,0.0,0.0,0.0,0.855,NaN,"(45,50]","[0,25]",芝麻租物
1,A20240901005509136,1412789,2024-09-01,2024-09,57.9553,40.473,83.095,0.0,0.0,0.0,0.0,0.0,0.0,5.712,NaN,"(50,65]","[0,25]",芝麻租物


In [120]:
# （14-1）进件分布-非搜索渠道

# data_my[data_my['afz_v1_score'] == 65].shape
# data_my[data_my['蚁盾分'] == 65].shape

data_my['afz_v1_score_Bin_new'] = data_my.apply(lambda x: '[0,25]' if x.afz_v1_score >=0 and x.afz_v1_score <= 25
                                                                            else '(25,50]' if x.afz_v1_score > 25 and x.afz_v1_score  <= 50
                                                                             else '(50,65]' if x.afz_v1_score > 50 and x.afz_v1_score  <= 65
                                                                             else '(65,75)' if x.afz_v1_score > 65 and x.afz_v1_score  < 75
                                                                            #  else '(65,70)' if x.afz_v1_score > 65 and x.afz_v1_score  < 70
                                                                            #  else '[70,75)' if x.afz_v1_score >= 70 and x.afz_v1_score  < 75                                        
                                                                             else '[75,80)' if x.afz_v1_score >= 75 and x.afz_v1_score  < 80
                                                                             else '[80,inf)' if x.afz_v1_score >= 80
                                                                             else '[-999,-999]' if x.afz_v1_score == -999
                                                                             else 'other'
                                                                             ,axis = 1)

data_my['蚁盾分_Bin_new'] = data_my.apply(lambda x: '[0,25]' if x.蚁盾分 >=0 and x.蚁盾分 <= 25
                                                                             else '(25,50]' if x.蚁盾分 > 25 and x.蚁盾分  <= 50
                                                                             else '(50,65]' if x.蚁盾分 > 50 and x.蚁盾分<= 65
                                                                             else '(65,75)' if x.蚁盾分 > 65 and x.蚁盾分  < 75
                                                                            #  else '(65,70)' if x.蚁盾分 > 65 and x.蚁盾分  < 70
                                                                            #  else '[70,75)' if x.蚁盾分 >= 70 and x.蚁盾分  < 75
                                                                             else '[75,80)' if x.蚁盾分 >= 75 and x.蚁盾分  < 80
                                                                             else '[80,inf)' if x.蚁盾分 >= 80
                                                                             else '[-999,-999]' if x.蚁盾分 == -999
                                                                             else 'other'
                                                                            ,axis = 1)

pd.pivot_table(data_my[(data_my.search_time == '2024-12') & (data_my.归属渠道 == '芝麻租物')],index='下单日期',columns='afz_v1_score_Bin_new',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-12') & (data_my.归属渠道 == '芝麻租物')],index='下单日期',columns='蚁盾分_Bin_new',values='order_number',aggfunc='count')

pd.pivot_table(data_my[(data_my.search_time == '2024-12') & (~data_my.归属渠道.isin(['芝麻租物','搜索渠道']))],index='下单日期',columns='afz_v1_score_Bin_new',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-12') & (~data_my.归属渠道.isin(['芝麻租物','搜索渠道']))],index='下单日期',columns='蚁盾分_Bin_new',values='order_number',aggfunc='count')

afz_v1_score_Bin_new,"(25,50]","(50,65]","(65,75)","[-999,-999]","[0,25]","[75,80)","[80,inf)"
下单日期,,,,,,,
2024-12-01,24.0,177.0,71.0,NaN,NaN,8.0,1.0
2024-12-02,39.0,196.0,54.0,NaN,NaN,4.0,NaN
2024-12-03,26.0,224.0,59.0,NaN,NaN,4.0,2.0
2024-12-04,37.0,214.0,55.0,NaN,NaN,9.0,2.0
2024-12-05,61.0,233.0,59.0,NaN,NaN,3.0,2.0
2024-12-06,34.0,259.0,49.0,1.0,NaN,6.0,NaN
2024-12-07,34.0,239.0,45.0,NaN,NaN,2.0,4.0
2024-12-08,30.0,195.0,42.0,NaN,NaN,2.0,1.0
2024-12-09,44.0,238.0,46.0,NaN,NaN,3.0,1.0


蚁盾分_Bin_new,"(25,50]","(50,65]","(65,75)","[-999,-999]","[0,25]","[75,80)","[80,inf)"
下单日期,,,,,,,
2024-12-01,84,35,9,28,106,5,14
2024-12-02,80,33,22,27,108,6,17
2024-12-03,95,37,20,34,107,6,16
2024-12-04,89,42,11,37,105,11,22
2024-12-05,92,37,22,44,138,10,15
2024-12-06,97,37,26,45,122,4,18
2024-12-07,89,39,19,36,110,10,21
2024-12-08,78,30,13,31,98,7,13
2024-12-09,92,46,20,35,114,4,21


afz_v1_score_Bin_new,"(25,50]","(50,65]","(65,75)","[-999,-999]","[0,25]","[75,80)","[80,inf)"
下单日期,,,,,,,
2024-12-01,4.0,52.0,30.0,NaN,NaN,3.0,1.0
2024-12-02,11.0,72.0,22.0,NaN,NaN,5.0,NaN
2024-12-03,8.0,81.0,35.0,NaN,NaN,1.0,3.0
2024-12-04,13.0,90.0,21.0,NaN,NaN,4.0,1.0
2024-12-05,14.0,99.0,36.0,NaN,NaN,2.0,3.0
2024-12-06,15.0,77.0,23.0,NaN,NaN,2.0,1.0
2024-12-07,12.0,66.0,27.0,NaN,NaN,1.0,3.0
2024-12-08,9.0,55.0,17.0,1.0,NaN,3.0,2.0
2024-12-09,17.0,92.0,21.0,NaN,NaN,2.0,2.0


蚁盾分_Bin_new,"(25,50]","(50,65]","(65,75)","[-999,-999]","[0,25]","[75,80)","[80,inf)"
下单日期,,,,,,,
2024-12-01,32.0,3.0,6.0,6.0,34.0,4.0,5.0
2024-12-02,36.0,18.0,5.0,5.0,37.0,NaN,9.0
2024-12-03,34.0,19.0,9.0,15.0,39.0,2.0,10.0
2024-12-04,33.0,15.0,11.0,8.0,51.0,3.0,8.0
2024-12-05,49.0,20.0,9.0,21.0,44.0,3.0,8.0
2024-12-06,35.0,7.0,11.0,16.0,37.0,4.0,8.0
2024-12-07,27.0,17.0,7.0,15.0,30.0,3.0,10.0
2024-12-08,23.0,14.0,7.0,8.0,23.0,2.0,10.0
2024-12-09,40.0,21.0,11.0,12.0,40.0,1.0,9.0


In [114]:
# （14-2）进件分布-搜索渠道
data_my['afz_v1_score_Bin_new'] = data_my.apply(lambda x: '[0,25]' if x.afz_v1_score >=0 and x.afz_v1_score <= 25
                                                                             else '(25,45]' if x.afz_v1_score > 25 and x.afz_v1_score  <= 45
                                                                             else '(45,50]' if x.afz_v1_score > 45 and x.afz_v1_score  <= 50
                                                                             else '(50,65]' if x.afz_v1_score > 50 and x.afz_v1_score  <= 65
                                                                             else '(65,70)' if x.afz_v1_score > 65 and x.afz_v1_score  < 70
                                                                             else '[70,75)' if x.afz_v1_score >= 70 and x.afz_v1_score  < 75
                                                                             else '[75,inf)' if x.afz_v1_score >= 75
                                                                             else '[-999,-999]' if x.afz_v1_score == -999
                                                                             else 'other'
                                                                             ,axis = 1)

data_my['蚁盾分_Bin_new'] = data_my.apply(lambda x: '[0,25]' if x.蚁盾分 >=0 and x.蚁盾分 <= 25
                                                                             else '(25,45]' if x.蚁盾分 > 25 and x.蚁盾分  <= 45
                                                                             else '(45,50]' if x.蚁盾分 > 45 and x.蚁盾分  <= 50
                                                                             else '(50,65]' if x.蚁盾分 > 50 and x.蚁盾分  <= 65
                                                                             else '(65,70)' if x.蚁盾分 > 65 and x.蚁盾分  < 70
                                                                             else '[70,75)' if x.蚁盾分 >= 70 and x.蚁盾分  < 75
                                                                             else '[75,inf)' if x.蚁盾分 >= 75
                                                                             else '[-999,-999]' if x.蚁盾分 == -999
                                                                             else 'other'
                                                                             ,axis = 1)
pd.pivot_table(data_my[(data_my.search_time == '2024-12') & (data_my.归属渠道 == '搜索渠道')],index='下单日期',columns='afz_v1_score_Bin_new',values='order_number',aggfunc='count')
pd.pivot_table(data_my[(data_my.search_time == '2024-12') & (data_my.归属渠道 == '搜索渠道')],index='下单日期',columns='蚁盾分_Bin_new',values='order_number',aggfunc='count')

afz_v1_score_Bin_new,"(25,45]","(45,50]","(50,65]","(65,70)","[-999,-999]","[0,25]","[70,75)","[75,inf)"
下单日期,,,,,,,,
2024-12-01,1.0,2.0,145.0,62.0,NaN,NaN,23.0,3.0
2024-12-02,4.0,24.0,247.0,51.0,2.0,NaN,17.0,10.0
2024-12-03,3.0,28.0,239.0,42.0,NaN,NaN,16.0,9.0
2024-12-04,4.0,38.0,253.0,30.0,NaN,1.0,18.0,9.0
2024-12-05,9.0,38.0,236.0,34.0,NaN,NaN,14.0,6.0
2024-12-06,9.0,26.0,216.0,37.0,1.0,NaN,14.0,4.0
2024-12-07,1.0,29.0,207.0,30.0,NaN,NaN,12.0,2.0
2024-12-08,9.0,20.0,203.0,20.0,1.0,NaN,8.0,4.0
2024-12-09,3.0,28.0,260.0,46.0,NaN,NaN,14.0,3.0


蚁盾分_Bin_new,"(25,45]","(45,50]","(50,65]","(65,70)","[-999,-999]","[0,25]","[70,75)","[75,inf)"
下单日期,,,,,,,,
2024-12-01,64,16,28,2,8,92,3,23
2024-12-02,92,18,47,13,19,134,8,24
2024-12-03,75,18,41,9,19,146,8,21
2024-12-04,84,22,34,15,15,141,11,31
2024-12-05,83,15,40,16,10,130,11,32
2024-12-06,92,11,33,22,12,90,10,37
2024-12-07,59,12,29,16,13,113,14,25
2024-12-08,65,12,32,10,9,100,11,26
2024-12-09,102,19,42,19,20,112,7,33


In [122]:
# （15）数据导出看规则命中情况
data_my.shape
data_my[:1]
data_my.to_excel('tmp.xlsx', index = False)

(17764, 18)

,order_number,order_id,下单日期,search_time,afz_v1_score,aff_v0_mapr_score,aff_v0_hapr_score,逾期天数,FPD,SPD,TPD,y_fraud,y_credit,蚁盾分,拒绝理由,afz_v1_score_Bin_new,蚁盾分_Bin_new,归属渠道
0,A20240901005306132,1412785,2024-09-01,2024-09,47.5799,0.289,3.118,0.0,0.0,0.0,0.0,0.0,0.0,0.855,NaN,"(25,50]","[0,25]",芝麻租物


### 4_1_2.2025.3

In [ ]:
# （1）样本挑选

# 上次回溯：
# （1）2024.9-2024.11 的出库订单 --评估稳定性、区分度
# （2）2024.12.1-2024.12.17 的进件 --评估单条规则命中率、独立命中率
# 本次回溯：
# （1）2024.12.16-2025.1.31 有租后表现的 + 2025.2.1-2025.2.19 免押进件（2025.2.19 上线）

# df_202410_202502：融360 生成
df_20241216_20250219 = df_202410_202502[(df_202410_202502.下单日期 >= '2024-12-16') & (df_202410_202502.下单日期 <= '2025-02-19')]
df_20241216_20250219 = df_20241216_20250219.drop(columns = ['user_name_md5','idcard_md5','mobile_md5'])
df_20241216_20250219 = df_20241216_20250219.rename(columns = {'mobile':'user_mobile'})
# 12305
df_20241216_20250219.shape
# df_20241216_20250219[:1]

# df_202410_202502.search_time.value_counts()
# df_20241216_20250219.search_time.value_counts()

# 身份证、手机号 sha256加密
# create_date	idcard_sha256	mobile_sha256
import hashlib
 
def sha256_encryption(var):
    sha256_hash = hashlib.sha256()  # 创建一个md5对象
    sha256_hash.update(var.encode('utf-8'))  # 使用utf-8编码数据
    return sha256_hash.hexdigest()  # 返回加密后的十六进制字符串

df_20241216_20250219['id_card_num'] = df_20241216_20250219['id_card_num'].astype('str')
df_20241216_20250219['user_mobile'] = df_20241216_20250219['user_mobile'].astype('str')
df_20241216_20250219['idcard_sha256'] = df_20241216_20250219.apply(lambda x: sha256_encryption(x.id_card_num), axis = 1)
df_20241216_20250219['mobile_sha256'] = df_20241216_20250219.apply(lambda x: sha256_encryption(x.user_mobile), axis = 1)
df_20241216_20250219.shape
df_20241216_20250219.tail()

df_20241216_20250219['下单日期'] = df_20241216_20250219['下单日期'].astype('str')
# 15228562930	511902199201051212
# abca52676f514401575b35192a28d250a70e99ef12c7826f3f5018f2a8cd53fa  5794bb8b804ffcea6f90d50f3df270f227ec01e9766c561ea5f95ad79576791e
# card_id = '522121199705104611'
# sha256 = hashlib.sha256(card_id.encode())
# sha256.hexdigest()
path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/2_租赁分洞察分/2.20250320/'
df_20241216_20250219.to_excel(path + 'my_sample_20250320.xlsx', index = False)

(12401, 17)

(12401, 19)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,true_name,id_card_num,user_mobile,idcard_sha256,mobile_sha256
8886,A20250219214035159,NaN,2025-02-19,2025-02,搜索渠道,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,王毅,230303200108314314,13089583654,d02f876a2b70a8b0e5e44fffbcc2af0ccb1233eeeb125d...,3246ec579b6fc6f2920961b8cd91457a7d0f4d25da2176...
8887,A20250219215648218,NaN,2025-02-19,2025-02,芝麻租物,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,胡建峰,360281199810177417,15067955891,28c61712796bf8661529b1405ea321cd48d790238cd646...,a930b321d85ad4b82ce5c4b44d7e324095629a5762e801...
8888,A2025021922071916,NaN,2025-02-19,2025-02,芝麻租物,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,李凯,341281199806112837,17730358134,92c9c8ffe692b09673080d494edc91c89501abc54c421a...,fafa6a3c5258788abbe7eeb3384655b642266cc1ce0c5f...
8889,A20250219234808118,NaN,2025-02-19,2025-02,芝麻租物,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,罗顺莲,350481199903137042,18054951189,93188bbd025253d45996e7f423fab18cc4efdca8532292...,1ff0b048bd5ce01e4456657302c92a003a6f83851452df...
8890,A20250219113000233,NaN,2025-02-19,2025-02,搜索渠道,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,王超,370284199011190413,15726219009,24749c8393f263d36274a4af68843041deff99d22a0810...,6420487cf8d8cb0bc3c929463379bd7423e83b45799b85...


## 4_2.风险等级

In [65]:
# （1）取数
# 背景：数控风险等级没有在维护了
# 20241224 鑫宗帮忙取数
# 命中非免押用户拒绝 跟机审其他规则是串行的，若命中，则其他数据不调用了
# 综合风险等级	细化综合风险等级  蚁盾分  是 T+1 更新
# 蚂蚁数控风险等级 实时
path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/4_策略分析/'
my_risk_level = pd.read_excel(path + '风险等级_更新20241225.xlsx')
my_risk_level.shape
my_risk_level.tail()

# 2023-05-23
# my_risk_level[my_risk_level.综合风险等级.notnull()].下单日期.min()
# 2024-09-11，2024.7 蚂蚁上线该产品
# my_risk_level[my_risk_level.细化综合风险等级.notnull()].下单日期.min()
# 2023-08-12
# my_risk_level[my_risk_level.蚂蚁数控风险等级.notnull()].下单日期.min()
# 2023-01-09
# my_risk_level[my_risk_level.蚁盾分.notnull()].下单日期.min()

(227003, 11)

,order_id,trace_id,下单日期,order_number,拒绝理由,综合风险等级,细化综合风险等级,共租行为等级,履约历史等级,蚂蚁数控风险等级,蚁盾分
226998,1914457,NaN,2024-12-25,A20241225104937310,NaN,NaN,NaN,NaN,NaN,989.0,NaN
226999,1914486,NaN,2024-12-25,A20241225105352339,NaN,NaN,NaN,NaN,NaN,996.0,NaN
227000,1914556,NaN,2024-12-25,A2024122511041831,NaN,NaN,NaN,NaN,NaN,989.0,NaN
227001,1914585,NaN,2024-12-25,A2024122511070560,NaN,NaN,NaN,NaN,NaN,979.0,NaN
227002,1914661,NaN,2024-12-25,A20241225111624136,NaN,NaN,NaN,NaN,NaN,989.0,NaN


In [66]:
# （2）关联 贷后表现
my_risk_level = my_risk_level.merge(df_analy_data[['order_number','归属渠道','是否二手','是否带锁','逾期天数','FPD','SPD','TPD']], on = 'order_number', how = 'left')
my_risk_level['search_time'] = my_risk_level['下单日期'].dt.strftime('%Y-%m')
my_risk_level.shape
my_risk_level[my_risk_level.逾期天数 >= 0].shape
my_risk_level.tail()

(227003, 19)

(29092, 19)

,order_id,trace_id,下单日期,order_number,拒绝理由,综合风险等级,细化综合风险等级,共租行为等级,履约历史等级,蚂蚁数控风险等级,蚁盾分,归属渠道,是否二手,是否带锁,逾期天数,FPD,SPD,TPD,search_time
226998,1914457,NaN,2024-12-25,A20241225104937310,NaN,NaN,NaN,NaN,NaN,989.0,NaN,芝麻租物,全新,未带锁,0.0,0.0,0.0,0.0,2024-12
226999,1914486,NaN,2024-12-25,A20241225105352339,NaN,NaN,NaN,NaN,NaN,996.0,NaN,芝麻租物,全新,未带锁,0.0,0.0,0.0,0.0,2024-12
227000,1914556,NaN,2024-12-25,A2024122511041831,NaN,NaN,NaN,NaN,NaN,989.0,NaN,芝麻租物,全新,未带锁,0.0,0.0,0.0,0.0,2024-12
227001,1914585,NaN,2024-12-25,A2024122511070560,NaN,NaN,NaN,NaN,NaN,979.0,NaN,芝麻租物,全新,未带锁,0.0,0.0,0.0,0.0,2024-12
227002,1914661,NaN,2024-12-25,A20241225111624136,NaN,NaN,NaN,NaN,NaN,989.0,NaN,芝麻租物,全新,未带锁,0.0,0.0,0.0,0.0,2024-12


In [67]:
# （3）分布
my_risk_level[(my_risk_level.search_time >= '2023-05')&(my_risk_level.拒绝理由 != '命中非免押用户拒绝')].search_time.value_counts()

pd.pivot_table(my_risk_level, index='search_time',columns='综合风险等级',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level, index='search_time',columns='细化综合风险等级',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level, index='search_time',columns='履约历史等级',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level, index='search_time',columns='共租行为等级',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level, index='search_time',columns='蚂蚁数控风险等级',values='order_number',aggfunc='count')

# my_risk_level[(my_risk_level.拒绝理由.notna()) & (my_risk_level.逾期天数 >= 0)].shape
# my_risk_level[(my_risk_level.拒绝理由.notna()) & (my_risk_level.逾期天数 >= 31)]['拒绝理由'].value_counts()
# my_risk_level[(my_risk_level.拒绝理由.notna()) & (my_risk_level.逾期天数 >= 31) & (my_risk_level.拒绝理由 == '命中云商分低于515强拒(芝麻综合风险等级0豁免)')].tail()
# pd.pivot_table(my_risk_level[(my_risk_level.拒绝理由.notna()) & (my_risk_level.逾期天数 >= 31)],index='拒绝理由',columns='search_time',values='order_number',aggfunc='count')
# my_risk_level[(my_risk_level.拒绝理由.notna()) & (my_risk_level.逾期天数 >= 31)].shape
# my_risk_level[(my_risk_level.拒绝理由.notna()) & (my_risk_level.逾期天数 >= 31) & (my_risk_level.拒绝理由 == '命中云商分低于515强拒(芝麻综合风险等级0豁免)')].shape
# var = ['order_number','order_id','trace_id','下单日期','search_time','拒绝理由','综合风险等级','逾期天数']
# my_risk_level[(my_risk_level.拒绝理由.notna()) & (my_risk_level.逾期天数 >= 31) & (my_risk_level.拒绝理由 == '命中云商分低于515强拒(芝麻综合风险等级0豁免)')][var].to_excel('tmp.xlsx', index = False)

search_time
2024-09    16594
2024-12    16370
2024-11    15479
2024-10    13504
2024-08    13126
2024-07    12907
2023-11    11982
2024-06    11202
2024-04     9089
2023-10     8782
2024-05     8595
2023-12     7851
2023-08     7541
2023-09     7539
2024-01     7406
2023-07     6805
2024-03     5809
2023-06     4735
2023-05     4309
2024-02     2773
Name: count, dtype: int64

综合风险等级,-2,0,1,2,3,DEFAULT
search_time,,,,,,
2023-05,20.0,177.0,367.0,510.0,240.0,NaN
2023-06,24.0,656.0,1389.0,1722.0,854.0,NaN
2023-07,NaN,1023.0,2004.0,2450.0,953.0,NaN
2023-08,1.0,1041.0,2324.0,3023.0,910.0,NaN
2023-09,NaN,1022.0,2102.0,2437.0,602.0,1.0
2023-10,NaN,1418.0,2745.0,3224.0,865.0,NaN
2023-11,NaN,2263.0,4062.0,4341.0,971.0,NaN
2023-12,NaN,1560.0,2963.0,2956.0,202.0,1.0
2024-01,NaN,1534.0,2848.0,2740.0,152.0,NaN


细化综合风险等级,-2,0,1,2,3,4,5,6,DEFAULT
search_time,,,,,,,,,
2024-09,NaN,129.0,199.0,190.0,196.0,168.0,1.0,1.0,NaN
2024-10,1.0,118.0,201.0,197.0,186.0,179.0,NaN,1.0,NaN
2024-11,2.0,717.0,1095.0,1088.0,957.0,792.0,10.0,13.0,2.0
2024-12,NaN,735.0,1186.0,1090.0,1054.0,1591.0,12.0,9.0,NaN


履约历史等级,0.0,1.0
search_time,,
2023-05,1045,269
2023-06,3696,949
2023-07,5093,1337
2023-08,5969,1330
2023-09,5078,1086
2023-10,6820,1432
2023-11,9823,1814
2023-12,6759,923
2024-01,6527,747


共租行为等级,0.0,1.0,2.0
search_time,,,
2023-05,766.0,468.0,80.0
2023-06,2850.0,1566.0,229.0
2023-07,4235.0,1900.0,295.0
2023-08,4919.0,2153.0,227.0
2023-09,3928.0,2044.0,192.0
2023-10,5216.0,2748.0,288.0
2023-11,7643.0,3699.0,295.0
2023-12,5223.0,2431.0,28.0
2024-01,5057.0,2216.0,1.0


蚂蚁数控风险等级,0.0,969.0,975.0,979.0,989.0,996.0,998.0
search_time,,,,,,,
2023-08,NaN,137.0,609.0,540.0,173.0,398.0,77.0
2023-09,NaN,72.0,301.0,291.0,71.0,131.0,15.0
2023-10,NaN,75.0,351.0,303.0,76.0,195.0,21.0
2023-11,NaN,217.0,784.0,778.0,256.0,474.0,49.0
2023-12,NaN,108.0,479.0,531.0,175.0,312.0,54.0
2024-01,NaN,110.0,607.0,532.0,475.0,328.0,31.0
2024-02,NaN,60.0,246.0,239.0,368.0,107.0,17.0
2024-03,NaN,87.0,389.0,374.0,496.0,183.0,18.0
2024-04,6.0,172.0,807.0,676.0,839.0,352.0,25.0


In [68]:
# （4）打标签
my_risk_level['y_1'] = my_risk_level.apply(lambda x: 1 if x.逾期天数 >= 31 
                                                         else 0.5 if x.逾期天数 > 0 and x.逾期天数 < 31
                                                         else 0 if x.逾期天数 == 0
                                                         else -1, axis = 1)

my_risk_level['y_2'] = my_risk_level.apply(lambda x: 1 if x.FPD >= 7 
                                                         else 0.5 if x.FPD > 0 and x.FPD < 7
                                                         else 0 if x.FPD == 0
                                                         else -1, axis = 1)

pd.pivot_table(my_risk_level,index='search_time',columns='y_1',values='order_number',aggfunc='count')
condition_1 = (my_risk_level.y_1 >= 0)
condition_2 = (my_risk_level.y_2 >= 0)

y_1,-1.0,0.0,0.5,1.0
search_time,,,,
2022-06,6.0,NaN,NaN,NaN
2022-07,262.0,2.0,10.0,10.0
2022-08,1316.0,3.0,23.0,30.0
2022-09,1484.0,4.0,22.0,18.0
2022-10,1098.0,4.0,21.0,16.0
2022-11,690.0,8.0,22.0,13.0
2022-12,1032.0,14.0,28.0,24.0
2023-01,2551.0,41.0,88.0,91.0
2023-02,3543.0,103.0,108.0,121.0


In [69]:
# （5）综合风险等级
var_1 = '综合风险等级'
pd.pivot_table(my_risk_level[(my_risk_level.search_time >= '2023-05') & (my_risk_level.search_time <= '2024-03') & condition_1],index=var_1,columns='y_1',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-04') & condition_1],index=var_1,columns='y_1',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-04') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-05') & condition_1],index=var_1,columns='y_1',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-05') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-06') & condition_1],index=var_1,columns='y_1',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-06') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-07') & condition_1],index=var_1,columns='y_1',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-07') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-08') & condition_1],index=var_1,columns='y_1',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-08') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-09') & condition_1],index=var_1,columns='y_1',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-09') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-10') & condition_1],index=var_1,columns='y_1',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-10') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.下单日期 >= '2024-11-01') & (my_risk_level.下单日期 <= '2024-11-15') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

y_1,0.0,0.5,1.0
综合风险等级,,,
-2,NaN,3.0,1.0
0,1163.0,759.0,692.0
1,1191.0,910.0,1015.0
2,866.0,747.0,1078.0
3,56.0,65.0,98.0


y_1,0.0,0.5,1.0
综合风险等级,,,
0,383,176,119
1,283,139,135
2,153,76,94


y_2,0.0,0.5,1.0
综合风险等级,,,
0,622,42,14
1,509,29,19
2,279,21,23


y_1,0.0,0.5,1.0
综合风险等级,,,
0,372,142,99
1,215,87,77
2,128,82,45


y_2,0.0,0.5,1.0
综合风险等级,,,
0,573,28,12
1,350,16,13
2,236,13,6


...

y_1,0.0,0.5,1.0
综合风险等级,,,
0,1123.0,192.0,58.0
1,522.0,145.0,45.0
2,345.0,90.0,30.0
3,1.0,NaN,1.0


y_2,0.0,0.5,1.0
综合风险等级,,,
0,1287.0,47.0,39.0
1,634.0,44.0,34.0
2,422.0,19.0,24.0
3,1.0,NaN,1.0


y_1,0.0,0.5,1.0
综合风险等级,,,
-2,1.0,NaN,NaN
0,868.0,81.0,8.0
1,317.0,67.0,10.0
2,198.0,31.0,8.0


y_2,0.0,0.5,1.0
综合风险等级,,,
-2,1.0,NaN,NaN
0,909.0,30.0,18.0
1,355.0,22.0,17.0
2,214.0,12.0,11.0


y_2,0.0,0.5,1.0
综合风险等级,,,
0,555,23,11
1,298,18,13
2,191,16,8


In [70]:
# （6）细化综合风险等级
var_1 = '细化综合风险等级'
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-09') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-10') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.下单日期 >= '2024-11-01') & (my_risk_level.下单日期 <= '2024-11-15') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

tmp1 = my_risk_level[(my_risk_level.细化综合风险等级.notnull()) & (my_risk_level.综合风险等级.notnull()) & (my_risk_level.综合风险等级 != 'DEFAULT')][['细化综合风险等级','综合风险等级','order_number']]
tmp1['细化综合风险等级'] = tmp1['细化综合风险等级'].astype(int)
tmp1['综合风险等级'] = tmp1['综合风险等级'].astype(int)
tmp1[['细化综合风险等级', '综合风险等级']] .corr()

pd.pivot_table(tmp1,index='综合风险等级',columns='细化综合风险等级',values='order_number',aggfunc='count')

y_2,0.0,0.5,1.0
细化综合风险等级,,,
0,33.0,NaN,NaN
1,31.0,2.0,2.0
2,39.0,2.0,1.0
3,26.0,2.0,NaN
4,17.0,1.0,NaN
6,1.0,NaN,NaN


y_2,0.0,0.5,1.0
细化综合风险等级,,,
0,16.0,NaN,NaN
1,29.0,NaN,1.0
2,29.0,1.0,1.0
3,19.0,1.0,2.0
4,10.0,2.0,NaN


y_2,0.0,0.5,1.0
细化综合风险等级,,,
0,71.0,2.0,3.0
1,66.0,7.0,2.0
2,50.0,6.0,1.0
3,44.0,1.0,NaN
4,17.0,1.0,1.0
5,1.0,NaN,NaN


,细化综合风险等级,综合风险等级
细化综合风险等级,1.000000,0.297984
综合风险等级,0.297984,1.000000


细化综合风险等级,-2,0,1,2,3,4,5,6
综合风险等级,,,,,,,,
-2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,861.0,954.0,751.0,537.0,398.0,NaN,NaN
1,NaN,556.0,1036.0,960.0,898.0,936.0,NaN,NaN
2,NaN,278.0,684.0,844.0,939.0,1362.0,16.0,13.0
3,NaN,4.0,7.0,10.0,19.0,34.0,7.0,11.0


In [71]:
# （7）履约历史等级
var_1 = '履约历史等级'
pd.pivot_table(my_risk_level[(my_risk_level.search_time >= '2023-05') & (my_risk_level.search_time <= '2024-03') & condition_1],index=var_1,columns='y_1',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time >= '2023-05') & (my_risk_level.search_time <= '2024-03') & condition_1 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_1',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time >= '2023-05') & (my_risk_level.search_time <= '2023-12') & condition_1 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_1',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time >= '2024-01') & (my_risk_level.search_time <= '2024-03') & condition_1 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_1',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time >= '2023-05') & (my_risk_level.search_time <= '2023-12') & condition_1 & (my_risk_level.归属渠道 != '芝麻租物')],index=var_1,columns='y_1',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time >= '2024-01') & (my_risk_level.search_time <= '2024-03') & condition_1 & (my_risk_level.归属渠道 != '芝麻租物')],index=var_1,columns='y_1',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-04') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-05') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-06') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-07') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-08') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-09') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-10') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.下单日期 >= '2024-11-01') & (my_risk_level.下单日期 <= '2024-11-15') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-04') & condition_2 & (my_risk_level.归属渠道 != '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-05') & condition_2 & (my_risk_level.归属渠道 != '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-06') & condition_2 & (my_risk_level.归属渠道 != '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-07') & condition_2 & (my_risk_level.归属渠道 != '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-08') & condition_2 & (my_risk_level.归属渠道 != '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-09') & condition_2 & (my_risk_level.归属渠道 != '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-10') & condition_2 & (my_risk_level.归属渠道 != '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.下单日期 >= '2024-11-01') & (my_risk_level.下单日期 <= '2024-11-15') & condition_2 & (my_risk_level.归属渠道 != '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')

y_1,0.0,0.5,1.0
履约历史等级,,,
0.0,2960,2132,2492
1.0,316,352,392


y_1,0.0,0.5,1.0
履约历史等级,,,
0.0,2008,1350,1436
1.0,248,266,241


y_1,0.0,0.5,1.0
履约历史等级,,,
0.0,1375,1032,1131
1.0,199,214,215


y_1,0.0,0.5,1.0
履约历史等级,,,
0.0,633,318,305
1.0,49,52,26


y_1,0.0,0.5,1.0
履约历史等级,,,
0.0,716,635,925
1.0,68,86,150


...

y_2,0.0,0.5,1.0
履约历史等级,,,
0.0,416.0,32.0,11.0
1.0,6.0,NaN,1.0


y_2,0.0,0.5,1.0
履约历史等级,,,
0.0,565,34,25


y_2,0.0,0.5,1.0
履约历史等级,,,
0.0,747.0,36.0,41.0
1.0,3.0,NaN,NaN


y_2,0.0,0.5,1.0
履约历史等级,,,
0.0,818.0,36.0,24.0
1.0,3.0,NaN,NaN


y_2,0.0,0.5,1.0
履约历史等级,,,
0.0,562,32,19


In [72]:
# （8）共租行为等级
var_1 = '共租行为等级'
pd.pivot_table(my_risk_level[(my_risk_level.search_time >= '2023-05') & (my_risk_level.search_time <= '2024-03') & condition_1],index=var_1,columns='y_1',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-04') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-05') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-06') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-07') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-08') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-09') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-10') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.下单日期 >= '2024-11-01') & (my_risk_level.下单日期 <= '2024-11-15') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

y_1,0.0,0.5,1.0
共租行为等级,,,
0.0,2288,1700,1938
1.0,984,781,943
2.0,4,3,3


y_2,0.0,0.5,1.0
共租行为等级,,,
0.0,930.0,61.0,42.0
1.0,478.0,31.0,14.0
2.0,2.0,NaN,NaN


y_2,0.0,0.5,1.0
共租行为等级,,,
0.0,798.0,39.0,23.0
1.0,360.0,18.0,8.0
2.0,1.0,NaN,NaN


y_2,0.0,0.5,1.0
共租行为等级,,,
0.0,971,79,38
1.0,396,29,19


y_2,0.0,0.5,1.0
共租行为等级,,,
0.0,1098,70,25
1.0,466,27,13


y_2,0.0,0.5,1.0
共租行为等级,,,
0.0,1406,71,59
1.0,590,21,27


y_2,0.0,0.5,1.0
共租行为等级,,,
0.0,1636.0,81.0,73.0
1.0,703.0,28.0,25.0
2.0,5.0,1.0,NaN


y_2,0.0,0.5,1.0
共租行为等级,,,
0.0,972,44,36
1.0,507,20,10


y_2,0.0,0.5,1.0
共租行为等级,,,
0.0,702,41,24
1.0,342,16,8


In [ ]:
# （9）数控风险等级
# （9-1）总体
var_1 = '蚂蚁数控风险等级'
pd.pivot_table(my_risk_level[(my_risk_level.search_time >= '2023-05') & (my_risk_level.search_time <= '2024-03') & condition_1],index=var_1,columns='y_1',values='order_number',aggfunc='count')

pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-04') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-05') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-06') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-07') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-08') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-09') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-10') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.下单日期 >= '2024-11-01') & (my_risk_level.下单日期 <= '2024-11-15') & condition_2],index=var_1,columns='y_2',values='order_number',aggfunc='count')

y_1,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,172,85,89
975.0,647,362,410
979.0,478,441,453
989.0,493,215,154
996.0,218,207,294
998.0,21,22,31


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
0.0,2.0,NaN,NaN
969.0,94.0,4.0,3.0
975.0,414.0,19.0,11.0
979.0,319.0,29.0,28.0
989.0,422.0,25.0,6.0
996.0,166.0,15.0,8.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,90,5,2
975.0,321,16,6
979.0,277,16,11
989.0,374,12,7
996.0,105,8,5


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,96,3,1
975.0,367,29,13
979.0,328,40,19
989.0,471,27,17
996.0,129,10,10


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,144,4,2
975.0,556,35,15
979.0,372,36,12
989.0,606,29,10
996.0,111,8,7


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,140,4,2
975.0,636,35,22
979.0,416,27,33
989.0,695,24,20
996.0,112,3,8


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
0.0,60,4,3
969.0,185,8,7
975.0,711,34,25
979.0,509,32,32
989.0,741,23,16
996.0,160,11,15


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
0.0,6.0,1.0,NaN
969.0,135.0,3.0,4.0
975.0,473.0,20.0,12.0
979.0,297.0,19.0,15.0
989.0,485.0,14.0,12.0
996.0,97.0,7.0,2.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,94,6,2
975.0,354,18,8
979.0,236,18,13
989.0,300,10,5
996.0,65,5,4


In [74]:
# （9-2）芝麻租物
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-04') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-05') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-06') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-07') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-08') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-09') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-10') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.下单日期 >= '2024-11-01') & (my_risk_level.下单日期 <= '2024-11-15') & condition_2 & (my_risk_level.归属渠道 == '芝麻租物')],index=var_1,columns='y_2',values='order_number',aggfunc='count')

y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
0.0,1.0,NaN,NaN
969.0,58.0,3.0,3.0
975.0,279.0,12.0,7.0
979.0,211.0,19.0,19.0
989.0,279.0,16.0,2.0
996.0,127.0,9.0,5.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,62,4,1
975.0,228,10,5
979.0,198,11,9
989.0,270,10,4
996.0,86,6,3


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,67,3,1
975.0,263,24,9
979.0,238,28,17
989.0,353,18,11
996.0,105,9,7


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,104,1,1
975.0,420,23,9
979.0,276,26,9
989.0,423,16,7
996.0,88,5,5


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,94,1,1
975.0,461,23,14
979.0,289,17,22
989.0,495,16,17
996.0,94,2,7


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
0.0,45,2,2
969.0,119,5,4
975.0,464,24,16
979.0,357,17,21
989.0,502,18,6
996.0,118,9,8


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
0.0,4.0,NaN,NaN
969.0,63.0,NaN,2.0
975.0,208.0,5.0,5.0
979.0,140.0,10.0,7.0
989.0,191.0,9.0,5.0
996.0,56.0,4.0,2.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,49.0,1.0,1.0
975.0,178.0,9.0,5.0
979.0,106.0,9.0,5.0
989.0,111.0,4.0,NaN
996.0,38.0,2.0,2.0


In [ ]:
# （9-3）搜索渠道
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-04') & condition_2 & (my_risk_level.归属渠道 == '搜索渠道')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-05') & condition_2 & (my_risk_level.归属渠道 == '搜索渠道')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-06') & condition_2 & (my_risk_level.归属渠道 == '搜索渠道')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-07') & condition_2 & (my_risk_level.归属渠道 == '搜索渠道')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-08') & condition_2 & (my_risk_level.归属渠道 == '搜索渠道')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-09') & condition_2 & (my_risk_level.归属渠道 == '搜索渠道')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-10') & condition_2 & (my_risk_level.归属渠道 == '搜索渠道')],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.下单日期 >= '2024-11-01') & (my_risk_level.下单日期 <= '2024-11-15') & condition_2 & (my_risk_level.归属渠道 == '搜索渠道')],index=var_1,columns='y_2',values='order_number',aggfunc='count')

y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,31.0,1.0,NaN
975.0,106.0,3.0,4.0
979.0,82.0,8.0,5.0
989.0,112.0,7.0,4.0
996.0,39.0,6.0,3.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,23,1,1
975.0,79,5,1
979.0,50,3,2
989.0,79,2,1
996.0,19,2,2


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,22.0,NaN,NaN
975.0,63.0,2.0,4.0
979.0,60.0,10.0,1.0
989.0,90.0,8.0,4.0
996.0,24.0,1.0,3.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,34,2,1
975.0,111,10,4
979.0,72,9,2
989.0,154,12,2
996.0,23,3,2


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,34.0,3.0,NaN
975.0,117.0,10.0,6.0
979.0,71.0,9.0,3.0
989.0,149.0,5.0,3.0
996.0,18.0,1.0,1.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
0.0,3.0,NaN,NaN
969.0,39.0,3.0,2.0
975.0,151.0,4.0,4.0
979.0,102.0,12.0,4.0
989.0,182.0,5.0,6.0
996.0,42.0,2.0,7.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
0.0,1.0,1.0,NaN
969.0,51.0,1.0,1.0
975.0,198.0,15.0,5.0
979.0,111.0,5.0,5.0
989.0,234.0,4.0,5.0
996.0,41.0,3.0,NaN


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,40.0,5.0,NaN
975.0,128.0,5.0,2.0
979.0,100.0,7.0,2.0
989.0,139.0,6.0,5.0
996.0,27.0,3.0,2.0


In [78]:
# （9-4）非芝麻租物非搜索渠道
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-04') & condition_2 & (~my_risk_level.归属渠道.isin(['芝麻租物','搜索渠道']))],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-05') & condition_2 & (~my_risk_level.归属渠道.isin(['芝麻租物','搜索渠道']))],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-06') & condition_2 & (~my_risk_level.归属渠道.isin(['芝麻租物','搜索渠道']))],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-07') & condition_2 & (~my_risk_level.归属渠道.isin(['芝麻租物','搜索渠道']))],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-08') & condition_2 & (~my_risk_level.归属渠道.isin(['芝麻租物','搜索渠道']))],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-09') & condition_2 & (~my_risk_level.归属渠道.isin(['芝麻租物','搜索渠道']))],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.search_time == '2024-10') & condition_2 & (~my_risk_level.归属渠道.isin(['芝麻租物','搜索渠道']))],index=var_1,columns='y_2',values='order_number',aggfunc='count')
pd.pivot_table(my_risk_level[(my_risk_level.下单日期 >= '2024-11-01') & (my_risk_level.下单日期 <= '2024-11-15') & condition_2 & (~my_risk_level.归属渠道.isin(['芝麻租物','搜索渠道']))],index=var_1,columns='y_2',values='order_number',aggfunc='count')

y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
0.0,1.0,NaN,NaN
969.0,5.0,NaN,NaN
975.0,29.0,4.0,NaN
979.0,26.0,2.0,4.0
989.0,31.0,2.0,NaN


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,5.0,NaN,NaN
975.0,14.0,1.0,NaN
979.0,29.0,2.0,NaN
989.0,25.0,NaN,2.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,7.0,NaN,NaN
975.0,41.0,3.0,NaN
979.0,30.0,2.0,1.0
989.0,28.0,1.0,2.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,6.0,1.0,NaN
975.0,25.0,2.0,2.0
979.0,24.0,1.0,1.0
989.0,29.0,1.0,1.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,12.0,NaN,1.0
975.0,58.0,2.0,2.0
979.0,56.0,1.0,8.0
989.0,51.0,3.0,NaN


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
0.0,12.0,2.0,1.0
969.0,27.0,NaN,1.0
975.0,96.0,6.0,5.0
979.0,50.0,3.0,7.0
989.0,57.0,NaN,4.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
0.0,1.0,NaN,NaN
969.0,21.0,2.0,1.0
975.0,67.0,NaN,2.0
979.0,46.0,4.0,3.0
989.0,60.0,1.0,2.0


y_2,0.0,0.5,1.0
蚂蚁数控风险等级,,,
969.0,5.0,NaN,1.0
975.0,48.0,4.0,1.0
979.0,30.0,2.0,6.0
989.0,50.0,NaN,NaN


## 4_3.智租风控高级版

<!-- 
（1）智租风控高级版含 综合风险、共租风险、逾期风险 三个输出结果，芝麻免押也有这几个输出，且综合风险含专业版和普通版
（2）测试：近半年数据、订单号+申请日期+手机号/身份证号/支付宝ID，订单号、申请日期明文，手机号/身份证号/支付宝ID 三者选其一，sha256加密
（3）若通用版效果不佳，则可根据我们客群定制开发 
-->

In [77]:
# （1）样本挑选方案
# 2025.1.2挑选
df_zhizu_data = df_analy_data.copy()

df_zhizu_data['y_fraud_FPD'] = df_zhizu_data.apply(lambda x: 1 if x.FPD >= 7 
                                       else 0.5 if x.FPD > 0 and x.FPD < 7
                                       else 0, axis = 1)
df_zhizu_data['y_fraud_SPD'] = df_zhizu_data.apply(lambda x: 1 if x.SPD >= 7 
                                       else 0.5 if x.SPD > 0 and x.SPD < 7
                                       else 0, axis = 1)
df_zhizu_data['y_fraud_TPD'] = df_zhizu_data.apply(lambda x: 1 if x.TPD >= 7 
                                       else 0.5 if x.TPD > 0 and x.TPD < 7
                                       else 0, axis = 1)

# df_zhizu_data[df_zhizu_data.FPD >= 7].下单日期.max()
# df_zhizu_data[df_zhizu_data.SPD >= 7].下单日期.max()
# df_zhizu_data[df_zhizu_data.TPD >= 7].下单日期.max()
pd.pivot_table(df_zhizu_data[(df_zhizu_data.下单日期 >= '2024-07-01') & (df_zhizu_data.下单日期 <= '2024-11-23')], index='search_time',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(df_zhizu_data[(df_zhizu_data.下单日期 >= '2024-07-01') & (df_zhizu_data.下单日期 <= '2024-10-23')], index='search_time',columns='y_fraud_SPD',values='order_number',aggfunc='count')
pd.pivot_table(df_zhizu_data[(df_zhizu_data.下单日期 >= '2024-07-01') & (df_zhizu_data.下单日期 <= '2024-09-23')], index='search_time',columns='y_fraud_TPD',values='order_number',aggfunc='count')

# 标签定义
# 数据效果评估可参照 FSTPD7+为1，FSTPD0为0
# SPD>=1的样本从FPD0来，TPD>=1的样本从SPD0来，故先抽取FPD0和FPD1的样本去测试，测试回来后再做STPD的分解看STPD的效果
sample_zhizu_data = df_zhizu_data[(df_zhizu_data.下单日期 >= '2024-07-01') & (df_zhizu_data.下单日期 <= '2024-11-23') & (df_zhizu_data.y_fraud_FPD.isin([0,1]))]
sample_zhizu_data.shape

y_fraud_FPD,0.0,0.5,1.0
search_time,,,
2024-07,1858,133,70
2024-08,2037,99,110
2024-09,2384,115,100
2024-10,1497,64,46
2024-11,1711,80,49


y_fraud_SPD,0.0,0.5,1.0
search_time,,,
2024-07,1902,88,71
2024-08,2066,92,88
2024-09,2396,105,98
2024-10,1081,42,47


y_fraud_TPD,0.0,0.5,1.0
search_time,,,
2024-07,1928,72,61
2024-08,2102,73,71
2024-09,1936,68,66


(9862, 14)

In [82]:
# （2）样本三要素
# （2-1）全量三要素 姓名、身份证、手机号
#   `status` int DEFAULT NULL COMMENT '状态 1,待支付,2,待发货,3,待收货,4,租赁中,5,待归还,6,待商家收货/归还中,8,已完成,10,已退款,\r12,待审核,13,订单取消, 15,检测中',
sql = '''-- 租后应收监控  
SELECT  om.id as order_id, om.order_number, om.create_time, om.merchant_name
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
, tmu.true_name, tmu.id_card_num, tmu.mobile 
, tod.sku_attributes
, top.total_describes
, pa.name as activity_name        -- 活动名称
from db_digua_business.t_order om
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on om.id = tod.order_id
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
;
'''

df_info = query(sql)
df_info["下单日期"] = df_info['create_time'].dt.date
df_info["下单日期"] = pd.to_datetime(df_info["下单日期"],errors="coerce")

# （2-2） 关联目标客户三要素
sample_zhizu_data = sample_zhizu_data.merge(df_info[['order_number','true_name','id_card_num','mobile']], on = 'order_number', how = 'left')
sample_zhizu_data.shape

# （2—3）身份证 sha256加密 手机号不用提供
# create_date	idcard_sha256	mobile_sha256
import hashlib
 
def sha256_encryption(var):
    sha256_hash = hashlib.sha256()  # 创建一个md5对象
    sha256_hash.update(var.encode('utf-8'))  # 使用utf-8编码数据
    return sha256_hash.hexdigest()  # 返回加密后的十六进制字符串

sample_zhizu_data['id_card_num'] = sample_zhizu_data['id_card_num'].astype('str')
# sample_zhizu_data['user_mobile'] = sample_zhizu_data['mobile'].astype('str')
sample_zhizu_data['idcard_sha256'] = sample_zhizu_data.apply(lambda x: sha256_encryption(x.id_card_num), axis = 1)
# sample_zhizu_data['mobile_sha256'] = sample_zhizu_data.apply(lambda x: sha256_encryption(x.mobile), axis = 1)
sample_zhizu_data.shape
sample_zhizu_data.tail(1)

path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/3_智租风控高级版/1_2025.01/'
sample_zhizu_data['下单日期'] = sample_zhizu_data['下单日期'].astype('str')
sample_zhizu_data.to_excel(path + 'zhizu_sample_20250102.xlsx', index = False)

(9862, 18)

(9862, 18)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数,FPD,SPD,TPD,y_fraud_FPD,y_fraud_SPD,y_fraud_TPD,true_name,id_card_num,mobile,idcard_sha256
9861,A2024112322583295,1762964,2024-11-23,2024-11,芝麻租物,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0,曹磊,320982199305196534,17551540519,acb175d929471cad04da011120aa43ebee93179c07c9da...


In [230]:
# （3）返回-数据导入
# 近半年样本，FPD1-6 的没有测，看FSTpd7+
# risk_rank	multi_rank	overdue_rank	fraud_rank
path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/3_智租风控高级版/1_2025.01/2_数据返回/'
df_zz_risk = pd.read_excel(path + '202501080000000000276216样本试跑详情.xlsx')
var_1 = ['order_no','risk_rank','multi_rank','overdue_rank','fraud_rank']
df_zz_risk = df_zz_risk[var_1]
df_zz_risk = df_zz_risk.rename(columns = {'order_no':'order_number'})
df_zz_risk.shape
df_zz_risk[:1]

(9732, 5)

,order_number,risk_rank,multi_rank,overdue_rank,fraud_rank
0,A20240711095843127,RANK4,RANK4,RANK3,NaN


In [231]:
# （4）关联租后表现
df_analy_data['fraud_tmp'] = np.where(df_analy_data.FPD >= df_analy_data.SPD, df_analy_data.FPD, df_analy_data.SPD)
df_analy_data['fraud_tmp'] = np.where(df_analy_data.fraud_tmp >= df_analy_data.TPD, df_analy_data.fraud_tmp, df_analy_data.TPD)
df_analy_data['y_fraud'] = df_analy_data.apply(lambda x: 0 if x.fraud_tmp == 0 
                                               else 0.5 if x.fraud_tmp >0 and x.fraud_tmp < 7
                                               else 1 if x.fraud_tmp >= 7
                                               else 2, axis = 1)
df_analy_data['y_fraud_FPD'] = df_analy_data.apply(lambda x: 0 if x.FPD == 0 
                                               else 0.5 if x.FPD >0 and x.FPD < 7
                                               else 1 if x.FPD >= 7
                                               else 2, axis = 1)

df_analy_data.drop(columns=['fraud_tmp'],inplace = True)

df_zz_risk = df_zz_risk.merge(df_analy_data[['order_number','下单日期','search_time','y_fraud','y_fraud_FPD']], on = 'order_number', how = 'left')
df_zz_risk.shape
df_zz_risk.tail(2)

(9732, 9)

,order_number,risk_rank,multi_rank,overdue_rank,fraud_rank,下单日期,search_time,y_fraud,y_fraud_FPD
9730,A20241028133942157,RANK2,RANK3,RANK3,NaN,2024-10-28,2024-10,0.5,0.0
9731,A20241028134445177,RANK1,RANK1,RANK3,NaN,2024-10-28,2024-10,0.0,0.0


In [235]:
# （5）统计
# 2025.1.21 分析，下单日期 2024.12.11的 FPD最大表现 7+，下单日期 2024.11.11的 SPD最大表现出 7+， 下单日期 2024.10.11的 TPD最大表现出 7+
# pd.pivot_table(df_zz_risk[df_zz_risk.search_time == '2024-07'],index='risk_rank',columns='y_fraud',values='order_number',aggfunc='count') 
# pd.pivot_table(df_zz_risk[df_zz_risk.search_time == '2024-08'],index='risk_rank',columns='y_fraud',values='order_number',aggfunc='count') 
# pd.pivot_table(df_zz_risk[(df_zz_risk.下单日期 >= '2024-09-01')&(df_zz_risk.下单日期 <= '2024-10-11')],index='risk_rank',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(df_zz_risk[(df_zz_risk.下单日期 >= '2024-10-12')],index='risk_rank',columns='y_fraud_FPD',values='order_number',aggfunc='count') 

# pd.pivot_table(df_zz_risk[df_zz_risk.search_time == '2024-07'],index='multi_rank',columns='y_fraud',values='order_number',aggfunc='count') 
# pd.pivot_table(df_zz_risk[df_zz_risk.search_time == '2024-08'],index='multi_rank',columns='y_fraud',values='order_number',aggfunc='count') 
# pd.pivot_table(df_zz_risk[(df_zz_risk.下单日期 >= '2024-09-01')&(df_zz_risk.下单日期 <= '2024-10-11')],index='multi_rank',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(df_zz_risk[(df_zz_risk.下单日期 >= '2024-10-12')],index='multi_rank',columns='y_fraud_FPD',values='order_number',aggfunc='count') 

# pd.pivot_table(df_zz_risk[df_zz_risk.search_time == '2024-07'],index='overdue_rank',columns='y_fraud',values='order_number',aggfunc='count') 
# pd.pivot_table(df_zz_risk[df_zz_risk.search_time == '2024-08'],index='overdue_rank',columns='y_fraud',values='order_number',aggfunc='count') 
# pd.pivot_table(df_zz_risk[(df_zz_risk.下单日期 >= '2024-09-01')&(df_zz_risk.下单日期 <= '2024-10-11')],index='overdue_rank',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(df_zz_risk[(df_zz_risk.下单日期 >= '2024-10-12')],index='overdue_rank',columns='y_fraud_FPD',values='order_number',aggfunc='count') 

pd.pivot_table(df_zz_risk[df_zz_risk.search_time == '2024-07'],index='fraud_rank',columns='y_fraud',values='order_number',aggfunc='count') 
pd.pivot_table(df_zz_risk[df_zz_risk.search_time == '2024-08'],index='fraud_rank',columns='y_fraud',values='order_number',aggfunc='count') 
pd.pivot_table(df_zz_risk[(df_zz_risk.下单日期 >= '2024-09-01')&(df_zz_risk.下单日期 <= '2024-10-11')],index='fraud_rank',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(df_zz_risk[(df_zz_risk.下单日期 >= '2024-10-12')],index='fraud_rank',columns='y_fraud_FPD',values='order_number',aggfunc='count') 

y_fraud,0.0,0.5,1.0
fraud_rank,,,
RANK0,46,7,12


y_fraud,0.0,0.5,1.0
fraud_rank,,,
RANK0,10,1,2


y_fraud,0.0
fraud_rank,
RANK0,10


y_fraud_FPD,0.0,1.0
fraud_rank,,
RANK0,51,3


## 4_4.反欺诈洞鉴分

In [113]:
# （1）数据导入
# （9）二次导入
# 2024.9-2024.11 的出库订单 --评估稳定性、区分度
# 2024.12.1-2024.12.15 的进件 --评估单条规则命中率、独立命中率
# 2024.12.25 返回反欺诈洞鉴分、蚁盾分2.0
# 分析日期：2025.2.2
# afz_v1_score ：租赁通用分，取值 -1 表示库无记录
# 蚁盾分2.0 ：Mob1 & FPD7+
# 反欺诈洞鉴分 ：Mob1 & FPD15+
path_my = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/2_租赁分洞察分测试_20241218/'
data_my_out = pd.read_excel(path_my + 'my_sample_20241218.xlsx')
data_my_in_1 = pd.read_csv(path_my + '租赁通用分V1_迪瓜_241220.csv')
data_my_in_2 = pd.read_csv(path_my + '反欺诈标准版_迪瓜_20241220.csv')

data_my_in_1 = data_my_in_1[['certno_hash','mob_hash','apply_date','afz_v1_score']]
data_my_in_1['afz_v1_score'] = np.where(data_my_in_1.afz_v1_score == -1, -999, data_my_in_1.afz_v1_score)
data_my_in_1["apply_date"] = data_my_in_1["apply_date"].astype('str')
# data_my_in_1[data_my_in_1.afz_v1_score == -999].shape
data_my_in_1.shape
data_my_in_1 = data_my_in_1.drop_duplicates(subset = ['certno_hash','mob_hash','apply_date','afz_v1_score'])
data_my_in_1.shape

data_my_in_2 = data_my_in_2[['certno_hash','mob_hash','apply_date','aff_v0_mapr_score','aff_v0_hapr_score']]
data_my_in_2['aff_v0_mapr_score'] = np.where(data_my_in_2.aff_v0_mapr_score == -1, -999, data_my_in_2.aff_v0_mapr_score)
data_my_in_2['aff_v0_hapr_score'] = np.where(data_my_in_2.aff_v0_hapr_score == -1, -999, data_my_in_2.aff_v0_hapr_score)
data_my_in_2["apply_date"] = data_my_in_2["apply_date"].astype('str')
data_my_in_2.shape
data_my_in_2 = data_my_in_2.drop_duplicates(subset = ['certno_hash','mob_hash','apply_date','aff_v0_mapr_score','aff_v0_hapr_score'])
data_my_in_2.shape

data_my_out = data_my_out[['order_number','order_id', '下单日期', '归属渠道','idcard_sha256','mobile_sha256']]
data_my_out = data_my_out.rename(columns = {'idcard_sha256': 'certno_hash', 'mobile_sha256': 'mob_hash'})
data_my_out["apply_date"] = data_my_out["下单日期"].astype('str')
data_my_out["apply_date"] = data_my_out["apply_date"].str.replace('-','')
data_my_out['search_time'] = pd.to_datetime(data_my_out["下单日期"]).dt.strftime('%Y-%m')

data_my_out.shape


# data_my = data_my_out.merge(data_my_in_1, on = ['certno_hash','mob_hash','apply_date'], how = 'left')
# data_my = data_my.merge(data_my_in_2, on = ['certno_hash','mob_hash','apply_date'], how = 'left')
# data_my = data_my_out.merge(data_my_in_2, on = ['certno_hash','mob_hash','apply_date'], how = 'left')
# 有5条样本的身份证和手机号为空，没有在测试样本内，故用inner即可（其中 2024-09 的 4条，2024-11 的 1条）
data_my = data_my_out.merge(data_my_in_2, on = ['certno_hash','mob_hash','apply_date'], how = 'inner')
data_my = data_my.merge(data_my_in_1, on = ['certno_hash','mob_hash','apply_date'], how = 'inner')
data_my['aff_v0_mapr_score'] = data_my['aff_v0_mapr_score'].fillna(-999)
data_my['aff_v0_hapr_score'] = data_my['aff_v0_hapr_score'].fillna(-999)
data_my['afz_v1_score'] = data_my['afz_v1_score'].fillna(-999)
data_my[data_my.aff_v0_mapr_score == -999].shape
data_my[data_my.aff_v0_hapr_score == -999].shape
data_my[data_my.afz_v1_score == -999].shape
data_my.drop(columns=['apply_date','certno_hash','mob_hash'],inplace = True)
data_my.shape
data_my[:2]

del data_my_in_1
del data_my_in_2
del data_my_out

(17759, 4)

(17757, 4)

(17759, 5)

(17757, 5)

(17764, 8)

(0, 11)

(0, 11)

(16, 11)

(17759, 8)

,order_number,order_id,下单日期,归属渠道,search_time,aff_v0_mapr_score,aff_v0_hapr_score,afz_v1_score
0,A20240901005306132,1412785,2024-09-01,芝麻租物,2024-09,0.289,3.118,47.5799
1,A20240901005509136,1412789,2024-09-01,芝麻租物,2024-09,40.473,83.095,57.9553


In [114]:
# （2）关联租后表现
data_my = data_my.merge(df_analy_data[['order_number','FPD']], on = 'order_number', how = 'left')
data_my['y_fraud'] = data_my.apply(lambda x: 1 if x.FPD >= 15 
                                       else 0.5 if x.FPD > 0 and x.FPD < 15
                                       else 0 if x.FPD == 0
                                       else 2 , axis = 1)
data_my.shape
data_my[:2]
pd.pivot_table(data_my,index='search_time',columns='y_fraud',values='order_number',aggfunc='count')

(17759, 10)

,order_number,order_id,下单日期,归属渠道,search_time,aff_v0_mapr_score,aff_v0_hapr_score,afz_v1_score,FPD,y_fraud
0,A20240901005306132,1412785,2024-09-01,芝麻租物,2024-09,0.289,3.118,47.5799,0.0,0.0
1,A20240901005509136,1412789,2024-09-01,芝麻租物,2024-09,40.473,83.095,57.9553,0.0,0.0


y_fraud,0.0,0.5,1.0,2.0
search_time,,,,
2024-09,2379.0,138.0,78.0,NaN
2024-10,1497.0,73.0,37.0,NaN
2024-11,2332.0,115.0,54.0,NaN
2024-12,1519.0,88.0,39.0,9410.0


In [115]:
# （3）更新渠道
# 更新2024.12 的 归属渠道
path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/4_现有策略分析/'
my_risk_level = pd.read_excel(path + '风险等级_更新20250103.xlsx')
my_risk_level = my_risk_level[['order_number', '归属渠道']]
my_risk_level[:2]
data_my = data_my.merge(my_risk_level, on = 'order_number', how = 'left')
data_my['归属渠道']  = np.where(data_my.下单日期 >= '2024-12-01', data_my.归属渠道_y, data_my.归属渠道_x)
data_my.drop(columns=['归属渠道_x','归属渠道_y'],inplace = True)
data_my = data_my.drop_duplicates(subset = ['order_number'])
data_my.shape
data_my[:2]

,order_number,归属渠道
0,A202411010000411,芝麻租物
1,A2024110100085515,搜索渠道


(17759, 10)

,order_number,order_id,下单日期,search_time,aff_v0_mapr_score,aff_v0_hapr_score,afz_v1_score,FPD,y_fraud,归属渠道
0,A20240901005306132,1412785,2024-09-01,2024-09,0.289,3.118,47.5799,0.0,0.0,芝麻租物
1,A20240901005509136,1412789,2024-09-01,2024-09,40.473,83.095,57.9553,0.0,0.0,芝麻租物


In [116]:
# （3）分箱
# data_my['aff_v0_mapr_score_Bin_new'] = data_my.apply(lambda x: '20.[0,25]' if x.aff_v0_mapr_score >=0 and x.aff_v0_mapr_score <= 25
#                                                                              else '19.(25,45]' if x.aff_v0_mapr_score > 25 and x.aff_v0_mapr_score  <= 45
#                                                                              else '18.(45,50]' if x.aff_v0_mapr_score > 45 and x.aff_v0_mapr_score  <= 50
#                                                                              else '17.(50,65)' if x.aff_v0_mapr_score > 50 and x.aff_v0_mapr_score  < 65
#                                                                              else '16.[65,70)' if x.aff_v0_mapr_score >= 65 and x.aff_v0_mapr_score  < 70
#                                                                              else '15.[70,75)' if x.aff_v0_mapr_score >= 70 and x.aff_v0_mapr_score  < 75
#                                                                              else '14.[75,80)' if x.aff_v0_mapr_score >= 75 and x.aff_v0_mapr_score < 80
#                                                                              else '13.[80,85)' if x.aff_v0_mapr_score >= 80 and x.aff_v0_mapr_score < 85
#                                                                              else '12.[85,90)' if x.aff_v0_mapr_score >= 85 and x.aff_v0_mapr_score < 90
#                                                                              else '11.[90,95)' if x.aff_v0_mapr_score >= 90 and x.aff_v0_mapr_score < 95
#                                                                              else '10.[95,100]' if x.aff_v0_mapr_score >= 95
#                                                                              else '[-999,-999]' if x.aff_v0_mapr_score == -999
#                                                                              else 'other'
#                                                                              ,axis = 1)
# data_my['aff_v0_hapr_score_Bin_new'] = data_my.apply(lambda x: '20.[0,25]' if x.aff_v0_hapr_score >=0 and x.aff_v0_hapr_score <= 25
#                                                                              else '19.(25,45]' if x.aff_v0_hapr_score > 25 and x.aff_v0_hapr_score  <= 45
#                                                                              else '18.(45,50]' if x.aff_v0_hapr_score > 45 and x.aff_v0_hapr_score  <= 50
#                                                                              else '17.(50,65)' if x.aff_v0_hapr_score > 50 and x.aff_v0_hapr_score  < 65
#                                                                              else '16.[65,70)' if x.aff_v0_hapr_score >= 65 and x.aff_v0_hapr_score  < 70
#                                                                              else '15.[70,75)' if x.aff_v0_hapr_score >= 70 and x.aff_v0_hapr_score  < 75
#                                                                              else '14.[75,80)' if x.aff_v0_hapr_score >= 75 and x.aff_v0_hapr_score < 80
#                                                                              else '13.[80,85)' if x.aff_v0_hapr_score >= 80 and x.aff_v0_hapr_score < 85
#                                                                              else '12.[85,90)' if x.aff_v0_hapr_score >= 85 and x.aff_v0_hapr_score < 90
#                                                                              else '11.[90,95)' if x.aff_v0_hapr_score >= 90 and x.aff_v0_hapr_score < 95
#                                                                              else '10.[95,100]' if x.aff_v0_hapr_score >= 95
#                                                                              else '[-999,-999]' if x.aff_v0_hapr_score == -999
#                                                                              else 'other'
#                                                                              ,axis = 1)
data_my['aff_v0_mapr_score_Bin_new'] = data_my.apply(lambda x: '19.[0,25]' if x.aff_v0_mapr_score >=0 and x.aff_v0_mapr_score <= 25
                                                                             else '18.(25,45]' if x.aff_v0_mapr_score > 25 and x.aff_v0_mapr_score  <= 45
                                                                             else '17.(45,50]' if x.aff_v0_mapr_score > 45 and x.aff_v0_mapr_score  <= 50
                                                                             else '16.(50,65)' if x.aff_v0_mapr_score > 50 and x.aff_v0_mapr_score  < 65
                                                                             else '15.[65,70)' if x.aff_v0_mapr_score >= 65 and x.aff_v0_mapr_score  < 70
                                                                             else '14.[70,75)' if x.aff_v0_mapr_score >= 70 and x.aff_v0_mapr_score  < 75
                                                                             else '13.[75,80)' if x.aff_v0_mapr_score >= 75 and x.aff_v0_mapr_score < 80
                                                                             else '12.[80,85)' if x.aff_v0_mapr_score >= 80 and x.aff_v0_mapr_score < 85
                                                                             else '11.[85,90)' if x.aff_v0_mapr_score >= 85 and x.aff_v0_mapr_score < 90
                                                                             else '10.[90,100]' if x.aff_v0_mapr_score >= 90
                                                                             else '[-999,-999]' if x.aff_v0_mapr_score == -999
                                                                             else 'other'
                                                                             ,axis = 1)
data_my['aff_v0_hapr_score_Bin_new'] = data_my.apply(lambda x: '19.[0,25]' if x.aff_v0_hapr_score >=0 and x.aff_v0_hapr_score <= 25
                                                                             else '18.(25,45]' if x.aff_v0_hapr_score > 25 and x.aff_v0_hapr_score  <= 45
                                                                             else '17.(45,50]' if x.aff_v0_hapr_score > 45 and x.aff_v0_hapr_score  <= 50
                                                                             else '16.(50,65)' if x.aff_v0_hapr_score > 50 and x.aff_v0_hapr_score  < 65
                                                                             else '15.[65,70)' if x.aff_v0_hapr_score >= 65 and x.aff_v0_hapr_score  < 70
                                                                             else '14.[70,75)' if x.aff_v0_hapr_score >= 70 and x.aff_v0_hapr_score  < 75
                                                                             else '13.[75,80)' if x.aff_v0_hapr_score >= 75 and x.aff_v0_hapr_score < 80
                                                                             else '12.[80,85)' if x.aff_v0_hapr_score >= 80 and x.aff_v0_hapr_score < 85
                                                                             else '11.[85,90)' if x.aff_v0_hapr_score >= 85 and x.aff_v0_hapr_score < 90
                                                                             else '10.[90,100]' if x.aff_v0_hapr_score >= 90
                                                                             else '[-999,-999]' if x.aff_v0_hapr_score == -999
                                                                             else 'other'
                                                                             ,axis = 1)

In [106]:
# （4）统计-aff_v0_mapr_score
# （4-1）总体
# pd.pivot_table(data_my[data_my.search_time == '2024-09'],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[data_my.search_time == '2024-10'],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[data_my.search_time == '2024-11'],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[data_my.search_time == '2024-12'],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

# （4-2）芝麻租物
# condition_3 = (data_my.归属渠道 == '芝麻租物')
# pd.pivot_table(data_my[(data_my.search_time == '2024-09') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-10') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-11') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-12') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

# （4-3）芝麻租物
# condition_3 = (data_my.归属渠道 == '搜索渠道')
# pd.pivot_table(data_my[(data_my.search_time == '2024-09') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-10') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-11') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-12') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

# （4-4）非芝麻非搜索
# condition_3 = (~data_my.归属渠道.isin(['芝麻租物','搜索渠道']))
# pd.pivot_table(data_my[(data_my.search_time == '2024-09') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-10') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-11') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-12') & condition_3],index='aff_v0_mapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

y_fraud,0.0,0.5,1.0
aff_v0_mapr_score_Bin_new,,,
"10.[90,100]",20.0,3.0,3.0
"11.[85,90)",7.0,1.0,3.0
"12.[80,85)",7.0,3.0,3.0
"13.[75,80)",10.0,1.0,1.0
"14.[70,75)",21.0,1.0,1.0
"15.[65,70)",2.0,1.0,1.0
"16.(50,65)",49.0,1.0,NaN
"17.(45,50]",14.0,1.0,1.0
"18.(25,45]",47.0,2.0,2.0


y_fraud,0.0,0.5,1.0
aff_v0_mapr_score_Bin_new,,,
"10.[90,100]",5.0,NaN,NaN
"11.[85,90)",8.0,2.0,NaN
"12.[80,85)",4.0,2.0,NaN
"13.[75,80)",7.0,1.0,1.0
"14.[70,75)",7.0,NaN,NaN
"15.[65,70)",1.0,NaN,NaN
"16.(50,65)",37.0,3.0,1.0
"17.(45,50]",12.0,1.0,NaN
"18.(25,45]",50.0,2.0,NaN


y_fraud,0.0,0.5,1.0
aff_v0_mapr_score_Bin_new,,,
"10.[90,100]",18.0,4.0,1.0
"11.[85,90)",16.0,NaN,NaN
"12.[80,85)",19.0,2.0,NaN
"13.[75,80)",20.0,NaN,1.0
"14.[70,75)",18.0,3.0,2.0
"15.[65,70)",7.0,NaN,NaN
"16.(50,65)",43.0,4.0,2.0
"17.(45,50]",16.0,NaN,1.0
"18.(25,45]",63.0,1.0,2.0


y_fraud,0.0,0.5,1.0,2.0
aff_v0_mapr_score_Bin_new,,,,
"10.[90,100]",21.0,5.0,3.0,305.0
"11.[85,90)",10.0,1.0,1.0,142.0
"12.[80,85)",5.0,2.0,1.0,120.0
"13.[75,80)",7.0,2.0,NaN,112.0
"14.[70,75)",13.0,1.0,NaN,123.0
"15.[65,70)",3.0,NaN,NaN,43.0
"16.(50,65)",30.0,1.0,2.0,224.0
"17.(45,50]",5.0,NaN,NaN,50.0
"18.(25,45]",21.0,2.0,3.0,221.0


In [110]:
# （5）统计-aff_v0_hapr_score
# （5-1）总体
# pd.pivot_table(data_my[data_my.search_time == '2024-09'],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[data_my.search_time == '2024-10'],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[data_my.search_time == '2024-11'],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[data_my.search_time == '2024-12'],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

# （5-）芝麻租物
# condition_3 = (data_my.归属渠道 == '芝麻租物')
# pd.pivot_table(data_my[(data_my.search_time == '2024-09') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-10') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-11') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-12') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

# （5-3）芝麻租物
# condition_3 = (data_my.归属渠道 == '搜索渠道')
# pd.pivot_table(data_my[(data_my.search_time == '2024-09') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-10') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-11') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-12') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

# （5-4）非芝麻非搜索
# condition_3 = (~data_my.归属渠道.isin(['芝麻租物','搜索渠道']))
# pd.pivot_table(data_my[(data_my.search_time == '2024-09') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-10') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-11') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')
# pd.pivot_table(data_my[(data_my.search_time == '2024-12') & condition_3],index='aff_v0_hapr_score_Bin_new',columns='y_fraud',values='order_number',aggfunc='count')

y_fraud,0.0,0.5,1.0
aff_v0_hapr_score_Bin_new,,,
"10.[90,100]",54.0,6.0,3.0
"11.[85,90)",35.0,2.0,1.0
"12.[80,85)",37.0,1.0,NaN
"13.[75,80)",19.0,1.0,1.0
"14.[70,75)",35.0,3.0,NaN
"15.[65,70)",19.0,1.0,1.0
"16.(50,65)",82.0,3.0,2.0
"17.(45,50]",24.0,1.0,2.0
"18.(25,45]",108.0,7.0,2.0


y_fraud,0.0,0.5,1.0
aff_v0_hapr_score_Bin_new,,,
"10.[90,100]",88.0,4.0,4.0
"11.[85,90)",22.0,6.0,1.0
"12.[80,85)",33.0,3.0,1.0
"13.[75,80)",48.0,2.0,NaN
"14.[70,75)",34.0,2.0,NaN
"15.[65,70)",21.0,NaN,NaN
"16.(50,65)",100.0,3.0,4.0
"17.(45,50]",36.0,2.0,NaN
"18.(25,45]",135.0,7.0,1.0


y_fraud,0.0,0.5,1.0
aff_v0_hapr_score_Bin_new,,,
"10.[90,100]",144.0,11.0,6.0
"11.[85,90)",88.0,6.0,2.0
"12.[80,85)",74.0,NaN,1.0
"13.[75,80)",66.0,2.0,1.0
"14.[70,75)",63.0,3.0,2.0
"15.[65,70)",35.0,4.0,2.0
"16.(50,65)",144.0,12.0,6.0
"17.(45,50]",41.0,4.0,1.0
"18.(25,45]",168.0,6.0,2.0


y_fraud,0.0,0.5,1.0,2.0
aff_v0_hapr_score_Bin_new,,,,
"10.[90,100]",104.0,6.0,2.0,1143.0
"11.[85,90)",47.0,2.0,1.0,401.0
"12.[80,85)",41.0,1.0,2.0,325.0
"13.[75,80)",43.0,2.0,1.0,266.0
"14.[70,75)",42.0,1.0,1.0,203.0
"15.[65,70)",25.0,1.0,1.0,172.0
"16.(50,65)",85.0,6.0,3.0,515.0
"17.(45,50]",31.0,NaN,NaN,149.0
"18.(25,45]",107.0,4.0,6.0,489.0


In [117]:
# （6）相关性
# 与蚁盾分、蚁盾分2.0的相关性
data_my[['aff_v0_mapr_score', 'aff_v0_hapr_score', 'afz_v1_score']].corr()

,aff_v0_mapr_score,aff_v0_hapr_score,afz_v1_score
aff_v0_mapr_score,1.000000,0.423556,0.046247
aff_v0_hapr_score,0.423556,1.000000,0.033684
afz_v1_score,0.046247,0.033684,1.000000


# 5_融360数据产品

## 5_1.202412

<!-- 背景：
新产品，20241114 提供测试样本，20241206返回测试数据，本次返回6个数据产品
（占御全息欺诈分、综合行为评估V2.0、综合行为评估报告V2.4、占信分臻藏版-支付风险、占信分标准版-支付风险、占信分标准版-消费风险） -->

In [170]:
# （1）测试数据导入
# 1-占御全息欺诈分
path_360 = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/维客/'
data_360_1 = pd.read_excel(path_360 + '维客壹佰-200993占御全息欺诈报告V2.1-99.99%.xlsx')
data_360_2 = pd.read_excel(path_360 + '维客壹佰-201046综合行为评估报告V2.0-99.94%.xlsx')
data_360_3 = pd.read_excel(path_360 + '维客壹佰-201053综合行为评估报告V2.4-100%.xlsx')
data_360_4 = pd.read_excel(path_360 + '维客壹佰-201048占信分臻藏版-支付风险V3.0-100%.xlsx')
data_360_5 = pd.read_excel(path_360 + '维客壹佰-202124占信分标准版-支付风险V3.0-100%.xlsx')
data_360_6 = pd.read_excel(path_360 + '维客壹佰-202123占信分标准版-消费风险V3.0-100%.xlsx')

data_360_1 = data_360_1.rename(columns = {'客户编号':'order_number', 'score':'score_zy'})
data_360_1 = data_360_1[['order_number','score_zy']]

data_360_2 = data_360_2.rename(columns = {'客户编号':'order_number', 'score':'score_zhxw0'})
data_360_2 = data_360_2[['order_number','score_zhxw0']]

data_360_3 = data_360_3.rename(columns = {'客户编号':'order_number', 'score':'score_zhxw4'})
data_360_3 = data_360_3[['order_number','score_zhxw4']]

data_360_4 = data_360_4.rename(columns = {'客户编号':'order_number', 'score':'score_zxfzcb'})
data_360_4 = data_360_4[['order_number','score_zxfzcb']]

data_360_5 = data_360_5.rename(columns = {'客户编号':'order_number', 'score':'score_zxfbzb'})
data_360_5 = data_360_5[['order_number','score_zxfbzb']]

data_360_6 = data_360_6.rename(columns = {'客户编号':'order_number', 'score':'score_zxfbzb_xfrisk'})
data_360_6 = data_360_6[['order_number','score_zxfbzb_xfrisk']]


data_360_1.shape
data_360_2.shape
data_360_3.shape
data_360_4.shape
data_360_5.shape
data_360_6.shape

data_360_score =  data_360_1.merge(data_360_2, on = 'order_number', how = 'inner')
data_360_score =  data_360_score.merge(data_360_3, on = 'order_number', how = 'inner')
data_360_score =  data_360_score.merge(data_360_4, on = 'order_number', how = 'inner')
data_360_score =  data_360_score.merge(data_360_5, on = 'order_number', how = 'inner')
data_360_score =  data_360_score.merge(data_360_6, on = 'order_number', how = 'inner')

data_360_score = data_360_score.fillna(-999)
data_360_score.shape
data_360_score[:2]

(5000, 2)

(5000, 2)

(5000, 2)

(5000, 2)

(5000, 2)

(5000, 2)

(5000, 7)

,order_number,score_zy,score_zhxw0,score_zhxw4,score_zxfzcb,score_zxfbzb,score_zxfbzb_xfrisk
0,A202301020249553,546.0,479.0,459,498,449,495
1,A202301020323036,627.0,506.0,522,411,464,537


In [171]:
# （2）测试数据关联贷后表现
data_360_score = data_360_score.merge(df_analy_data[['order_number', '下单日期', 'search_time', '逾期天数','FPD']], on = 'order_number', how = 'left')
data_360_score.shape
data_360_score[:1]

(5000, 11)

,order_number,score_zy,score_zhxw0,score_zhxw4,score_zxfzcb,score_zxfbzb,score_zxfbzb_xfrisk,下单日期,search_time,逾期天数,FPD
0,A202301020249553,546.0,479.0,459,498,449,495,2023-01-02,2023-01,4,2.0


In [173]:
# （3）样本分析
data_360_score['y_fraud'] = data_360_score.apply(lambda x: 1 if x.FPD >= 31 
                                       else 0.5 if x.FPD > 0 and x.FPD < 31
                                       else 0, axis = 1)
data_360_score['y_credit'] = data_360_score.apply(lambda x: 1 if x.逾期天数 >= 31 
                                       else 0.5 if x.逾期天数 > 0 and x.逾期天数 < 31
                                       else 0, axis = 1)

df_analy_data['y_fraud_31'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 31 
                                       else 0.5 if x.FPD > 0 and x.FPD < 31
                                       else 0, axis = 1)
# df_analy_data['y_credit'] = df_analy_data.apply(lambda x: 1 if x.逾期天数 >= 31 
#                                        else 0.5 if x.逾期天数 > 0 and x.逾期天数 < 31
#                                        else 0, axis = 1)
# data_360_score['y'].value_counts()
# data_360_score[data_360_score.逾期天数 > 0].shape
pd.pivot_table(data_360_score,index='search_time',columns='y_fraud',values='order_number',aggfunc='count')
pd.pivot_table(data_360_score,index='search_time',columns='y_credit',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_data,index='search_time',columns='y_fraud_31',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_data,index='search_time',columns='y_credit',values='order_number',aggfunc='count')

data_360_score_train = data_360_score[data_360_score.search_time >= '2024-01']
data_360_score_train.shape	

y_fraud,0.0,0.5,1.0
search_time,,,
2023-01,21,21,4
2023-02,20,27,10
2023-03,33,13,4
2023-04,86,42,12
2023-05,65,35,7
2023-06,72,49,7
2023-07,81,72,10
2023-08,78,53,13
2023-09,53,47,13


y_credit,0.0,0.5,1.0
search_time,,,
2023-01,NaN,12.0,34.0
2023-02,NaN,14.0,43.0
2023-03,NaN,13.0,37.0
2023-04,NaN,51.0,89.0
2023-05,NaN,29.0,78.0
2023-06,NaN,34.0,94.0
2023-07,NaN,34.0,129.0
2023-08,NaN,29.0,115.0
2023-09,NaN,24.0,89.0


y_fraud_31,0.0,0.5,1.0
search_time,,,
2022-07,18.0,2.0,2.0
2022-08,34.0,15.0,7.0
2022-09,35.0,6.0,3.0
2022-10,34.0,5.0,2.0
2022-11,31.0,8.0,4.0
2022-12,53.0,8.0,5.0
2023-01,168.0,41.0,12.0
2023-02,258.0,47.0,27.0
2023-03,276.0,18.0,22.0


y_credit,0.0,0.5,1.0
search_time,,,
2022-07,2.0,10.0,10.0
2022-08,3.0,23.0,30.0
2022-09,4.0,22.0,18.0
2022-10,4.0,21.0,16.0
2022-11,8.0,22.0,13.0
2022-12,14.0,28.0,24.0
2023-01,41.0,88.0,92.0
2023-02,103.0,108.0,121.0
2023-03,112.0,81.0,123.0


In [175]:
# （4）分箱
%run C:/Users/Administrator/Desktop/mine/4_专题分析/1_回捞分析/2.model_func.ipynb
continous_merged_dict = {}
var_bin_list = []
trainData_kf_Bin = data_360_score_train[data_360_score_train.y_fraud.isin([0,1])]
trainData_kf_Bin['y'] = trainData_kf_Bin['y_fraud']
numeric_desc_var = ['score_zy']

# start_time = time.time()

for col in numeric_desc_var:
    print ("{} is in processing".format(col))
    if -1111 not in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]): 
        max_interval = 5 
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval,special_attribute=[],minBinPcnt=0)
        trainData_kf_Bin[col+'_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff,special_attribute=[]))
        monotone = BadRateMonotone(trainData_kf_Bin, col+'_Bin', 'y')
        while(not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute = [],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = []))
            if max_interval == 2:
                break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y')
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [])) 
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    else:
        max_interval = 7 #正常取值最多分5箱
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval = max_interval, special_attribute = [-1111, -999],
                                      minBinPcnt = 0)
        trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1111, -999]))
        monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2'])
        while (not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute=[-1111, -999],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))
            if max_interval == 3:
                break
#             if max_interval == 3 and -1111 in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 3 and -1111 not in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 4 and -1111 in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2']) 
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))  
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    continous_merged_dict[col] = cutOff
    
# print("- Time consumed: %s seconds" % int(time.time() - start_time))

score_zy is in processing


In [176]:
# （5）改变映射方式
trainData_kf_Bin_1 = data_360_score_train[data_360_score_train.y_fraud.isin([0,1])]
trainData_kf_Bin_1['y'] = trainData_kf_Bin_1['y_fraud']

# start_time = time.time()
for col in numeric_desc_var:
    if -1111 not in set(trainData_kf_Bin_1[col]) and -999 not in set(trainData_kf_Bin_1[col]):
        trainData_kf_Bin_1[col+'_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col]))
        del trainData_kf_Bin_1[col]
    else:
        trainData_kf_Bin_1[col + '_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col],[-1111, -999]))
        del trainData_kf_Bin_1[col]
        
# print("- Time consumed: %s seconds" % int(time.time() - start_time))

In [177]:
# （6）结果统计
data_360_score_train['y'] = data_360_score_train['y_fraud']
data_360_score_train['score_zy_Bin_new'] = data_360_score_train.apply(lambda x: '[0,450]' if x.score_zy >=0 and x.score_zy <= 450
                                                                             else '(450,500]' if x.score_zy > 450 and x.score_zy  <= 500
                                                                             else '(500,525]' if x.score_zy > 500 and x.score_zy  <= 525
                                                                             else '(525,550]' if x.score_zy > 525 and x.score_zy  <= 550
                                                                             else '(550,575]' if x.score_zy > 550 and x.score_zy  <= 575
                                                                             else '(575,inf)' if x.score_zy > 575
                                                                             else '[-999,-999]' if x.score_zy == -999
                                                                             else 'other'
                                                                             ,axis = 1)
pd.pivot_table(trainData_kf_Bin_1,index='score_zy_Bin',columns='y',values='order_number',aggfunc='count')
pd.pivot_table(data_360_score_train,index='score_zy_Bin_new',columns='y',values='order_number',aggfunc='count')

y,0.0,1.0
score_zy_Bin,,
"(560.0,inf)",1155,5
"[0,560.0]",2078,25


y,0.0,0.5,1.0
score_zy_Bin_new,,,
"(450,500]",669,76,9
"(500,525]",502,40,6
"(525,550]",465,46,5
"(550,575]",467,40,3
"(575,inf)",882,59,4
"[0,450]",248,19,3


In [269]:
# （7）重新提供样本
# 本次测试有欺诈和信用产品
# 2023.7-2024.9 有贷后表现的订单

# （7-1）全量三要素 姓名、身份证、手机号
#   `status` int DEFAULT NULL COMMENT '状态 1,待支付,2,待发货,3,待收货,4,租赁中,5,待归还,6,待商家收货/归还中,8,已完成,10,已退款,\r12,待审核,13,订单取消, 15,检测中',
sql = '''-- 租后应收监控  
SELECT  om.id as order_id, om.order_number, om.create_time, om.merchant_name
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
, tmu.true_name, tmu.id_card_num, tmu.mobile 
, tod.sku_attributes
, top.total_describes
, pa.name as activity_name        -- 活动名称
from db_digua_business.t_order om
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on om.id = tod.order_id
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
;
'''

df_info = query(sql)
df_info["下单日期"] = df_info['create_time'].dt.date
df_info["下单日期"] = pd.to_datetime(df_info["下单日期"],errors="coerce")

# （7-2） 关联目标客户三要素
df_analy_data = df_analy_data.merge(df_info[['order_number','true_name','id_card_num','mobile']], on = 'order_number', how = 'left')
df_analy_data.shape

df_202307_ = df_analy_data[(df_analy_data.search_time >= '2023-07') & (df_analy_data.search_time <= '2024-09')]
df_202307_.shape
df_202307_[df_202307_.id_card_num.isna()].shape
df_202307_[df_202307_.mobile.isna()].shape

df_202307_ = df_202307_[df_202307_.id_card_num.notnull()]
df_202307_.shape

# （7-3）md5加密
import hashlib
 
def md5_encryption(var):
    md5 = hashlib.md5()  # 创建一个md5对象
    md5.update(var.encode('utf-8'))  # 使用utf-8编码数据
    return md5.hexdigest()  # 返回加密后的十六进制字符串

df_202307_['user_name_md5'] = df_202307_.apply(lambda x: md5_encryption(x.true_name), axis = 1)
df_202307_['idcard_md5'] = df_202307_.apply(lambda x: md5_encryption(x.id_card_num), axis = 1)
df_202307_['mobile_md5'] = df_202307_.apply(lambda x: md5_encryption(x.mobile), axis = 1)
df_202307_.shape
df_202307_.tail()

# # card_id = '533526200104110620'
# # md5 = hashlib.md5(card_id.encode())
# # md5.hexdigest()

df_202307_['下单日期'] = df_202307_['下单日期'].astype('str')
path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/2_20241218/'
df_202307_.to_excel(path + 'ceshi_sample_20241218.xlsx', index = False)

(19670, 15)

(1, 15)

(0, 15)

(19669, 15)

(19669, 18)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数,FPD,y_fraud,y_credit,y_fraud_31,true_name,id_card_num,mobile,user_name_md5,idcard_md5,mobile_md5
22516,A2024093018321187,1545277,2024-09-30,2024-09,芝麻租物,全新,未带锁,0,0.0,0.0,0.0,0.0,吴梦媛,120104199910013548,17695738480,cb838c9c9b2f1cc9925c96f38bd26a7a,93d69d75e6a1e50bc3d0328977d14a84,26c8ba0e6e8ba21b4fb7d50ea4b1dec1
22517,A20240930184225114,1545304,2024-09-30,2024-09,芝麻租物,全新,未带锁,0,0.0,0.0,0.0,0.0,刘韵茹,430102199905045529,17673119842,dd5a65e3629e380c24fb13510a19054b,ed48a2f8c4d11cb89b2a5fef2b169086,e6e595eed6c5758c131289060b56db3a
22518,A20240930185759156,1545346,2024-09-30,2024-09,搜索渠道,全新,未带锁,0,0.0,0.0,0.0,0.0,朱冬娣,331021199006043467,15355988899,41aa81d405288e2521b0a256680d4f82,160645b8973e0e65a5bf9b6af2f171be,5a0083be1e8aeea6dbadf8ccc66fed5d
22519,A202409301904457,1545358,2024-09-30,2024-09,芝麻租物,全新,未带锁,0,0.0,0.0,0.0,0.0,郭磊,321282199504143016,15052904387,68d2121c670bda04dbb69d3e0b612bc7,a2c6251c80218f90d8ca084554551fc2,d54140cdb333dd6f27b05224f7aa8275
22520,A2024093022470182,1545745,2024-09-30,2024-09,付费灯火,二手,未带锁,1,0.0,0.0,0.5,0.0,卢帆,45242720010921391X,18677457721,43f055ba12917d34892b310d628c1416,98632eba6740f81e1424de76b77ea0fe,17a72e1468f1e181e98b14187d8e89a9


## 5_2.202502

In [250]:
# （1）测试数据读入
# 20250203开始分析
path_360 = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/2_20241218/1.样本发送/'
data_360_sample = pd.read_excel(path_360 + 'ceshi_sample_20241218.xlsx')
data_360_sample = data_360_sample[['order_number','idcard_md5','mobile_md5','下单日期']]
data_360_sample.shape

path_360 = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/2_20241218/2.数据返回/维客壹佰/'
data_360_1 = pd.read_csv(path_360 + '维客壹佰-201048占信分臻藏版-支付风险V3.0-100%.csv')
data_360_2 = pd.read_csv(path_360 + '维客壹佰-201053综合行为评估报告V2.4-100%.csv')
data_360_3 = pd.read_csv(path_360 + '维客壹佰-201054综合行为评估报告V2.2-100%.csv', encoding = 'gbk')
data_360_4 = pd.read_csv(path_360 + '维客壹佰-201055综合行为评估报告V2.3-100%.csv', encoding = 'gbk')
data_360_5 = pd.read_csv(path_360 + '维客壹佰-202098占信分臻藏版-行为风险V3.0-99.83%.csv', encoding = 'gbk')
data_360_6 = pd.read_csv(path_360 + '维客壹佰-202100占信分臻藏版-支付风险V3.0-100%.csv', encoding = 'gbk')
data_360_7 = pd.read_csv(path_360 + '维客壹佰-202123占信分标准版-消费风险V3.0-99.97%.csv')


data_360_1 = data_360_1.rename(columns = {'score':'score_201048'})
data_360_1 = data_360_1[['idcard_md5','mobile_md5','下单日期','score_201048']]

data_360_2 = data_360_2.rename(columns = {'score':'score_201053'})
data_360_2 = data_360_2[['idcard_md5','mobile_md5','下单日期','score_201053']]

data_360_3 = data_360_3.rename(columns = {'score':'score_201054'})
data_360_3 = data_360_3[['idcard_md5','mobile_md5','下单日期','score_201054']]

data_360_4 = data_360_4.rename(columns = {'score':'score_201055'})
data_360_4 = data_360_4[['idcard_md5','mobile_md5','下单日期','score_201055']]

data_360_5 = data_360_5.rename(columns = {'score':'score_202098'})
data_360_5 = data_360_5[['idcard_md5','mobile_md5','下单日期','score_202098']]

data_360_6 = data_360_6.rename(columns = {'score':'score_202100'})
data_360_6 = data_360_6[['idcard_md5','mobile_md5','下单日期','score_202100']]

data_360_7 = data_360_7.rename(columns = {'score':'score_202123'})
data_360_7 = data_360_7[['idcard_md5','mobile_md5','下单日期','score_202123']]

data_360_1.shape
data_360_2.shape
data_360_3.shape
data_360_4.shape
data_360_5.shape
data_360_6.shape
data_360_7.shape

data_360_1.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_2.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_3.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_4.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_5.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_6.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_7.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)

data_360_score =  data_360_1.merge(data_360_2, on = ['idcard_md5','mobile_md5','下单日期'], how = 'inner')
data_360_score =  data_360_score.merge(data_360_3, on = ['idcard_md5','mobile_md5','下单日期'], how = 'inner')
data_360_score =  data_360_score.merge(data_360_4, on = ['idcard_md5','mobile_md5','下单日期'], how = 'inner')
data_360_score =  data_360_score.merge(data_360_5, on = ['idcard_md5','mobile_md5','下单日期'], how = 'inner')
data_360_score =  data_360_score.merge(data_360_6, on = ['idcard_md5','mobile_md5','下单日期'], how = 'inner')
data_360_score =  data_360_score.merge(data_360_7, on = ['idcard_md5','mobile_md5','下单日期'], how = 'inner')
data_360_score.shape

data_360_score =  data_360_sample.merge(data_360_score, on = ['idcard_md5','mobile_md5','下单日期'], how = 'left')
data_360_score = data_360_score.fillna(-999)
data_360_score = data_360_score.drop(columns= ['idcard_md5', 'mobile_md5'])
data_360_score.shape
data_360_score[:2]

data_360_score[data_360_score.score_201048 == -999].shape
data_360_score[data_360_score.score_201053 == -999].shape
data_360_score[data_360_score.score_201054 == -999].shape
data_360_score[data_360_score.score_201055 == -999].shape
data_360_score[data_360_score.score_202098 == -999].shape
data_360_score[data_360_score.score_202100 == -999].shape
data_360_score[data_360_score.score_202123 == -999].shape

(19669, 4)

(19669, 4)

(19669, 4)

(19669, 4)

(19669, 4)

...

(0, 9)

(0, 9)

(4, 9)

(0, 9)

(5, 9)

In [251]:
# （2）关联租后表现
# 租后数据跑数日期：20250204
data_360_score = df_analy_data[['order_number','search_time','归属渠道','逾期天数','FPD','SPD','TPD','第二期逾期天数','第三期逾期天数','第四期逾期天数']].merge(data_360_score, on = 'order_number', how = 'inner')
data_360_score.shape
data_360_score[:1]

(19669, 18)

,order_number,search_time,归属渠道,逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,下单日期,score_201048,score_201053,score_201054,score_201055,score_202098,score_202100,score_202123
0,A202307010727469,2023-07,芝麻租物,0,0.0,0.0,0.0,0.0,0.0,0.0,2023-07-01,425.0,499.0,488,476,532.0,406,529.0


In [252]:
# （3）打标签
# 本次测试的样本为2023.7-2024.9 有贷后表现的订单，共7个评分产品，含反欺诈和信用场景
# 201048、202100、202098-租赁行业
# 信用分：mob3 & dpd30+；   欺诈分：Mob1 & FPD30+
# A20230807104214101	第四期逾期天数 为空

data_360_score['第四期逾期天数'] = data_360_score['第四期逾期天数'].fillna(0)
data_360_score['tmp'] = np.where(data_360_score.第二期逾期天数 >= data_360_score.第三期逾期天数, data_360_score.第二期逾期天数, data_360_score.第三期逾期天数)
data_360_score['tmp'] = np.where(data_360_score.tmp >= data_360_score.第四期逾期天数, data_360_score.tmp, data_360_score.第四期逾期天数)
data_360_score['y_credit_mob3_dpd30'] = data_360_score.apply(lambda x: 1 if x.tmp > 30
                                               else 0.5 if x.tmp >= 1 and x.tmp <= 30
                                               else 0 if x.tmp == 0
                                               else 999, axis = 1)
data_360_score['y_credit_dpd30'] = data_360_score.apply(lambda x: 1 if x.逾期天数 > 30
                                               else 0.5 if x.逾期天数 >= 1 and x.逾期天数 <= 30
                                               else 0 if x.逾期天数 == 0
                                               else 999, axis = 1)

data_360_score = data_360_score.drop(columns= ['tmp'])
data_360_score.shape
data_360_score[:1]
data_360_score.y_credit_mob3_dpd30.value_counts()
data_360_score.y_credit_dpd30.value_counts()
# data_360_score[(data_360_score.第二期逾期天数 > 30) | (data_360_score.第三期逾期天数 > 30) | (data_360_score.第四期逾期天数 > 30)].shape
# data_360_score[data_360_score.y_credit_mob3_dpd30 == 1].下单日期.max()
# '2024-09-30'

(19669, 20)

,order_number,search_time,归属渠道,逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,下单日期,score_201048,score_201053,score_201054,score_201055,score_202098,score_202100,score_202123,y_credit_mob3_dpd30,y_credit_dpd30
0,A202307010727469,2023-07,芝麻租物,0,0.0,0.0,0.0,0.0,0.0,0.0,2023-07-01,425.0,499.0,488,476,532.0,406,529.0,0.0,0.0


y_credit_mob3_dpd30
0.0    14527
0.5     3136
1.0     2006
Name: count, dtype: int64

y_credit_dpd30
0.0    10146
0.5     4842
1.0     4681
Name: count, dtype: int64

In [274]:
# （4）样本切分
# 本次测试的样本为2023.7-2024.9 有贷后表现的订单，共7个评分产品，含反欺诈和信用场景
# 训练集 2023.7-2024.6
# 测试集 2024.7、2024.8、2024.9
train_360_score = data_360_score[(data_360_score.search_time <= '2024-06') & (data_360_score.y_credit_mob3_dpd30.isin([0,1]))]
test_360_score = data_360_score[(data_360_score.search_time >= '2024-07') & (data_360_score.y_credit_mob3_dpd30.isin([0,1]))]
train_360_score.shape
test_360_score.shape

pd.pivot_table(data_360_score,index='search_time',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
pd.pivot_table(data_360_score,index='search_time',columns='y_credit_dpd30',values='order_number',aggfunc='count')

(10594, 20)

(5939, 20)

y_credit_mob3_dpd30,0.0,0.5,1.0
search_time,,,
2023-07,602,151,129
2023-08,595,168,116
2023-09,637,166,87
2023-10,822,215,161
2023-11,1122,264,177
2023-12,751,142,97
2024-01,540,102,55
2024-02,293,64,21
2024-03,626,129,86


y_credit_dpd30,0.0,0.5,1.0
search_time,,,
2023-07,257,270,355
2023-08,293,246,340
2023-09,325,242,323
2023-10,370,367,461
2023-11,635,422,506
2023-12,397,274,319
2024-01,288,216,193
2024-02,189,98,91
2024-03,399,213,229


In [278]:
# （5）分箱
%run C:/Users/Administrator/Desktop/mine/4_专题分析/1_回捞分析/2.model_func.ipynb
continous_merged_dict = {}
var_bin_list = []
# trainData_kf_Bin = train_360_score.copy()
trainData_kf_Bin = train_360_score[train_360_score.归属渠道 == '芝麻租物']
trainData_kf_Bin['y'] = trainData_kf_Bin['y_credit_mob3_dpd30']
numeric_desc_var = ['score_201048','score_201053','score_201054','score_201055','score_202098','score_202100','score_202123']

# start_time = time.time()

for col in numeric_desc_var:
    print ("{} is in processing".format(col))
    if -1111 not in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]): 
        max_interval = 5 
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval,special_attribute=[],minBinPcnt=0)
        trainData_kf_Bin[col+'_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff,special_attribute=[]))
        monotone = BadRateMonotone(trainData_kf_Bin, col+'_Bin', 'y')
        while(not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute = [],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = []))
            if max_interval == 2:
                break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y')
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [])) 
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    else:
        max_interval = 7 #正常取值最多分5箱
        cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval = max_interval, special_attribute = [-1111, -999],
                                      minBinPcnt = 0)
        trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1111, -999]))
        monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2'])
        while (not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute=[-1111, -999],
                                          minBinPcnt = 0)
            trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))
            if max_interval == 3:
                break
#             if max_interval == 3 and -1111 in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 3 and -1111 not in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
#             if max_interval == 4 and -1111 in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
#                 break
            monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2']) 
        newVar = col + '_Bin'
        trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))  
        del trainData_kf_Bin[col]
        var_bin_list.append(newVar)
    continous_merged_dict[col] = cutOff
    
# print("- Time consumed: %s seconds" % int(time.time() - start_time))

score_201048 is in processing
score_201053 is in processing
score_201054 is in processing
score_201055 is in processing
score_202098 is in processing
score_202100 is in processing
score_202123 is in processing


In [257]:
# （6）改变映射方式
trainData_kf_Bin_1  = train_360_score.copy()
trainData_kf_Bin_1 ['y'] = trainData_kf_Bin['y_credit_mob3_dpd30']

# start_time = time.time()
for col in numeric_desc_var:
    if -1111 not in set(trainData_kf_Bin_1[col]) and -999 not in set(trainData_kf_Bin_1[col]):
        trainData_kf_Bin_1[col+'_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col]))
        del trainData_kf_Bin_1[col]
    else:
        trainData_kf_Bin_1[col + '_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col],[-1111, -999]))
        del trainData_kf_Bin_1[col]
        
# print("- Time consumed: %s seconds" % int(time.time() - start_time))

In [258]:
# （7）映射到test
# start_time = time.time()
test = test_360_score.copy()
test ['y'] = test['y_credit_mob3_dpd30']
short_list_ori = numeric_desc_var

for var in short_list_ori:
    if -1111 not in set(test[var]) and -999 not in set(test[var]):
        test[var + '_Bin'] = test[var].map(lambda x: AssignBin_me(x, continous_merged_dict[var]))
        del test[var]
    else:
        test[var + '_Bin'] = test[var].map(lambda x: AssignBin_me(x, continous_merged_dict[var],[-1111, -999]))
        del test[var]

# print("- Time consumed: %s seconds" % int(time.time() - start_time))

test.shape

(5939, 21)

In [261]:
# （8）重新映射
# 包含灰色样本
# continous_merged_dict
# numeric_desc_var = ['score_201048','score_201053','score_201054','score_201055','score_202098','score_202100','score_202123']
data_360_score['score_201048_new_Bin'] = data_360_score.apply(lambda x: '1.[0,424]' if x.score_201048 >= 0 and x.score_201048 <= 424
                                                              else '2.(424,517]' if x.score_201048 > 424 and x.score_201048 <= 517
                                                              else '3.(517,543]' if x.score_201048 > 517 and x.score_201048 <= 543
                                                              else '4.(543,588]' if x.score_201048 > 543 and x.score_201048 <= 588
                                                              else '5.(588,inf)' if x.score_201048 > 588
                                                              else '6.[-999,-999]'
                                                              , axis = 1)
data_360_score['score_201053_new_Bin'] = data_360_score.apply(lambda x: '1.[0,464]' if x.score_201053 >= 0 and x.score_201053 <= 464
                                                              else '2.(464,491]' if x.score_201053 > 464 and x.score_201053 <= 491
                                                              else '3.(491,520]' if x.score_201053 > 491 and x.score_201053 <= 520
                                                              else '4.(520,596]' if x.score_201053 > 520 and x.score_201053 <= 596
                                                              else '5.(596,inf)' if x.score_201053 > 596
                                                              else '6.[-999,-999]'
                                                              , axis = 1)
data_360_score['score_201054_new_Bin'] = data_360_score.apply(lambda x: '1.[0,431]' if x.score_201054 >= 0 and x.score_201054 <= 431
                                                              else '2.(431,495]' if x.score_201054 > 431 and x.score_201054 <= 495
                                                              else '3.(495,549]' if x.score_201054 > 495 and x.score_201054 <= 549
                                                              else '4.(549,592]' if x.score_201054 > 549 and x.score_201054 <= 592
                                                              else '5.(592,inf)' if x.score_201054 > 592
                                                              else '6.[-999,-999]'
                                                              , axis = 1)
data_360_score['score_201055_new_Bin'] = data_360_score.apply(lambda x: '1.[0,452]' if x.score_201055 >= 0 and x.score_201055 <= 452
                                                              else '2.(452,541]' if x.score_201055 > 452 and x.score_201055 <= 541
                                                              else '3.(541,583]' if x.score_201055 > 541 and x.score_201055 <= 583
                                                              else '4.(583,628]' if x.score_201055 > 583 and x.score_201055 <= 628
                                                              else '5.(628,inf)' if x.score_201055 > 628
                                                              else '6.[-999,-999]'
                                                              , axis = 1)
data_360_score['score_202098_new_Bin'] = data_360_score.apply(lambda x: '1.[0,417]' if x.score_202098 >= 0 and x.score_202098 <= 417
                                                              else '2.(417,470]' if x.score_202098 > 417 and x.score_202098 <= 470
                                                              else '3.(470,511]' if x.score_202098 > 470 and x.score_202098 <= 511
                                                              else '4.(511,546]' if x.score_202098 > 511 and x.score_202098 <= 546
                                                              else '5.(546,602)' if x.score_202098 > 546 and x.score_202098 <= 602
                                                              else '6.(602,inf)' if x.score_202098 > 602
                                                              else '7.[-999,-999]'
                                                              , axis = 1)
data_360_score['score_202100_new_Bin'] = data_360_score.apply(lambda x: '1.[0,407]' if x.score_202100 >= 0 and x.score_202100 <= 407
                                                              else '2.(407,440]' if x.score_202100 > 407 and x.score_202100 <= 440
                                                              else '3.(440,477]' if x.score_202100 > 440 and x.score_202100 <= 477
                                                              else '4.(477,524]' if x.score_202100 > 477 and x.score_202100 <= 524
                                                              else '5.(524,inf)' if x.score_202100 > 524
                                                              else '6.[-999,-999]'
                                                              , axis = 1)
data_360_score['score_202123_new_Bin'] = data_360_score.apply(lambda x: '1.[0,401]' if x.score_202123 >= 0 and x.score_202123 <= 401
                                                              else '2.(401,472]' if x.score_202123 > 401 and x.score_202123 <= 472
                                                              else '3.(472,500]' if x.score_202123 > 472 and x.score_202123 <= 500
                                                              else '4.(500,538]' if x.score_202123 > 500 and x.score_202123 <= 538
                                                              else '5.(538,598)' if x.score_202123 > 538 and x.score_202123 <= 598
                                                              else '6.(598,inf)' if x.score_202123 > 598
                                                              else '7.[-999,-999]'
                                                              , axis = 1)

In [270]:
# （9）结果统计
# 201048、202100、202098-租赁行业
condition_4 = (data_360_score.search_time <= '2024-06')
condition_5 = (data_360_score.search_time == '2024-07')
condition_6 = (data_360_score.search_time == '2024-08')
condition_7 = (data_360_score.search_time == '2024-09')

# pd.pivot_table(data_360_score[condition_4],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4],index='score_201053_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5],index='score_201053_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6],index='score_201053_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7],index='score_201053_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4],index='score_201054_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5],index='score_201054_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6],index='score_201054_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7],index='score_201054_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4],index='score_201055_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5],index='score_201055_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6],index='score_201055_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7],index='score_201055_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

pd.pivot_table(data_360_score[condition_4],index='score_202123_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
pd.pivot_table(data_360_score[condition_5],index='score_202123_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
pd.pivot_table(data_360_score[condition_6],index='score_202123_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
pd.pivot_table(data_360_score[condition_7],index='score_202123_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# # pd.pivot_table(trainData_kf_Bin_1,index='score_201048_Bin',columns='y',values='order_number',aggfunc='count')
# # pd.pivot_table(trainData_kf_Bin_1,index='score_201053_Bin',columns='y',values='order_number',aggfunc='count')
# # pd.pivot_table(trainData_kf_Bin_1,index='score_201054_Bin',columns='y',values='order_number',aggfunc='count')
# # pd.pivot_table(trainData_kf_Bin_1,index='score_201055_Bin',columns='y',values='order_number',aggfunc='count')
# # pd.pivot_table(trainData_kf_Bin_1,index='score_202098_Bin',columns='y',values='order_number',aggfunc='count')
# # pd.pivot_table(trainData_kf_Bin_1,index='score_202100_Bin',columns='y',values='order_number',aggfunc='count')
# # pd.pivot_table(trainData_kf_Bin_1,index='score_202123_Bin',columns='y',values='order_number',aggfunc='count')

y_credit_mob3_dpd30,0.0,0.5,1.0
score_202123_new_Bin,,,
"1.[0,401]",140.0,80.0,78.0
"2.(401,472]",1870.0,627.0,483.0
"3.(472,500]",1224.0,323.0,218.0
"4.(500,538]",1884.0,426.0,265.0
"5.(538,598)",2678.0,484.0,242.0
"6.(598,inf)",1455.0,237.0,56.0
"7.[-999,-999]",1.0,NaN,NaN


y_credit_mob3_dpd30,0.0,0.5,1.0
score_202123_new_Bin,,,
"1.[0,401]",21.0,14.0,20.0
"2.(401,472]",309.0,80.0,61.0
"3.(472,500]",245.0,56.0,28.0
"4.(500,538]",328.0,60.0,29.0
"5.(538,598)",420.0,75.0,28.0
"6.(598,inf)",243.0,33.0,10.0
"7.[-999,-999]",NaN,1.0,NaN


y_credit_mob3_dpd30,0.0,0.5,1.0
score_202123_new_Bin,,,
"1.[0,401]",20,8,6
"2.(401,472]",339,75,78
"3.(472,500]",269,45,37
"4.(500,538]",358,53,50
"5.(538,598)",479,81,46
"6.(598,inf)",248,28,18


y_credit_mob3_dpd30,0.0,0.5,1.0
score_202123_new_Bin,,,
"1.[0,401]",18.0,6.0,5.0
"2.(401,472]",407.0,76.0,71.0
"3.(472,500]",288.0,65.0,44.0
"4.(500,538]",432.0,84.0,52.0
"5.(538,598)",553.0,94.0,67.0
"6.(598,inf)",296.0,24.0,14.0
"7.[-999,-999]",2.0,1.0,NaN


In [287]:
# （9-1）结果统计 - 细分渠道
condition_8 = (data_360_score.归属渠道 == '芝麻租物')
condition_9 = (data_360_score.归属渠道 == '搜索渠道')
condition_10 = (~data_360_score.归属渠道.isin(['芝麻租物','搜索渠道']))
# data_360_score.to_excel('data_360_score.xlsx', index = False)

# 芝麻租物手动调整第一组cutoff
# pd.pivot_table(data_360_score[condition_4 & condition_8],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5 & condition_8],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6 & condition_8],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7 & condition_8],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4 & condition_9],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5 & condition_9],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6 & condition_9],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7 & condition_9],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4 & condition_10],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5 & condition_10],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6 & condition_10],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7 & condition_10],index='score_201048_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4 & condition_8],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5 & condition_8],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6 & condition_8],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7 & condition_8],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4 & condition_9],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5 & condition_9],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6 & condition_9],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7 & condition_9],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4 & condition_10],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5 & condition_10],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6 & condition_10],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7 & condition_10],index='score_202098_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4 & condition_8],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5 & condition_8],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6 & condition_8],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7 & condition_8],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

# pd.pivot_table(data_360_score[condition_4 & condition_9],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_5 & condition_9],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_6 & condition_9],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
# pd.pivot_table(data_360_score[condition_7 & condition_9],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

pd.pivot_table(data_360_score[condition_4 & condition_10],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
pd.pivot_table(data_360_score[condition_5 & condition_10],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
pd.pivot_table(data_360_score[condition_6 & condition_10],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
pd.pivot_table(data_360_score[condition_7 & condition_10],index='score_202100_new_Bin',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

y_credit_mob3_dpd30,0.0,0.5,1.0
score_202100_new_Bin,,,
"1.[0,407]",82,39,52
"2.(407,440]",159,54,55
"3.(440,477]",273,85,85
"4.(477,524]",343,85,54
"5.(524,inf)",359,87,45


y_credit_mob3_dpd30,0.0,0.5,1.0
score_202100_new_Bin,,,
"1.[0,407]",11,7,8
"2.(407,440]",16,4,11
"3.(440,477]",22,7,10
"4.(477,524]",39,5,12
"5.(524,inf)",38,9,4


y_credit_mob3_dpd30,0.0,0.5,1.0
score_202100_new_Bin,,,
"1.[0,407]",17,4,12
"2.(407,440]",18,2,9
"3.(440,477]",43,12,15
"4.(477,524]",48,9,3
"5.(524,inf)",47,6,6


y_credit_mob3_dpd30,0.0,0.5,1.0
score_202100_new_Bin,,,
"1.[0,407]",9,5,6
"2.(407,440]",30,8,7
"3.(440,477]",31,9,14
"4.(477,524]",53,6,10
"5.(524,inf)",84,8,4


In [300]:
# （10）对比分析
# score_201048
# 芝麻租物：<= 409 强拒
# 搜索渠道：<= 424 强拒
# 非芝麻租物非搜索渠道：<= 424 强拒

# score_202098
# 芝麻租物：暂不应用
# 搜索渠道：<= 417 强拒
# 非芝麻租物非搜索渠道：<= 417 强拒

# score_202100
# 芝麻租物：<= 387 强拒
# 搜索渠道：<= 404 强拒
# 非芝麻租物非搜索渠道：<= 407 强拒

data_360_score['result_reject_1'] = data_360_score.apply(lambda x: 1 if x.归属渠道 == '芝麻租物' and x.score_201048 >= 0 and x.score_201048 <= 409 
                                                       else 1 if x.归属渠道 != '芝麻租物' and x.score_201048 >= 0 and x.score_201048 <= 424 
                                                       else 0, axis = 1)
data_360_score['result_reject_2'] = data_360_score.apply(lambda x: 1 if x.归属渠道 != '芝麻租物' and x.score_202098 >= 0 and x.score_202098 <= 417
                                                       else 0, axis = 1)
data_360_score['result_reject_3'] = data_360_score.apply(lambda x: 1 if x.归属渠道 == '芝麻租物' and x.score_202100 >= 0 and x.score_202100 <= 387 
                                                       else 1 if x.归属渠道 == '搜索渠道' and x.score_202100 >= 0 and x.score_202100 <= 404
                                                       else 1 if x.归属渠道 != '搜索渠道' and x.归属渠道 != '芝麻租物' and x.score_202100 >= 0 and x.score_202100 <= 407
                                                       else 0, axis = 1)
data_360_score['result_reject'] = data_360_score['result_reject_1']  + data_360_score['result_reject_2'] + data_360_score['result_reject_3'] 

data_360_score.result_reject_1.value_counts()
data_360_score.result_reject_2.value_counts()
data_360_score.result_reject_3.value_counts()
data_360_score.result_reject.value_counts()

pd.pivot_table(data_360_score[data_360_score.result_reject == 0],index='search_time',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')
pd.pivot_table(data_360_score[data_360_score.result_reject >= 1],index='search_time',columns='y_credit_mob3_dpd30',values='order_number',aggfunc='count')

pd.pivot_table(data_360_score[data_360_score.result_reject == 0],index='search_time',columns='y_credit_dpd30',values='order_number',aggfunc='count')
pd.pivot_table(data_360_score[data_360_score.result_reject >= 1],index='search_time',columns='y_credit_dpd30',values='order_number',aggfunc='count')

result_reject_1
0    19117
1      552
Name: count, dtype: int64

result_reject_2
0    19371
1      298
Name: count, dtype: int64

result_reject_3
0    18917
1      752
Name: count, dtype: int64

result_reject
0    18412
1      958
2      253
3       46
Name: count, dtype: int64

y_credit_mob3_dpd30,0.0,0.5,1.0
search_time,,,
2023-07,551,129,103
2023-08,551,150,79
2023-09,592,144,64
2023-10,779,200,134
2023-11,1078,245,149
2023-12,710,124,90
2024-01,520,91,50
2024-02,287,63,19
2024-03,604,124,74


y_credit_mob3_dpd30,0.0,0.5,1.0
search_time,,,
2023-07,51,22,26
2023-08,44,18,37
2023-09,45,22,23
2023-10,43,15,27
2023-11,44,19,28
2023-12,41,18,7
2024-01,20,11,5
2024-02,6,1,2
2024-03,22,5,12


y_credit_dpd30,0.0,0.5,1.0
search_time,,,
2023-07,243,241,299
2023-08,270,234,276
2023-09,309,219,272
2023-10,354,348,411
2023-11,615,395,462
2023-12,376,256,292
2024-01,277,204,180
2024-02,185,96,88
2024-03,387,206,209


y_credit_dpd30,0.0,0.5,1.0
search_time,,,
2023-07,14,29,56
2023-08,23,12,64
2023-09,16,23,51
2023-10,16,19,50
2023-11,20,27,44
2023-12,21,18,27
2024-01,11,12,13
2024-02,4,2,3
2024-03,12,7,20


In [273]:
# （11）相关性分析
numeric_desc_var = ['score_201048','score_201053','score_201054','score_201055','score_202098','score_202100','score_202123']
data_360_score[condition_4][numeric_desc_var].corr()
data_360_score[condition_5][numeric_desc_var].corr()
data_360_score[condition_6][numeric_desc_var].corr()
data_360_score[condition_7][numeric_desc_var].corr()

,score_201048,score_201053,score_201054,score_201055,score_202098,score_202100,score_202123
score_201048,1.000000,0.790042,0.904742,0.737699,0.508481,0.704330,0.406463
score_201053,0.790042,1.000000,0.930185,0.751757,0.444877,0.616551,0.436074
score_201054,0.904742,0.930185,1.000000,0.716606,0.507748,0.675109,0.422916
score_201055,0.737699,0.751757,0.716606,1.000000,0.528917,0.578452,0.546725
score_202098,0.508481,0.444877,0.507748,0.528917,1.000000,0.636553,0.867708
score_202100,0.704330,0.616551,0.675109,0.578452,0.636553,1.000000,0.491364
score_202123,0.406463,0.436074,0.422916,0.546725,0.867708,0.491364,1.000000


,score_201048,score_201053,score_201054,score_201055,score_202098,score_202100,score_202123
score_201048,1.000000,0.778205,0.900673,0.734026,0.419107,0.672378,0.339434
score_201053,0.778205,1.000000,0.925566,0.736617,0.364740,0.594199,0.354130
score_201054,0.900673,0.925566,1.000000,0.699686,0.405530,0.642051,0.333538
score_201055,0.734026,0.736617,0.699686,1.000000,0.506364,0.586340,0.508934
score_202098,0.419107,0.364740,0.405530,0.506364,1.000000,0.581379,0.928376
score_202100,0.672378,0.594199,0.642051,0.586340,0.581379,1.000000,0.453250
score_202123,0.339434,0.354130,0.333538,0.508934,0.928376,0.453250,1.000000


,score_201048,score_201053,score_201054,score_201055,score_202098,score_202100,score_202123
score_201048,1.000000,0.791138,0.901115,0.738749,0.476652,0.668077,0.368293
score_201053,0.791138,1.000000,0.931637,0.747753,0.435318,0.602395,0.407534
score_201054,0.901115,0.931637,1.000000,0.703375,0.469326,0.639222,0.368917
score_201055,0.738749,0.747753,0.703375,1.000000,0.531012,0.580877,0.526743
score_202098,0.476652,0.435318,0.469326,0.531012,1.000000,0.638162,0.893234
score_202100,0.668077,0.602395,0.639222,0.580877,0.638162,1.000000,0.462774
score_202123,0.368293,0.407534,0.368917,0.526743,0.893234,0.462774,1.000000


,score_201048,score_201053,score_201054,score_201055,score_202098,score_202100,score_202123
score_201048,1.000000,0.794121,0.903628,0.763641,0.395021,0.681754,0.292545
score_201053,0.794121,1.000000,0.929303,0.756716,0.365565,0.607961,0.335205
score_201054,0.903628,0.929303,1.000000,0.716451,0.396758,0.652440,0.305075
score_201055,0.763641,0.756716,0.716451,1.000000,0.440898,0.597520,0.415224
score_202098,0.395021,0.365565,0.396758,0.440898,1.000000,0.545424,0.868063
score_202100,0.681754,0.607961,0.652440,0.597520,0.545424,1.000000,0.380891
score_202123,0.292545,0.335205,0.305075,0.415224,0.868063,0.380891,1.000000


## 5_3.202503

### 5_3_1.样本挑选

In [ ]:
# （1）2024.10-2025.1 出库
# 信用分是mob3_30+，欺诈分是看fpd30
# 上一次线下回溯的时间范围是：2023.7-2024.9 有贷后表现的订单
# 本次回溯：2024.10-2025.1 有贷后表现的订单，2025.2.1-2025.2.25 进件免押（2025.2.25上线）

# （1-1）全量三要素 姓名、身份证、手机号
#   `status` int DEFAULT NULL COMMENT '状态 1,待支付,2,待发货,3,待收货,4,租赁中,5,待归还,6,待商家收货/归还中,8,已完成,10,已退款,\r12,待审核,13,订单取消, 15,检测中',
sql = '''-- 租后应收监控  
SELECT  om.id as order_id, om.order_number, om.create_time, om.merchant_name
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
, tmu.true_name, tmu.id_card_num, tmu.mobile 
, tod.sku_attributes
, top.total_describes
, pa.name as activity_name        -- 活动名称
from db_digua_business.t_order om
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on om.id = tod.order_id
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
;
'''

df_info = query(sql)
df_info["下单日期"] = df_info['create_time'].dt.date
df_info["下单日期"] = pd.to_datetime(df_info["下单日期"],errors="coerce")
df_info.shape

# （1-2） 关联目标客户三要素
df_analy_data = df_analy_data.merge(df_info[['order_number','true_name','id_card_num','mobile']], on = 'order_number', how = 'left')
df_analy_data.shape

df_202410_ = df_analy_data[(df_analy_data.search_time >= '2024-10') & (df_analy_data.search_time <= '2025-01')]
df_202410_.shape
df_202410_[df_202410_.id_card_num.isna()].shape
df_202410_[df_202410_.mobile.isna()].shape

df_202410_ = df_202410_[df_202410_.id_card_num.notnull()]
df_202410_.shape

# （7-3）md5加密
import hashlib
 
def md5_encryption(var):
    md5 = hashlib.md5()  # 创建一个md5对象
    md5.update(var.encode('utf-8'))  # 使用utf-8编码数据
    return md5.hexdigest()  # 返回加密后的十六进制字符串

df_202410_['user_name_md5'] = df_202410_.apply(lambda x: md5_encryption(x.true_name), axis = 1)
df_202410_['idcard_md5'] = df_202410_.apply(lambda x: md5_encryption(x.id_card_num), axis = 1)
df_202410_['mobile_md5'] = df_202410_.apply(lambda x: md5_encryption(x.mobile), axis = 1)
df_202410_.shape
df_202410_.tail()

# # card_id = '533526200104110620'
# # md5 = hashlib.md5(card_id.encode())
# # md5.hexdigest()

# df_202410_['下单日期'] = df_202410_['下单日期'].astype('str')
# path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/3_20250320/'
# df_202410_.to_excel(path + 'ceshi_sample_20250320.xlsx', index = False)

(2204847, 12)

(35298, 17)

(9172, 17)

(0, 17)

(0, 17)

(9172, 17)

(9172, 20)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,true_name,id_card_num,mobile,user_name_md5,idcard_md5,mobile_md5
31696,A2025012919442921,2050290,2025-01-29,2025-01,芝麻租物,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0,李阳,431126200512070317,16670925621,6944d9d6078e831516c34d85a309d191,06e96b79008ae2dae9ad36d5c9eca8c6,4e396f96ee2d342a092d2b5d1b95b958
31697,A202501310146535,2050963,2025-01-31,2025-01,搜索渠道,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0,蒋文鑫,320481199707091010,15861872080,d0a9eb1339737351ed20ba534407b1a9,ae3d34f4a255d64d34c5cdef6abee0b2,8871c61cd93451010fb5c51602c4e5f2
31698,A202501310303332,2050980,2025-01-31,2025-01,芝麻租物,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0,赵洋,120103198610155826,13911175789,3aab4721790fd92ffef023f061b3d1f1,ece7822d17b82209e4483c6beac29346,4a3e9220d9291aebb87fececc96ca288
31699,A2025013110494315,2051066,2025-01-31,2025-01,其他渠道场景渠道。,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0,苏业秋,452501198408242034,18677072767,1c73a9d1791e867c09526ce24170ccd3,ddea3dfed6d13cabe769134f0cdb22f5,c652fe4942465ac09f7709f76b0313f2
31700,A2025013113481433,2051203,2025-01-31,2025-01,芝麻租物,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0,张国鹏,513425200501225215,18981527326,20dda108173eb6c09ec0a2e005a752ed,08ce6125453632355b9330da572f4981,306131350fab9cf79821924546bad07d


In [81]:
# （2）2025.2 免押进件
path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/2.数据需求/'

df_my_202502 = pd.read_excel(path + '免押进件_202502.xlsx')
df_my_202502.shape

df_my_202502 = df_my_202502.merge(df_info[['order_number','true_name','id_card_num','mobile']], on = 'order_number', how = 'left')
df_my_202502.shape
df_my_202502 = df_my_202502[df_my_202502.下单日期 <= '2025-02-25']
df_my_202502.shape

# md5加密
import hashlib
 
def md5_encryption(var):
    md5 = hashlib.md5()  # 创建一个md5对象
    md5.update(var.encode('utf-8'))  # 使用utf-8编码数据
    return md5.hexdigest()  # 返回加密后的十六进制字符串

df_my_202502['user_name_md5'] = df_my_202502.apply(lambda x: md5_encryption(x.true_name), axis = 1)
df_my_202502['idcard_md5'] = df_my_202502.apply(lambda x: md5_encryption(x.id_card_num), axis = 1)
df_my_202502['mobile_md5'] = df_my_202502.apply(lambda x: md5_encryption(x.mobile), axis = 1)
df_my_202502.shape


# 合并
df_202410_202502 = pd.concat([df_202410_, df_my_202502])
df_202410_202502.shape
df_202410_202502.tail()

df_202410_202502['下单日期'] = df_202410_202502['下单日期'].astype('str')
path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/3_20250320/'
df_202410_202502.to_excel(path + 'ceshi_sample_20250320.xlsx', index = False)

(15784, 4)

(15784, 7)

(13272, 7)

(13272, 10)

(22444, 20)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,true_name,id_card_num,mobile,user_name_md5,idcard_md5,mobile_md5
13267,A202502252301185,NaN,2025-02-25,2025-02,抖音渠道,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,王聪,513723200009214796,13136520257,0b68f188d04f016aea35f3f681c68281,c3e528c00b5a72a3382f8f7fc0ad8905,7c91438d1a1273a3aacbb526ecae6f7b
13268,A20250225234300128,NaN,2025-02-25,2025-02,芝麻租物,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,卓玥,321202199509250628,19816321222,dcb4783f83bd24a891da060acf308e06,c059687efcde55a548183c20214125ca,ef3249e97731b3e8ee1ed1b8972a8e6a
13269,A20250225153712330,NaN,2025-02-25,2025-02,搜索渠道,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,廖敏,511025199705083942,18267652739,6ae77a7e096bfb65d895ab6c96932c42,17a48c47381140ed5679d730b29d542b,ad7869c1811a8789925a0c73b4d2a8ac
13270,A20250225155626516,NaN,2025-02-25,2025-02,芝麻租物,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,王权,450923200210055135,18877598244,390e875de00b800e031773db032f8fa4,122e146eeedb2581a2ed3ada2927acbc,65196dc1ab63b450a99e43dd1a3bf7f7
13271,A20250225165251501,NaN,2025-02-25,2025-02,搜索渠道,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,郑杨,230125198710150516,18145111661,a84e43dce7a4202b0c18ee017b355a15,a8b18bf50cba75506f4e59681f1b6c7a,bc632135c05f73e9c603d9081ada3880


# 6_飞鸟

<!-- 1、20241225 挑选样本，用于地址反欺诈测试 -->
<!-- 工作地和居住地大概的逻辑是这样：
设定工作和居住的时间
● 上班时段：工作日10点-18点
● 居住时段：工作日晚上9点-早上6点
结合位置的停留时长及停留天数，取以上时间段进行计算，然后通过算法区分工作地和居住地 -->

In [142]:
# （1）样本挑选
# （1-1）取客户基本信息
sql = '''-- 租后应收监控  
SELECT  om.id as order_id, om.order_number, om.create_time, om.merchant_name
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
, tmu.true_name, tmu.id_card_num, tmu.mobile 
, tod.sku_attributes
, top.total_describes
, pa.name as activity_name        -- 活动名称
, om.coordinate                   -- 下单GPS
, tol.delivery_detail_address  -- 收货人详细地址
, tol.delivery_province_name  -- 收货省
, tol.delivery_city_name      -- 收货市
, tol.delivery_county_name    -- 区县名称
from db_digua_business.t_order om
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on om.id = tod.order_id
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_digua_business.t_order_logistics tol on om.id = tol.order_id
;
'''

df_info = query(sql)
df_info["下单日期"] = df_info['create_time'].dt.date
df_info["下单日期"] = pd.to_datetime(df_info["下单日期"],errors="coerce")
df_info.delivery_province_name = df_info.delivery_province_name.fillna('未知')
df_info.delivery_city_name = df_info.delivery_city_name.fillna('未知')
df_info.delivery_county_name = df_info.delivery_county_name.fillna('未知')
df_info.delivery_detail_address = df_info.delivery_detail_address.fillna('未知')

# 人工检查样本的收货地址
df_info['delivery_detail_address'] = np.where(~df_info.delivery_detail_address.str.contains('市'), df_info.delivery_province_name + df_info.delivery_city_name + df_info.delivery_county_name + df_info.delivery_detail_address, df_info.delivery_detail_address)

# （1-2） 关联目标客户基本信息
df_analy_data = df_analy_data.merge(df_info[['order_number','true_name','id_card_num','mobile','coordinate','delivery_detail_address']], on = 'order_number', how = 'left')
df_analy_data.shape

# 2024.12.25 取数，标签定义 FPD7+ ，坏样本不抽样，好样本随机抽样，2024.9.1-2024.11.15 的 样本，共计500个
# 好客户按比例抽样（占好样本的6.55%）
# 2024-09:156
# 2024-10:98
# 2024-11:70
df_sample_1 = df_analy_data[(df_analy_data.下单日期 >= '2024-09-01') & (df_analy_data.下单日期 <= '2024-11-15') & (df_analy_data.FPD >= 7)]
df_sample_1.shape

df_sample_2 = df_analy_data[(df_analy_data.下单日期 >= '2024-09-01') & (df_analy_data.下单日期 <= '2024-11-15') & (df_analy_data.FPD == 0)].sample(n = 324)
df_sample_2.shape

df_sample = pd.concat([df_sample_1, df_sample_2])
df_sample.shape
df_sample[:2]

# 有几条地址无法转化为 经纬度，增加样本
# tmp = df_analy_data[(df_analy_data.search_time == '2024-09') & (df_analy_data.FPD == 0)].sample(n = 2)
# tmp1 = df_analy_data[(df_analy_data.search_time == '2024-10') & (df_analy_data.FPD == 0)].sample(n = 10)
# tmp = pd.concat([tmp, tmp1])

(29094, 16)

(176, 16)

(324, 16)

(500, 16)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数,FPD,SPD,TPD,true_name,id_card_num,mobile,coordinate,delivery_detail_address
19976,A20240901163947167,1415228,2024-09-01,2024-09,芝麻租物,全新,未带锁,9,9.0,0.0,0.0,宋梦云,410725200602100024,13598708859,"113.932652,35.060306",河南省新乡市原阳县三鑫和谐社区7号楼3单元4楼东户
19989,A2024090118245878,1415625,2024-09-01,2024-09,芝麻租物,全新,未带锁,81,81.0,0.0,0.0,许成坤,130433200302012715,18303027637,"114.478725,36.575375",河北省邯郸市邯山区星魔方c栋7楼713


In [145]:
# （2）样本分布查看
df_sample['y'] = df_sample.apply(lambda x: 1 if x.FPD >= 7 else 0, axis = 1)
pd.pivot_table(df_sample,index='search_time',columns='y',values='order_number',aggfunc='count')

y,0,1
search_time,,
2024-09,156,100
2024-10,93,46
2024-11,75,30


In [154]:
# （3）md5加密
# 入参：手机号 md5 加密，地址 经纬度
import hashlib
 
def md5_encryption(var):
    md5 = hashlib.md5()  # 创建一个md5对象
    md5.update(var.encode('utf-8'))  # 使用utf-8编码数据
    return md5.hexdigest()  # 返回加密后的十六进制字符串

df_sample['mobile_md5'] = df_sample.apply(lambda x: md5_encryption(x.mobile), axis = 1)
df_sample.to_excel('sample_LBS_fraud_20241225.xlsx', index = False)

# tmp['mobile_md5'] = tmp.apply(lambda x: md5_encryption(x.mobile), axis = 1)
# tmp.to_excel('tmp.xlsx', index = False)

In [156]:
# （4）返回-数据导入
# 20250114
# 同一手机号返回多条的，以有数据的为准 如：a8b723b73740f9bd700dfb4d306653dd、a310849a4c402654a0075b90a2bdf250
# work_distance_score	live_distance_score ： -100分 为距离工作地超过50公里的无法判断、-1分 为没有工作地或者居住地信息，或者原地址无法进行经纬度转换
# work_level_score	live_level_score        ：0 为没有工作地或者居住地信息，或者原地址无法进行经纬度转换
path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/3_飞鸟/202412/2.数据返回/'
path_1 = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/3_飞鸟/202412/1.样本发送/'

df_lbs_tmp1 = pd.read_excel(path + '迪瓜租机result_20241231_梳理.xlsx', sheet_name='距离分')
df_lbs_tmp2 = pd.read_excel(path + '迪瓜租机result_20241231_梳理.xlsx', sheet_name='等级分')
df_lbs_tmp1 = df_lbs_tmp1[['mobile_md5', 'distance_score', 'work_distance_score', 'live_distance_score']]
df_lbs_tmp2 = df_lbs_tmp2[['mobile_md5', 'live_score', 'work_level_score', 'live_level_score']]

df_lbs_tmp1.shape
df_lbs_tmp2.shape
# df_lbs_tmp1.drop_duplicates(subset = ['mobile_md5']).shape
# df_lbs_tmp2.drop_duplicates(subset = ['mobile_md5']).shape
# df_lbs_tmp1.mobile_md5.value_counts()
# df_lbs_tmp1[:1]
# df_lbs_tmp2[:2]


df_lbs_sample_tmp = pd.read_excel(path_1 + 'sample_LBS_fraud_20241225.xlsx', sheet_name='all')
df_lbs_sample_tmp = df_lbs_sample_tmp[['order_number', 'mobile_md5']]
df_lbs_sample_tmp.shape

df_lbs_sample = pd.read_excel(path_1 + 'sample_LBSfraud_20241226_发送.xlsx')
df_lbs_sample = df_lbs_sample[['mobile_md5']]
df_lbs_sample.shape

df_lbs_sample = df_lbs_sample.merge(df_lbs_sample_tmp, on = 'mobile_md5', how = 'inner')
df_lbs_sample.shape

df_lbs_sample = df_lbs_sample.merge(df_lbs_tmp1, on = 'mobile_md5', how = 'outer')
df_lbs_sample = df_lbs_sample.merge(df_lbs_tmp2, on = 'mobile_md5', how = 'outer')
df_lbs_sample.shape
df_lbs_sample.drop_duplicates(subset = ['mobile_md5']).shape

df_lbs_sample['work_distance_score'] = df_lbs_sample['work_distance_score'].replace(-100, -1111)
df_lbs_sample['work_distance_score'] = df_lbs_sample['work_distance_score'].replace(-1, -999)
df_lbs_sample['work_distance_score'] = df_lbs_sample['work_distance_score'].fillna(-999)

df_lbs_sample['live_distance_score'] = df_lbs_sample['live_distance_score'].replace(-100, -1111)
df_lbs_sample['live_distance_score'] = df_lbs_sample['live_distance_score'].replace(-1, -999)
df_lbs_sample['live_distance_score'] = df_lbs_sample['live_distance_score'].fillna(-999)

df_lbs_sample['work_level_score'] = df_lbs_sample['work_level_score'].replace(0, -999)
df_lbs_sample['work_level_score'] = df_lbs_sample['work_level_score'].fillna(-999)

df_lbs_sample['live_level_score'] = df_lbs_sample['live_level_score'].replace(0, -999)
df_lbs_sample['live_level_score'] = df_lbs_sample['live_level_score'].fillna(-999)

df_lbs_sample[df_lbs_sample.work_distance_score == -1111].shape
df_lbs_sample[df_lbs_sample.work_distance_score == -999].shape

df_lbs_sample[df_lbs_sample.live_distance_score == -1111].shape
df_lbs_sample[df_lbs_sample.live_distance_score == -999].shape

df_lbs_sample[df_lbs_sample.work_level_score == -999].shape
df_lbs_sample[df_lbs_sample.live_level_score == -999].shape

df_lbs_sample[:2]
del df_lbs_tmp1
del df_lbs_tmp2
del df_lbs_sample_tmp

(182, 4)

(315, 4)

(506, 2)

(499, 1)

(499, 2)

...

(9, 8)

(399, 8)

(305, 8)

(491, 8)

,mobile_md5,order_number,distance_score,work_distance_score,live_distance_score,live_score,work_level_score,live_level_score
0,fd4763ff184cf65600ed1ca690fc986c,A20240901163947167,NaN,-999.0,-999.0,NaN,-999.0,-999.0
1,b3141ae704c044745a2fed29a5235005,A2024090118245878,NaN,-999.0,-999.0,"12,0",12.0,-999.0


In [157]:
# （5）关联租后表现
df_lbs_data = df_analy_data.merge(df_lbs_sample, on = 'order_number', how = 'inner')
df_lbs_data.shape
df_lbs_data[:2]

(499, 18)

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数,FPD,SPD,TPD,mobile_md5,distance_score,work_distance_score,live_distance_score,live_score,work_level_score,live_level_score
0,A2024090108211431,1413383,2024-09-01,2024-09,芝麻租物,全新,未带锁,0,0.0,0.0,0.0,355f10f2add95569ab71afade30caf24,-100，-1,-1111.0,-999.0,NaN,-999.0,-999.0
1,A20240901095638143,1413604,2024-09-01,2024-09,芝麻租物,全新,未带锁,1,0.0,0.0,0.0,6ffdf779e2c62e49feaca45c38b40934,NaN,-999.0,-999.0,NaN,-999.0,-999.0


In [164]:
# （6）打标签
# 2024.12.25 取数，标签定义 FPD7+ ，坏样本不抽样，好样本随机抽样，2024.9.1-2024.11.15 的 样本，共计500个
# 好客户按月按比例抽样（占好样本的6.55%）
# 2024-09:156
# 2024-10:98
# 2024-11:70
# df_lbs_data.search_time.value_counts()
# df_lbs_data.下单日期.max()
# df_lbs_data['fraud_tmp'] = np.where(df_lbs_data.FPD >= df_lbs_data.SPD, df_lbs_data.FPD, df_lbs_data.SPD)
# df_lbs_data['fraud_tmp'] = np.where(df_lbs_data.fraud_tmp >= df_lbs_data.TPD, df_lbs_data.fraud_tmp, df_lbs_data.TPD)
# df_lbs_data['y_fraud'] = df_lbs_data.apply(lambda x: 0 if x.fraud_tmp == 0 
#                                                else 0.5 if x.fraud_tmp >0 and x.fraud_tmp < 7
#                                                else 1 if x.fraud_tmp >= 7
#                                                else 2, axis = 1)
df_analy_data['y_fraud_FPD'] = df_analy_data.apply(lambda x: 0 if x.FPD == 0 
                                               else 0.5 if x.FPD >0 and x.FPD < 7
                                               else 1 if x.FPD >= 7
                                               else 2, axis = 1)
df_lbs_data['y_fraud_FPD'] = df_lbs_data.apply(lambda x: 0 if x.FPD == 0 
                                               else 0.5 if x.FPD >0 and x.FPD < 7
                                               else 1 if x.FPD >= 7
                                               else 2, axis = 1)

df_lbs_data['work_distance_score_Bin'] = df_lbs_data.apply(lambda x: '1.[0,39]' if x.work_distance_score >= 0 and x.work_distance_score <= 39
                                                                    else '2.[40,59]' if x.work_distance_score >= 40 and x.work_distance_score <= 59
                                                                    else '3.[60,74]' if x.work_distance_score >= 60 and x.work_distance_score <= 74
                                                                    else '4.[75,89]' if x.work_distance_score >= 75 and x.work_distance_score <= 89
                                                                    else '5.[90,100]' if x.work_distance_score >= 90 and x.work_distance_score <= 100
                                                                    else '6.[-1111,-1111]' if x.work_distance_score == -1111
                                                                    else '7.[-999,-999]' if x.work_distance_score == -999
                                                                    else '8.other', axis = 1)

df_lbs_data['live_distance_score_Bin'] = df_lbs_data.apply(lambda x: '1.[0,39]' if x.live_distance_score >= 0 and x.live_distance_score <= 39
                                                                    else '2.[40,59]' if x.live_distance_score >= 40 and x.live_distance_score <= 59
                                                                    else '3.[60,74]' if x.live_distance_score >= 60 and x.live_distance_score <= 74
                                                                    else '4.[75,89]' if x.live_distance_score >= 75 and x.live_distance_score <= 89
                                                                    else '5.[90,100]' if x.live_distance_score >= 90 and x.live_distance_score <= 100
                                                                    else '6.[-1111,-1111]' if x.live_distance_score == -1111
                                                                    else '7.[-999,-999]' if x.live_distance_score == -999
                                                                    else '8.other', axis = 1) 

In [165]:
# （7）分布
pd.pivot_table(df_analy_data[(df_analy_data.search_time >= '2024-09')],index='search_time',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(df_analy_data[(df_analy_data.search_time >= '2024-09') & (df_analy_data.下单日期 <= '2024-11-15')],index='search_time',columns='y_fraud_FPD',values='order_number',aggfunc='count')

pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-09'],index='work_distance_score_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-10'],index='work_distance_score_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')   
pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-11'],index='work_distance_score_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')  

pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-09'],index='live_distance_score_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-10'],index='live_distance_score_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count')   
pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-11'],index='live_distance_score_Bin',columns='y_fraud_FPD',values='order_number',aggfunc='count') 

pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-09'],index='work_level_score',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-10'],index='work_level_score',columns='y_fraud_FPD',values='order_number',aggfunc='count')   
pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-11'],index='work_level_score',columns='y_fraud_FPD',values='order_number',aggfunc='count')  

pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-09'],index='live_level_score',columns='y_fraud_FPD',values='order_number',aggfunc='count')
pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-10'],index='live_level_score',columns='y_fraud_FPD',values='order_number',aggfunc='count')   
pd.pivot_table(df_lbs_data[df_lbs_data.search_time == '2024-11'],index='live_level_score',columns='y_fraud_FPD',values='order_number',aggfunc='count')  


y_fraud_FPD,0.0,0.5,1.0
search_time,,,
2024-09,2383.0,115.0,101.0
2024-10,1497.0,64.0,46.0
2024-11,2333.0,102.0,67.0
2024-12,3113.0,67.0,14.0
2025-01,1055.0,NaN,NaN


y_fraud_FPD,0.0,0.5,1.0
search_time,,,
2024-09,2383,115,101
2024-10,1497,64,46
2024-11,1062,57,32


y_fraud_FPD,0,1
work_distance_score_Bin,,
"1.[0,39]",1.0,NaN
"2.[40,59]",1.0,1.0
"3.[60,74]",1.0,1.0
"4.[75,89]",6.0,1.0
"5.[90,100]",6.0,8.0
"6.[-1111,-1111]",18.0,11.0
"7.[-999,-999]",123.0,77.0


y_fraud_FPD,0,1
work_distance_score_Bin,,
"2.[40,59]",1.0,NaN
"3.[60,74]",2.0,NaN
"4.[75,89]",1.0,4.0
"5.[90,100]",2.0,1.0
"6.[-1111,-1111]",19.0,11.0
"7.[-999,-999]",73.0,30.0


y_fraud_FPD,0,1
work_distance_score_Bin,,
"1.[0,39]",1.0,NaN
"2.[40,59]",1.0,NaN
"3.[60,74]",1.0,1.0
"4.[75,89]",3.0,1.0
"5.[90,100]",3.0,2.0
"6.[-1111,-1111]",9.0,3.0
"7.[-999,-999]",52.0,23.0


...

y_fraud_FPD,0,1
work_level_score,,
-999.0,63.0,26.0
1.0,2.0,1.0
2.0,2.0,3.0
3.0,5.0,1.0
4.0,2.0,2.0
5.0,4.0,1.0
6.0,4.0,2.0
7.0,3.0,3.0
8.0,NaN,2.0


y_fraud_FPD,0,1
work_level_score,,
-999.0,47.0,22.0
1.0,2.0,1.0
2.0,1.0,NaN
3.0,5.0,NaN
4.0,4.0,1.0
5.0,2.0,NaN
6.0,3.0,NaN
7.0,3.0,1.0
9.0,1.0,NaN


y_fraud_FPD,0,1
live_level_score,,
-999.0,155.0,99.0
2.0,1.0,NaN


y_fraud_FPD,0,1
live_level_score,,
-999.0,94.0,45.0
1.0,1.0,1.0
3.0,1.0,NaN
4.0,1.0,NaN
12.0,1.0,NaN


y_fraud_FPD,0,1
live_level_score,,
-999.0,70.0,28.0
1.0,NaN,1.0
4.0,NaN,1.0


# 7_百融

## 7_1.验证流程策略_001

In [ ]:
# （1）取数
# db_credit.risk_bairong：为百融的大表，含信息验证接口和贷前查询接口，先查信息验证，再查贷前查询；若没有拆出来的产品，都存这张表（如云商分单独存表）
sql1='''-- 风控-百融 
SELECT trace_id
      , verify_rs, verify_time
from db_credit.risk_bairong
;
'''

br_raf_df =query(sql1)
# br_raf_df['ran_score'] = br_raf_df['query_rs'].apply(lambda x: json.loads(x).get('RiskStrategy', {}).get('Score', {}).get('scoreprime', None))  
# br_raf_df['ran_score'] = pd.to_numeric(br_raf_df['ran_score'].replace('"null"', np.nan), errors='coerce')

br_raf_df['search_time'] = br_raf_df['verify_time'].dt.strftime('%Y-%m')

## 7_2.借贷风险勘测回溯

### 7_2_1.2025.4

In [ ]:
# （1）样本挑选
# 身份证或手机号最近申请距今天数>0
# 本次提供2024.7-2025.3.27的出库样本
# 2025.3.28接口切换

# 8_智安盾

## 2025.4

In [159]:
# （1）样本挑选
# 只支持回溯 24 年 10 月后的订单，每个商户只能回溯最多 1 万订单，订单必须为信用单
# 本次提供 2024.11.1-2025.3.27 的样本测试,2024.11-2024.12出库全量，2025.1-2025.3.27随机抽样，共1W单
# 2025.3.28正式上线
# appid	用户 uid	外部订单号	下单时间	商品名称
# （1-1）取 trace_id
sql1='''-- trace关联表 
SELECT trace_id, order_number
from db_credit.order_association
;
'''

trace_df =query(sql1)
trace_df.shape

# （1-2） out_order_no
sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT trace_id,out_order_no
from db_credit.risk_alipay_auth_query
;
'''
zm_fk_df = query(sql1)
zm_fk_df.shape

# （1-3）appid、payer_user_id
# `payer_user_id` varchar(255) CHARACTER SET utf8mb4 COLLATE utf8mb4_bin DEFAULT NULL COMMENT '支付宝用户id',
# `item_type` int DEFAULT '1' COMMENT '支付类型1，下单，2续租',
# `status` int DEFAULT NULL COMMENT '状态，1、待支付，2、已支付、3、退款中 4、已退款 5、已解约',


# sql = '''-- 租后应收监控  
# SELECT  om.order_number, om.appid, top.payer_user_id
# ,tod.product_name
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# left join db_digua_business.t_order om on tprm.order_id = om.id
# left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
# left join db_digua_business.t_order_pay top on tprm.order_id = top.order_id

# where top.pay_type = 'ZFBYSQ' and top.item_type = 1 and (top.status = 2 or top.status = 5)
# '''

# df_info_1 = query(sql)
# df_info_1.shape

sql = '''-- 租后应收监控  
SELECT  tprm.order_id, om.order_number, om.appid
,tod.product_name
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
'''

df_info_1 = query(sql)
df_info_1.shape

sql = '''
SELECT  top.order_id, top.payer_user_id, top.status
from db_digua_business.t_order_pay top
where top.pay_type = 'ZFBYSQ' and top.item_type = '1'
'''

df_info_2 = query(sql)
df_info_2.shape
df_info_2 = df_info_2[df_info_2.status.isin([2,5])]
df_info_2.shape

# df_info_3：order_number 或 order_id 重复
df_info_3 = df_info_1.merge(df_info_2[['order_id','payer_user_id']], on = 'order_id', how = 'left')
df_info_3.shape
# df_info_3[:3]
# df_info_3.drop_duplicates(subset= 'order_number').shape

del df_info_1
del df_info_2


# （1-4）表关联
trace_df = trace_df.merge(zm_fk_df, on = 'trace_id', how = 'left')
trace_df.shape

trace_df = trace_df.merge(df_info_3[['order_number','appid','payer_user_id','product_name']] ,on = 'order_number', how = 'left')
trace_df.shape

df_zad_1 = df_analy_data[['order_number','下单日期']].merge(trace_df[['order_number', 'trace_id', 'out_order_no','appid','payer_user_id','product_name']], on = 'order_number', how = 'left')
df_zad_1.shape
df_zad_1 = df_zad_1[(df_zad_1.下单日期 >= '2024-11-01') & ((df_zad_1.下单日期 <= '2025-03-27'))]
df_zad_1.shape
# out_order_no 为空的 无意义，out_order_no 重复的，客户取消了再下单
# df_zad_1 = df_zad_1[~(df_zad_1.trace_id.isnull())]
df_zad_1 = df_zad_1[~(df_zad_1.out_order_no.isnull())]
df_zad_1 = df_zad_1.drop_duplicates(subset = 'out_order_no')
df_zad_1.shape

tmp1 = df_zad_1[(df_zad_1.下单日期 >= '2024-11-01') & ((df_zad_1.下单日期 <= '2024-12-31'))]
tmp2 = df_zad_1[(df_zad_1.下单日期 >= '2025-01-01')]
tmp1.shape
tmp2.shape
tmp2 = tmp2.sample(n = 4442)

df_zad_1 = pd.concat([tmp1, tmp2])
df_zad_1 = df_zad_1.rename(columns = {'payer_user_id':'用户uid', 'out_order_no':'外部订单号', '下单日期':'下单时间', 'product_name':'商品名称'})
df_zad_1['下单时间'] = df_zad_1['下单时间'].astype(str)
df_zad_1.shape
df_zad_1[:1]

# t_member_user_openid 表的 appid 不用管
# sql = '''-- 租后应收监控  
# SELECT  om.order_number, om.appid, tmuo.openid
# ,tod.product_name
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# left join db_digua_business.t_order om on tprm.order_id = om.id
# left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
# left join db_digua_business.t_member_user_openid tmuo on om.user_id = tmuo.user_id
# where tmuo.openid_type = 3
# ;
# '''

# df_info = query(sql)
# df_info.shape

# df_zad_1 = df_zad_1.merge(df_info, on = 'order_number', how = 'left')
# df_zad_1.shape

del trace_df
del zm_fk_df
del df_info_3

# path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/4_智安盾/2025.4/1_发送/'
# df_zad_1.to_excel(path + 'ceshi_sample_zad_20250403.xlsx', index = False)

(1883199, 2)

(234842, 2)

(39084, 4)

(1439813, 3)

(346598, 3)

...

(11432, 7)

(5558, 7)

(5874, 7)

(10000, 7)

,order_number,下单时间,trace_id,外部订单号,appid,用户uid,商品名称
24146,A2024110109215352,2024-11-01,aa0e15bad1b267151bdf27ae2251aa0a,A2024110109215352,2021003131606511,2088422251502733,iPhone 16 Pro Max 全新未激活 国行正品 寄出包邮


In [ ]:
# （2）返回结果分析
sql1='''-- trace关联表 
SELECT trace_id, order_number
from db_credit.order_association
;
'''

trace_df =query(sql1)
trace_df.shape

# （1-2） out_order_no
sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT trace_id,out_order_no
from db_credit.risk_alipay_auth_query
;
'''
zm_fk_df = query(sql1)
zm_fk_df.shape

# （3）表关联
trace_df = trace_df.merge(zm_fk_df, on = 'trace_id', how = 'left')
trace_df.shape

# out_order_no和返回数据匹配

# 9_三方数据

## 9_1乐吧

In [67]:
df_analy_data

,order_number,order_id,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数
0,A202207050851373,8924,2022-07-05,2022-07,未知渠道,二手,未带锁,4,0.0,0.0,4.0,0.0,0.0,4.0
1,A202207131230485,9305,2022-07-13,2022-07,未知渠道,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0
2,A202207151707153,9514,2022-07-15,2022-07,我的小程序入口,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0
3,A202207162356303,9600,2022-07-16,2022-07,其他渠道场景渠道。,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0
4,A202207171820512,9683,2022-07-17,2022-07,其他渠道场景渠道。,全新,未带锁,27,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37588,A20250410184924245,2385056,2025-04-10,2025-04,搜索渠道,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0
37589,A20250410185512263,2385074,2025-04-10,2025-04,搜索渠道,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0
37590,A2025041019241975,2385161,2025-04-10,2025-04,搜索渠道,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0
37591,A20250410193144104,2385190,2025-04-10,2025-04,芝麻租物,全新,未带锁,0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
sql = '''-- 租后应收监控  
SELECT  om.id as order_id, om.order_number, om.create_time, om.merchant_name
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
, tmu.true_name, tmu.id_card_num, tmu.mobile 
, tod.sku_attributes
, top.total_describes
, pa.name as activity_name        -- 活动名称
,topay.total_freeze_fund_amount
from db_digua_business.t_order om
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on om.id = tod.order_id
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
;
'''
df_info = query(sql)
df_info["下单日期"] = df_info['create_time'].dt.date
df_info["下单日期"] = pd.to_datetime(df_info["下单日期"],errors="coerce")
df_info.loc[:,"进件"]=np.where((df_info["订单状态"]=="待支付")|(df_info["订单状态"]=="订单取消"),"未进件","进件")
df_info.loc[:,"押金类型"]=np.where(df_info["total_freeze_fund_amount"]>0,"部分免押","全免押")
df_info.shape


(2333319, 15)

In [73]:

df_info_04 = df_info[(df_info.下单日期>='2025-03-01')&(df_info.下单日期<='2025-03-31')&(df_info.进件=='进件')&(df_info.押金类型=='全免押')]
df_info_04[df_info_04.order_number=='A20250331234348112']

,order_id,order_number,create_time,merchant_name,订单状态,true_name,id_card_num,mobile,sku_attributes,total_describes,activity_name,total_freeze_fund_amount,下单日期,进件,押金类型
2273644,2328751,A20250331234348112,2025-03-31 23:43:49,维客自营账户,已退款,冯露,230622200701071558,15245939769,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...","[""系统操作审核订单A20250331234348112，出库前风控审核结果：不通过""]",端内价格-全新苹果,0.0,2025-03-31,进件,全免押


In [81]:
# （7-2） 关联目标客户三要素
df_analy_data_m = df_analy_data.merge(df_info[['order_number','true_name','id_card_num','mobile']], on = 'order_number', how = 'left')
df_analy_data_m.shape

df_ck_ = df_analy_data_m[(df_analy_data_m.search_time >= '2024-01') & (df_analy_data_m.search_time <= '2025-02')]
df_ck_.shape
df_ck_[df_ck_.id_card_num.isna()].shape
df_ck_[df_ck_.mobile.isna()].shape

df_ck_ = df_ck_[df_ck_.id_card_num.notnull()]
df_ck_.shape

# 免押进件 -- 2025-03
df_info_03 = df_info[(df_info.下单日期>='2025-03-01')&(df_info.下单日期<='2025-03-31')&(df_info.进件=='进件')&(df_info.押金类型=='全免押')]
df_info_03_merge = df_info_03[['下单日期', 'order_number','true_name','id_card_num','mobile']]#.merge(df_analy_data, on='order_number', how='left')
df_info_03_merge = df_info_03_merge[df_info_03_merge.id_card_num.notnull()]
df_info_03_merge = df_info_03_merge[df_info_03_merge.mobile.notnull()]
df_info_03_merge.shape
# 拼接出库和免押进件订单
df_concat = pd.concat([df_ck_[['下单日期', 'order_number','true_name','id_card_num','mobile']], df_info_03_merge[['下单日期', 'order_number','true_name','id_card_num','mobile']]])
df_concat.shape


# （7-3）md5加密
import hashlib

def md5_encryption(var):
    md5 = hashlib.md5()  # 创建一个md5对象
    md5.update(var.encode('utf-8'))  # 使用utf-8编码数据
    return md5.hexdigest()  # 返回加密后的十六进制字符串
# df_202307_['下单日期_md5'] = df_202307_.apply(lambda x: md5_encryption(x.下单日期), axis = 1)
df_concat['user_name_md5'] = df_concat.apply(lambda x: md5_encryption(x.true_name), axis = 1)
df_concat['idcard_md5'] = df_concat.apply(lambda x: md5_encryption(x.id_card_num), axis = 1)
df_concat['mobile_md5'] = df_concat.apply(lambda x: md5_encryption(x.mobile), axis = 1)
df_concat.shape
df_concat[['order_number', '下单日期', 'user_name_md5', 'idcard_md5', 'mobile_md5']].tail()
df_concat_new = df_concat[['order_number', '下单日期', 'user_name_md5', 'idcard_md5', 'mobile_md5']].rename(columns={'order_number': '订单号', 'user_name_md5': '姓名', 'idcard_md5': '身份证号', 'mobile_md5': '手机号'})
with pd.ExcelWriter('F:/需求/总需求/乐吧数据.xlsx', engine='xlsxwriter') as writer:
    df_concat_new.to_excel(writer, sheet_name='免押进件出库数据样本', index=False)

(37593, 17)

(24511, 17)

(1, 17)

(0, 17)

(24510, 17)

(22107, 5)

(46617, 5)

(46617, 8)

,order_number,下单日期,user_name_md5,idcard_md5,mobile_md5
2273644,A20250331234348112,2025-03-31,cf055755186379b0ebfbea42daeb4b7e,960761a484e68453774846c4e611ba1e,290c42b76242f54d4a964869ed20c4b5
2273648,A20250331234500116,2025-03-31,ff642b7559e1d37604857b5816525217,acc038058b3c8e3e69bd0fe58ba4808e,ae68c9997fa9d3de302502571210f8f4
2273649,A20250331234502117,2025-03-31,f06a6cb6e2c797124cb441318f323289,06bd7042385c912aee168686c8c4cfaa,32d6ddce0d7097afe67b9b553700f60d
2273669,A20250331235111137,2025-03-31,a66574ba3a1dcb766e7ecac14dcfab41,7b8ca638e73f15f217f1d99ce1e42733,906f447023484204a5066f02b7f4386a
2273676,A20250331235445144,2025-03-31,bebec7c431c9c4237026d824fb42dae1,3187c7220a00119f0bf8e25a74fee233,80cbfaac0bef3ec083b6f134c58fbe5d


In [80]:
df_info_03_merge[df_info_03_merge.order_number=='A20250331234348112']

,下单日期,order_number,true_name,id_card_num,mobile
2273644,2025-03-31,A20250331234348112,冯露,230622200701071558,15245939769
